#### ZRS - Calculadora Came
____

> Carlos Alvarez (c.alvarez@hocelot.com)

> Junio 2023

# 0. Import packages

In [1]:
import os
import pickle
import joblib
import re
import string
import subprocess
import random
import numpy as np
import warnings
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime,timedelta


import lightgbm
from xgboost import XGBClassifier
from multiprocessing import cpu_count
from sklearn.feature_selection import RFE
from sklearn.metrics import roc_auc_score, roc_curve

from hocelot.dlutils import get_col_feats, read_raw_data
from hocelot.dlutils import open as open_dlutils # open for saving PKL
#from hocelot.dlutils.miscellaneous import RANDOM_SEED, hocelot_colors
from hocelot.rdt import RDTransformer
from hocelot.dlmodels import XGBClassifierSelector
from hocelot.dlplots import ModelResultsGrapher, ModelExplainabilityGrapher, DefaultGrapher
#from hocelot.dlplots.classification.general_plots import plot_target_stability
from hocelot.dlmetadata import HOCELOT_DTYPES, HOCELOT_FTYPES, LOG_TRANSFORMABLE_1

from hocelot.dlutils import open

plt.style.use('ggplot')
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 100)
pd.set_option('display.max_rows', 100)

HOCELOT_YELLOW = '#FDD226'
HOCELOT_BLACK = '#000000'
HOCELOT_RED = '#FF5753'
HOCELOT_BLUE = '#2B3252'
HOCELOT_WHITE = '#FFFFFF'

%matplotlib inline
warnings.filterwarnings('ignore')
import ast

# 1. Preparación del Dataset

### Tratamiento de la tabla original post QA0

In [2]:
data_original =pd.read_csv('gs://hocelot_mx_analytics/mexico/progress/mx-00007-Came/data_original_juntada.csv', sep=';', index_col=False)
print(data_original.shape)
data_original.head()

(300903, 31)


,NoCredito,NoCliente,Nombre,FechaNacimiento,CURP,Calle,NumExt,Colonia,Ciudad,Estado,Email,Telefono,EstadoCivil,TipoProducto,Monto,Plazo,Tasa,FechaOtorgamiento,FechaCierre,CuotaTotal,Frecuencia,Calificacion,di_fecha_ini,di_fecha_ven,dividendo,Cuota,Impago,Pagado,Impagado,Recuperado,Fecha Ultimo Pago
0,1920492,35447,BENITO RUIZ RAMOS,1965-12-23,RURB651223HNLZMN06,LA MADRID,501,RAMON BRAVO,PIEDRAS NEGRAS,COAHUILA,benitoruiz522@gmail.com,8787004573,SOLTERO,Crédito Micro-Empresario,22440.0,12,114.0,2023-03-08,NaN,3460.0,Mensual,"Aprobado, verde",2023-03-07,2023-04-10,1,3460.00,NO,3460.00,0.00,0.00,2023-04-10
1,1920492,35447,BENITO RUIZ RAMOS,1965-12-23,RURB651223HNLZMN06,LA MADRID,501,RAMON BRAVO,PIEDRAS NEGRAS,COAHUILA,benitoruiz522@gmail.com,8787004573,SOLTERO,Crédito Micro-Empresario,22440.0,12,114.0,2023-03-08,NaN,3460.0,Mensual,"Aprobado, verde",2023-04-11,2023-05-08,2,3491.95,SI,3491.95,3451.95,3451.95,2023-05-16
2,1920492,35447,BENITO RUIZ RAMOS,1965-12-23,RURB651223HNLZMN06,LA MADRID,501,RAMON BRAVO,PIEDRAS NEGRAS,COAHUILA,benitoruiz522@gmail.com,8787004573,SOLTERO,Crédito Micro-Empresario,22440.0,12,114.0,2023-03-08,NaN,3460.0,Mensual,"Aprobado, verde",2023-05-09,2023-06-08,3,3491.75,NaN,0.00,0.00,0.00,NaN
3,1920492,35447,BENITO RUIZ RAMOS,1965-12-23,RURB651223HNLZMN06,LA MADRID,501,RAMON BRAVO,PIEDRAS NEGRAS,COAHUILA,benitoruiz522@gmail.com,8787004573,SOLTERO,Crédito Micro-Empresario,22440.0,12,114.0,2023-03-08,NaN,3460.0,Mensual,"Aprobado, verde",2023-06-09,2023-07-10,4,3460.00,NaN,0.00,0.00,0.00,NaN
4,1920492,35447,BENITO RUIZ RAMOS,1965-12-23,RURB651223HNLZMN06,LA MADRID,501,RAMON BRAVO,PIEDRAS NEGRAS,COAHUILA,benitoruiz522@gmail.com,8787004573,SOLTERO,Crédito Micro-Empresario,22440.0,12,114.0,2023-03-08,NaN,3460.0,Mensual,"Aprobado, verde",2023-07-11,2023-08-08,5,3460.00,NaN,0.00,0.00,0.00,NaN


In [3]:
data_fact = pd.read_csv('gs://hocelot_mx_analytics/mexico/progress/mx-00007-Came/data_Came_post_QA0_v0.csv', sep=';', index_col=False)
print(data_fact.shape)
data_fact.head(2)

(54717, 40)


,NoCredito,NoCliente,Nombre,FechaNacimiento,CURP,Calle,NumExt,Colonia,Ciudad,Estado,Email,Telefono,EstadoCivil,Monto,Plazo,Tasa,FechaOtorgamiento,FechaCierre,CuotaTotal,Frecuencia,Calificacion,di_fecha_ini,di_fecha_ven,dividendo,Cuota,Impago,Pagado,Impagado,Recuperado,Fecha Ultimo Pago,Años_pide_cred,Mes_Emision,fecha_recuperacion,fecha_recuperacion_aux,dias_en_default,Default_1_dias,Default_90_dias,Default_90_dias_cont,Default_1_dias_cont,Default_90_dias_pers
0,39792683,48993,VIANNEY CHABLE MORA,2002-08-12,CAMV020812MMCHRNA2,MANUEL ALTAMIRANO,2,ARTESANOS,ESTADO DE MEXICO,ESTADO DE MEXICO,vianneychable001@gmail.com,5588825117,UNIÓN LIBRE,21600.0,12,106.8,2022-09-30,NaN,3230.41,Mensual,"Aprobado, amarillo",2022-09-30,2022-10-24,1,3231.69,SI,3231.69,3231.69,3231.69,2022-10-29,20,2022-10,2022-10-29,2022-10-29,5,1,0,1,1,1
1,39792683,48993,VIANNEY CHABLE MORA,2002-08-12,CAMV020812MMCHRNA2,MANUEL ALTAMIRANO,2,ARTESANOS,ESTADO DE MEXICO,ESTADO DE MEXICO,vianneychable001@gmail.com,5588825117,UNIÓN LIBRE,21600.0,12,106.8,2022-09-30,NaN,3230.41,Mensual,"Aprobado, amarillo",2022-10-25,2022-11-22,2,3360.34,SI,3360.34,129.34,129.34,2022-12-22,20,2022-11,2022-12-22,2022-12-22,30,1,0,1,1,1


In [4]:
df_predicted_test = pd.read_csv('gs://hocelot_mx_analytics/mexico/output/mx-00007-Came/model_predictions.csv', sep = ',')
print(df_predicted_test.shape)
df_predicted_test.head(2)

(13262, 6)


,Unnamed: 0,pk,proba,real_value,NoCliente,Default_90_dias_pers
0,0,2463,0.519277,1,187769220,1
1,1,11575,0.483736,0,868759223,0


In [5]:
df_predicted_test[df_predicted_test['real_value']!=df_predicted_test['Default_90_dias_pers']]

,Unnamed: 0,pk,proba,real_value,NoCliente,Default_90_dias_pers


In [6]:
df_predicted_test.drop(columns=['Unnamed: 0','Default_90_dias_pers'], inplace=True)

In [7]:
data_fact_con_score = df_predicted_test.merge(data_fact, how='inner', on='NoCliente')
print(data_fact_con_score.shape)
data_fact_con_score.head()

(54729, 43)


,pk,proba,real_value,NoCliente,NoCredito,Nombre,FechaNacimiento,CURP,Calle,NumExt,Colonia,Ciudad,Estado,Email,Telefono,EstadoCivil,Monto,Plazo,Tasa,FechaOtorgamiento,FechaCierre,CuotaTotal,Frecuencia,Calificacion,di_fecha_ini,di_fecha_ven,dividendo,Cuota,Impago,Pagado,Impagado,Recuperado,Fecha Ultimo Pago,Años_pide_cred,Mes_Emision,fecha_recuperacion,fecha_recuperacion_aux,dias_en_default,Default_1_dias,Default_90_dias,Default_90_dias_cont,Default_1_dias_cont,Default_90_dias_pers
0,2463,0.519277,1,187769220,238891837,GRISELDA GUTIERREZ CADENA,1993-09-09,GUCG930909MDFTDR05,TENANITLA,18,AMPLIACIÓN POTRERILLO,LA MAGDALENA CONTRERAS,CIUDAD DE MÉXICO,taconwine@gmail.com,5573803423,UNIÓN LIBRE,10600.0,12,120.0,2022-05-09,NaN,2063.49,Mensual,NaN,2022-05-09,2022-06-09,1,2116.26,SI,0.0,2116.26,0.0,NaN,28,2022-06,NaN,2023-05-31,356,1,1,1,1,1
1,2463,0.519277,1,187769220,238891837,GRISELDA GUTIERREZ CADENA,1993-09-09,GUCG930909MDFTDR05,TENANITLA,18,AMPLIACIÓN POTRERILLO,LA MAGDALENA CONTRERAS,CIUDAD DE MÉXICO,taconwine@gmail.com,5573803423,UNIÓN LIBRE,10600.0,12,120.0,2022-05-09,NaN,2063.49,Mensual,NaN,2022-06-10,2022-07-11,2,2009.23,SI,0.0,2009.23,0.0,NaN,28,2022-07,NaN,2023-05-31,324,1,1,1,1,1
2,2463,0.519277,1,187769220,238891837,GRISELDA GUTIERREZ CADENA,1993-09-09,GUCG930909MDFTDR05,TENANITLA,18,AMPLIACIÓN POTRERILLO,LA MAGDALENA CONTRERAS,CIUDAD DE MÉXICO,taconwine@gmail.com,5573803423,UNIÓN LIBRE,10600.0,12,120.0,2022-05-09,NaN,2063.49,Mensual,NaN,2022-07-12,2022-08-09,3,2035.11,SI,0.0,2035.11,0.0,NaN,28,2022-08,NaN,2023-05-31,295,1,1,1,1,1
3,2463,0.519277,1,187769220,238891837,GRISELDA GUTIERREZ CADENA,1993-09-09,GUCG930909MDFTDR05,TENANITLA,18,AMPLIACIÓN POTRERILLO,LA MAGDALENA CONTRERAS,CIUDAD DE MÉXICO,taconwine@gmail.com,5573803423,UNIÓN LIBRE,10600.0,12,120.0,2022-05-09,NaN,2063.49,Mensual,NaN,2022-08-10,2022-09-09,4,1974.61,SI,0.0,1974.61,0.0,NaN,28,2022-09,NaN,2023-05-31,264,1,1,1,1,1
4,2463,0.519277,1,187769220,238891837,GRISELDA GUTIERREZ CADENA,1993-09-09,GUCG930909MDFTDR05,TENANITLA,18,AMPLIACIÓN POTRERILLO,LA MAGDALENA CONTRERAS,CIUDAD DE MÉXICO,taconwine@gmail.com,5573803423,UNIÓN LIBRE,10600.0,12,120.0,2022-05-09,NaN,2063.49,Mensual,NaN,2022-09-10,2022-10-10,5,1680.00,SI,0.0,1680.00,0.0,NaN,28,2022-10,NaN,2023-05-31,233,1,1,1,1,1


In [8]:
data_fact_con_score[data_fact_con_score['real_value']!=data_fact_con_score['Default_90_dias_pers']]

,pk,proba,real_value,NoCliente,NoCredito,Nombre,FechaNacimiento,CURP,Calle,NumExt,Colonia,Ciudad,Estado,Email,Telefono,EstadoCivil,Monto,Plazo,Tasa,FechaOtorgamiento,FechaCierre,CuotaTotal,Frecuencia,Calificacion,di_fecha_ini,di_fecha_ven,dividendo,Cuota,Impago,Pagado,Impagado,Recuperado,Fecha Ultimo Pago,Años_pide_cred,Mes_Emision,fecha_recuperacion,fecha_recuperacion_aux,dias_en_default,Default_1_dias,Default_90_dias,Default_90_dias_cont,Default_1_dias_cont,Default_90_dias_pers


# CALCULO EL IMPORTE MOROSO, EL RECUPERADO MOROSO Y LAS FECHAS

In [9]:
data_fact_con_score[0:2]

,pk,proba,real_value,NoCliente,NoCredito,Nombre,FechaNacimiento,CURP,Calle,NumExt,Colonia,Ciudad,Estado,Email,Telefono,EstadoCivil,Monto,Plazo,Tasa,FechaOtorgamiento,FechaCierre,CuotaTotal,Frecuencia,Calificacion,di_fecha_ini,di_fecha_ven,dividendo,Cuota,Impago,Pagado,Impagado,Recuperado,Fecha Ultimo Pago,Años_pide_cred,Mes_Emision,fecha_recuperacion,fecha_recuperacion_aux,dias_en_default,Default_1_dias,Default_90_dias,Default_90_dias_cont,Default_1_dias_cont,Default_90_dias_pers
0,2463,0.519277,1,187769220,238891837,GRISELDA GUTIERREZ CADENA,1993-09-09,GUCG930909MDFTDR05,TENANITLA,18,AMPLIACIÓN POTRERILLO,LA MAGDALENA CONTRERAS,CIUDAD DE MÉXICO,taconwine@gmail.com,5573803423,UNIÓN LIBRE,10600.0,12,120.0,2022-05-09,NaN,2063.49,Mensual,NaN,2022-05-09,2022-06-09,1,2116.26,SI,0.0,2116.26,0.0,NaN,28,2022-06,NaN,2023-05-31,356,1,1,1,1,1
1,2463,0.519277,1,187769220,238891837,GRISELDA GUTIERREZ CADENA,1993-09-09,GUCG930909MDFTDR05,TENANITLA,18,AMPLIACIÓN POTRERILLO,LA MAGDALENA CONTRERAS,CIUDAD DE MÉXICO,taconwine@gmail.com,5573803423,UNIÓN LIBRE,10600.0,12,120.0,2022-05-09,NaN,2063.49,Mensual,NaN,2022-06-10,2022-07-11,2,2009.23,SI,0.0,2009.23,0.0,NaN,28,2022-07,NaN,2023-05-31,324,1,1,1,1,1


In [10]:
data_fact_con_score['periodo_impagado']=np.where(data_fact_con_score['Default_1_dias']==1, data_fact_con_score['di_fecha_ven'], '')
data_fact_con_score['importe_moroso'] = np.where(data_fact_con_score['Default_90_dias']>0, data_fact_con_score['Impagado'], 0)
data_fact_con_score['periodo_moroso']=np.where(data_fact_con_score['Default_90_dias']>0, data_fact_con_score['di_fecha_ven'], '')
data_fact_con_score['importe_recuperado_moroso'] = np.where(data_fact_con_score['Default_90_dias']>0, data_fact_con_score['Recuperado'], 0)
data_fact_con_score['periodo_recuperado_moroso'] = np.where(data_fact_con_score['Default_90_dias']>0, data_fact_con_score['fecha_recuperacion'], '')

# Ticket MEDIO

Para los ultimos 4 meses

In [11]:
data_original.head(2)

,NoCredito,NoCliente,Nombre,FechaNacimiento,CURP,Calle,NumExt,Colonia,Ciudad,Estado,Email,Telefono,EstadoCivil,TipoProducto,Monto,Plazo,Tasa,FechaOtorgamiento,FechaCierre,CuotaTotal,Frecuencia,Calificacion,di_fecha_ini,di_fecha_ven,dividendo,Cuota,Impago,Pagado,Impagado,Recuperado,Fecha Ultimo Pago
0,1920492,35447,BENITO RUIZ RAMOS,1965-12-23,RURB651223HNLZMN06,LA MADRID,501,RAMON BRAVO,PIEDRAS NEGRAS,COAHUILA,benitoruiz522@gmail.com,8787004573,SOLTERO,Crédito Micro-Empresario,22440.0,12,114.0,2023-03-08,NaN,3460.0,Mensual,"Aprobado, verde",2023-03-07,2023-04-10,1,3460.00,NO,3460.00,0.00,0.00,2023-04-10
1,1920492,35447,BENITO RUIZ RAMOS,1965-12-23,RURB651223HNLZMN06,LA MADRID,501,RAMON BRAVO,PIEDRAS NEGRAS,COAHUILA,benitoruiz522@gmail.com,8787004573,SOLTERO,Crédito Micro-Empresario,22440.0,12,114.0,2023-03-08,NaN,3460.0,Mensual,"Aprobado, verde",2023-04-11,2023-05-08,2,3491.95,SI,3491.95,3451.95,3451.95,2023-05-16


In [12]:
df_ticket_medio = data_original[(data_original['di_fecha_ven']<='2023-05-28')&(data_original['di_fecha_ven']>='2023-02-28')]
df_ticket_medio.head()

,NoCredito,NoCliente,Nombre,FechaNacimiento,CURP,Calle,NumExt,Colonia,Ciudad,Estado,Email,Telefono,EstadoCivil,TipoProducto,Monto,Plazo,Tasa,FechaOtorgamiento,FechaCierre,CuotaTotal,Frecuencia,Calificacion,di_fecha_ini,di_fecha_ven,dividendo,Cuota,Impago,Pagado,Impagado,Recuperado,Fecha Ultimo Pago
0,1920492,35447,BENITO RUIZ RAMOS,1965-12-23,RURB651223HNLZMN06,LA MADRID,501,RAMON BRAVO,PIEDRAS NEGRAS,COAHUILA,benitoruiz522@gmail.com,8787004573,SOLTERO,Crédito Micro-Empresario,22440.0,12,114.0,2023-03-08,NaN,3460.00,Mensual,"Aprobado, verde",2023-03-07,2023-04-10,1,3460.00,NO,3460.00,0.00,0.00,2023-04-10
1,1920492,35447,BENITO RUIZ RAMOS,1965-12-23,RURB651223HNLZMN06,LA MADRID,501,RAMON BRAVO,PIEDRAS NEGRAS,COAHUILA,benitoruiz522@gmail.com,8787004573,SOLTERO,Crédito Micro-Empresario,22440.0,12,114.0,2023-03-08,NaN,3460.00,Mensual,"Aprobado, verde",2023-04-11,2023-05-08,2,3491.95,SI,3491.95,3451.95,3451.95,2023-05-16
12,1994129,41771,ARELY GARCIA MENDEZ,1991-03-29,GAMA910329MCSRNR07,CALLE SIN NOMBRE,sn,ESTRRELLA ROJA,SOCOLTENANGO,CHIAPAS,eg145972@gmail.com,9921435866,SOLTERO,Crédito Micro-Empresario,12700.0,10,120.0,2023-03-17,NaN,2216.48,Mensual,"Aprobado, verde",2023-03-17,2023-04-17,1,2217.10,SI,2217.10,2216.19,2216.19,2023-04-19
13,1994129,41771,ARELY GARCIA MENDEZ,1991-03-29,GAMA910329MCSRNR07,CALLE SIN NOMBRE,sn,ESTRRELLA ROJA,SOCOLTENANGO,CHIAPAS,eg145972@gmail.com,9921435866,SOLTERO,Crédito Micro-Empresario,12700.0,10,120.0,2023-03-17,NaN,2216.48,Mensual,"Aprobado, verde",2023-04-18,2023-05-17,2,2220.37,SI,2220.37,2140.74,2140.74,2023-05-18
27,39792683,48993,VIANNEY CHABLE MORA,2002-08-12,CAMV020812MMCHRNA2,MANUEL ALTAMIRANO,2,ARTESANOS,ESTADO DE MEXICO,ESTADO DE MEXICO,vianneychable001@gmail.com,5588825117,UNIÓN LIBRE,Crédito Micro-Empresario,21600.0,12,106.8,2022-09-30,NaN,3230.41,Mensual,"Aprobado, amarillo",2023-02-23,2023-03-22,6,3906.51,SI,0.00,3906.51,0.00,NaN


In [13]:
df_ticket_medio['Cuota'].mean()

4607.498033251595

# Importe del credito medio

In [14]:
data_fact_con_score.head()

,pk,proba,real_value,NoCliente,NoCredito,Nombre,FechaNacimiento,CURP,Calle,NumExt,Colonia,Ciudad,Estado,Email,Telefono,EstadoCivil,Monto,Plazo,Tasa,FechaOtorgamiento,FechaCierre,CuotaTotal,Frecuencia,Calificacion,di_fecha_ini,di_fecha_ven,dividendo,Cuota,Impago,Pagado,Impagado,Recuperado,Fecha Ultimo Pago,Años_pide_cred,Mes_Emision,fecha_recuperacion,fecha_recuperacion_aux,dias_en_default,Default_1_dias,Default_90_dias,Default_90_dias_cont,Default_1_dias_cont,Default_90_dias_pers,periodo_impagado,importe_moroso,periodo_moroso,importe_recuperado_moroso,periodo_recuperado_moroso
0,2463,0.519277,1,187769220,238891837,GRISELDA GUTIERREZ CADENA,1993-09-09,GUCG930909MDFTDR05,TENANITLA,18,AMPLIACIÓN POTRERILLO,LA MAGDALENA CONTRERAS,CIUDAD DE MÉXICO,taconwine@gmail.com,5573803423,UNIÓN LIBRE,10600.0,12,120.0,2022-05-09,NaN,2063.49,Mensual,NaN,2022-05-09,2022-06-09,1,2116.26,SI,0.0,2116.26,0.0,NaN,28,2022-06,NaN,2023-05-31,356,1,1,1,1,1,2022-06-09,2116.26,2022-06-09,0.0,NaN
1,2463,0.519277,1,187769220,238891837,GRISELDA GUTIERREZ CADENA,1993-09-09,GUCG930909MDFTDR05,TENANITLA,18,AMPLIACIÓN POTRERILLO,LA MAGDALENA CONTRERAS,CIUDAD DE MÉXICO,taconwine@gmail.com,5573803423,UNIÓN LIBRE,10600.0,12,120.0,2022-05-09,NaN,2063.49,Mensual,NaN,2022-06-10,2022-07-11,2,2009.23,SI,0.0,2009.23,0.0,NaN,28,2022-07,NaN,2023-05-31,324,1,1,1,1,1,2022-07-11,2009.23,2022-07-11,0.0,NaN
2,2463,0.519277,1,187769220,238891837,GRISELDA GUTIERREZ CADENA,1993-09-09,GUCG930909MDFTDR05,TENANITLA,18,AMPLIACIÓN POTRERILLO,LA MAGDALENA CONTRERAS,CIUDAD DE MÉXICO,taconwine@gmail.com,5573803423,UNIÓN LIBRE,10600.0,12,120.0,2022-05-09,NaN,2063.49,Mensual,NaN,2022-07-12,2022-08-09,3,2035.11,SI,0.0,2035.11,0.0,NaN,28,2022-08,NaN,2023-05-31,295,1,1,1,1,1,2022-08-09,2035.11,2022-08-09,0.0,NaN
3,2463,0.519277,1,187769220,238891837,GRISELDA GUTIERREZ CADENA,1993-09-09,GUCG930909MDFTDR05,TENANITLA,18,AMPLIACIÓN POTRERILLO,LA MAGDALENA CONTRERAS,CIUDAD DE MÉXICO,taconwine@gmail.com,5573803423,UNIÓN LIBRE,10600.0,12,120.0,2022-05-09,NaN,2063.49,Mensual,NaN,2022-08-10,2022-09-09,4,1974.61,SI,0.0,1974.61,0.0,NaN,28,2022-09,NaN,2023-05-31,264,1,1,1,1,1,2022-09-09,1974.61,2022-09-09,0.0,NaN
4,2463,0.519277,1,187769220,238891837,GRISELDA GUTIERREZ CADENA,1993-09-09,GUCG930909MDFTDR05,TENANITLA,18,AMPLIACIÓN POTRERILLO,LA MAGDALENA CONTRERAS,CIUDAD DE MÉXICO,taconwine@gmail.com,5573803423,UNIÓN LIBRE,10600.0,12,120.0,2022-05-09,NaN,2063.49,Mensual,NaN,2022-09-10,2022-10-10,5,1680.00,SI,0.0,1680.00,0.0,NaN,28,2022-10,NaN,2023-05-31,233,1,1,1,1,1,2022-10-10,1680.00,2022-10-10,0.0,NaN


In [15]:
data_fact_con_score[data_fact_con_score['di_fecha_ven']>'2023-02-28']

,pk,proba,real_value,NoCliente,NoCredito,Nombre,FechaNacimiento,CURP,Calle,NumExt,Colonia,Ciudad,Estado,Email,Telefono,EstadoCivil,Monto,Plazo,Tasa,FechaOtorgamiento,FechaCierre,CuotaTotal,Frecuencia,Calificacion,di_fecha_ini,di_fecha_ven,dividendo,Cuota,Impago,Pagado,Impagado,Recuperado,Fecha Ultimo Pago,Años_pide_cred,Mes_Emision,fecha_recuperacion,fecha_recuperacion_aux,dias_en_default,Default_1_dias,Default_90_dias,Default_90_dias_cont,Default_1_dias_cont,Default_90_dias_pers,periodo_impagado,importe_moroso,periodo_moroso,importe_recuperado_moroso,periodo_recuperado_moroso


## Numero de creditos vivos en algun momento del periodo sin los ultimos meses

In [16]:
data_fact_con_score['NoCredito'].nunique()

13529

## Importe del credito medio

In [17]:
data_cred = data_fact_con_score[['NoCredito','Monto']].drop_duplicates()

In [18]:
data_cred['Monto'].sum()/data_cred['NoCredito'].nunique()

29136.757042649122

# Importe del credito medio anual

In [19]:
# NO NOS APLICA porque no tenemos un año, pero ya hemos sacado el importe del prestamo medio y de la factura media

Cogemos el periodo de un año

In [20]:
#df_anio = df_post_qa0_con_score[(df_post_qa0_con_score['Fecha_emision']>='2021-11-01')&(df_post_qa0_con_score['Fecha_emision']<='2022-10-31')]

In [21]:
#df_anio.head()

In [22]:
#df_anio['importe_cuota'].sum()

In [23]:
#df_anio['ID_Contrato'].nunique()

In [24]:
#df_anio['importe_cuota'].sum()/df_anio['ID_Contrato'].nunique()

# OBTENER VARIABLES POR CONTRATO NO POR FACTURA

In [25]:
print(data_fact_con_score.shape)
data_fact_con_score.head()

(54729, 48)


,pk,proba,real_value,NoCliente,NoCredito,Nombre,FechaNacimiento,CURP,Calle,NumExt,Colonia,Ciudad,Estado,Email,Telefono,EstadoCivil,Monto,Plazo,Tasa,FechaOtorgamiento,FechaCierre,CuotaTotal,Frecuencia,Calificacion,di_fecha_ini,di_fecha_ven,dividendo,Cuota,Impago,Pagado,Impagado,Recuperado,Fecha Ultimo Pago,Años_pide_cred,Mes_Emision,fecha_recuperacion,fecha_recuperacion_aux,dias_en_default,Default_1_dias,Default_90_dias,Default_90_dias_cont,Default_1_dias_cont,Default_90_dias_pers,periodo_impagado,importe_moroso,periodo_moroso,importe_recuperado_moroso,periodo_recuperado_moroso
0,2463,0.519277,1,187769220,238891837,GRISELDA GUTIERREZ CADENA,1993-09-09,GUCG930909MDFTDR05,TENANITLA,18,AMPLIACIÓN POTRERILLO,LA MAGDALENA CONTRERAS,CIUDAD DE MÉXICO,taconwine@gmail.com,5573803423,UNIÓN LIBRE,10600.0,12,120.0,2022-05-09,NaN,2063.49,Mensual,NaN,2022-05-09,2022-06-09,1,2116.26,SI,0.0,2116.26,0.0,NaN,28,2022-06,NaN,2023-05-31,356,1,1,1,1,1,2022-06-09,2116.26,2022-06-09,0.0,NaN
1,2463,0.519277,1,187769220,238891837,GRISELDA GUTIERREZ CADENA,1993-09-09,GUCG930909MDFTDR05,TENANITLA,18,AMPLIACIÓN POTRERILLO,LA MAGDALENA CONTRERAS,CIUDAD DE MÉXICO,taconwine@gmail.com,5573803423,UNIÓN LIBRE,10600.0,12,120.0,2022-05-09,NaN,2063.49,Mensual,NaN,2022-06-10,2022-07-11,2,2009.23,SI,0.0,2009.23,0.0,NaN,28,2022-07,NaN,2023-05-31,324,1,1,1,1,1,2022-07-11,2009.23,2022-07-11,0.0,NaN
2,2463,0.519277,1,187769220,238891837,GRISELDA GUTIERREZ CADENA,1993-09-09,GUCG930909MDFTDR05,TENANITLA,18,AMPLIACIÓN POTRERILLO,LA MAGDALENA CONTRERAS,CIUDAD DE MÉXICO,taconwine@gmail.com,5573803423,UNIÓN LIBRE,10600.0,12,120.0,2022-05-09,NaN,2063.49,Mensual,NaN,2022-07-12,2022-08-09,3,2035.11,SI,0.0,2035.11,0.0,NaN,28,2022-08,NaN,2023-05-31,295,1,1,1,1,1,2022-08-09,2035.11,2022-08-09,0.0,NaN
3,2463,0.519277,1,187769220,238891837,GRISELDA GUTIERREZ CADENA,1993-09-09,GUCG930909MDFTDR05,TENANITLA,18,AMPLIACIÓN POTRERILLO,LA MAGDALENA CONTRERAS,CIUDAD DE MÉXICO,taconwine@gmail.com,5573803423,UNIÓN LIBRE,10600.0,12,120.0,2022-05-09,NaN,2063.49,Mensual,NaN,2022-08-10,2022-09-09,4,1974.61,SI,0.0,1974.61,0.0,NaN,28,2022-09,NaN,2023-05-31,264,1,1,1,1,1,2022-09-09,1974.61,2022-09-09,0.0,NaN
4,2463,0.519277,1,187769220,238891837,GRISELDA GUTIERREZ CADENA,1993-09-09,GUCG930909MDFTDR05,TENANITLA,18,AMPLIACIÓN POTRERILLO,LA MAGDALENA CONTRERAS,CIUDAD DE MÉXICO,taconwine@gmail.com,5573803423,UNIÓN LIBRE,10600.0,12,120.0,2022-05-09,NaN,2063.49,Mensual,NaN,2022-09-10,2022-10-10,5,1680.00,SI,0.0,1680.00,0.0,NaN,28,2022-10,NaN,2023-05-31,233,1,1,1,1,1,2022-10-10,1680.00,2022-10-10,0.0,NaN


In [26]:
data_fact_con_score['FechaCierre_aux_mayo_23'] = np.where((data_fact_con_score['FechaCierre'].isnull())|(data_fact_con_score['FechaCierre']>'2023-05-28'), ('2023-05-28'), data_fact_con_score['FechaCierre'].astype(str))
data_fact_con_score['FechaCierre_aux_feb_23'] = np.where((data_fact_con_score['FechaCierre'].isnull())|(data_fact_con_score['FechaCierre']>'2023-02-28'), ('2023-02-28'), data_fact_con_score['FechaCierre'].astype(str))
data_fact_con_score['FechaCierre_aux_nov_22'] = np.where((data_fact_con_score['FechaCierre'].isnull())|(data_fact_con_score['FechaCierre']>'2022-11-28'), ('2022-11-28'), data_fact_con_score['FechaCierre'].astype(str))

In [27]:
data_fact_con_score['FechaCierre_aux_mayo_23'] = data_fact_con_score['FechaCierre_aux_mayo_23'].astype(str).str.split(' ').apply(lambda x: x[0])
data_fact_con_score['FechaCierre_aux_feb_23'] = data_fact_con_score['FechaCierre_aux_feb_23'].astype(str).str.split(' ').apply(lambda x: x[0])
data_fact_con_score['FechaCierre_aux_nov_22'] = data_fact_con_score['FechaCierre_aux_nov_22'].astype(str).str.split(' ').apply(lambda x: x[0])

In [28]:
data_fact_con_score['FechaCierre_aux_mayo_23'] = pd.to_datetime(data_fact_con_score['FechaCierre_aux_mayo_23'])
data_fact_con_score['FechaCierre_aux_feb_23'] = pd.to_datetime(data_fact_con_score['FechaCierre_aux_feb_23'])
data_fact_con_score['FechaCierre_aux_nov_22'] = pd.to_datetime(data_fact_con_score['FechaCierre_aux_nov_22'])

In [29]:
data_fact_con_score['FechaOtorgamiento'] = pd.to_datetime(data_fact_con_score['FechaOtorgamiento'])

In [30]:
data_fact_con_score['Tenure_mayo_23'] = (data_fact_con_score['FechaCierre_aux_mayo_23'] - data_fact_con_score['FechaOtorgamiento']).astype(str).str.split(' ').apply(lambda x: x[0])
data_fact_con_score['Tenure_feb_23'] = (data_fact_con_score['FechaCierre_aux_feb_23'] - data_fact_con_score['FechaOtorgamiento']).astype(str).str.split(' ').apply(lambda x: x[0])
data_fact_con_score['Tenure_nov_22'] = (data_fact_con_score['FechaCierre_aux_nov_22'] - data_fact_con_score['FechaOtorgamiento']).astype(str).str.split(' ').apply(lambda x: x[0])

# PASAR TABLA A LISTAS

In [31]:
print(data_fact_con_score.shape)
data_fact_con_score.head()

(54729, 54)


,pk,proba,real_value,NoCliente,NoCredito,Nombre,FechaNacimiento,CURP,Calle,NumExt,Colonia,Ciudad,Estado,Email,Telefono,EstadoCivil,Monto,Plazo,Tasa,FechaOtorgamiento,FechaCierre,CuotaTotal,Frecuencia,Calificacion,di_fecha_ini,di_fecha_ven,dividendo,Cuota,Impago,Pagado,Impagado,Recuperado,Fecha Ultimo Pago,Años_pide_cred,Mes_Emision,fecha_recuperacion,fecha_recuperacion_aux,dias_en_default,Default_1_dias,Default_90_dias,Default_90_dias_cont,Default_1_dias_cont,Default_90_dias_pers,periodo_impagado,importe_moroso,periodo_moroso,importe_recuperado_moroso,periodo_recuperado_moroso,FechaCierre_aux_mayo_23,FechaCierre_aux_feb_23,FechaCierre_aux_nov_22,Tenure_mayo_23,Tenure_feb_23,Tenure_nov_22
0,2463,0.519277,1,187769220,238891837,GRISELDA GUTIERREZ CADENA,1993-09-09,GUCG930909MDFTDR05,TENANITLA,18,AMPLIACIÓN POTRERILLO,LA MAGDALENA CONTRERAS,CIUDAD DE MÉXICO,taconwine@gmail.com,5573803423,UNIÓN LIBRE,10600.0,12,120.0,2022-05-09,NaN,2063.49,Mensual,NaN,2022-05-09,2022-06-09,1,2116.26,SI,0.0,2116.26,0.0,NaN,28,2022-06,NaN,2023-05-31,356,1,1,1,1,1,2022-06-09,2116.26,2022-06-09,0.0,NaN,2023-05-28,2023-02-28,2022-11-28,384,295,203
1,2463,0.519277,1,187769220,238891837,GRISELDA GUTIERREZ CADENA,1993-09-09,GUCG930909MDFTDR05,TENANITLA,18,AMPLIACIÓN POTRERILLO,LA MAGDALENA CONTRERAS,CIUDAD DE MÉXICO,taconwine@gmail.com,5573803423,UNIÓN LIBRE,10600.0,12,120.0,2022-05-09,NaN,2063.49,Mensual,NaN,2022-06-10,2022-07-11,2,2009.23,SI,0.0,2009.23,0.0,NaN,28,2022-07,NaN,2023-05-31,324,1,1,1,1,1,2022-07-11,2009.23,2022-07-11,0.0,NaN,2023-05-28,2023-02-28,2022-11-28,384,295,203
2,2463,0.519277,1,187769220,238891837,GRISELDA GUTIERREZ CADENA,1993-09-09,GUCG930909MDFTDR05,TENANITLA,18,AMPLIACIÓN POTRERILLO,LA MAGDALENA CONTRERAS,CIUDAD DE MÉXICO,taconwine@gmail.com,5573803423,UNIÓN LIBRE,10600.0,12,120.0,2022-05-09,NaN,2063.49,Mensual,NaN,2022-07-12,2022-08-09,3,2035.11,SI,0.0,2035.11,0.0,NaN,28,2022-08,NaN,2023-05-31,295,1,1,1,1,1,2022-08-09,2035.11,2022-08-09,0.0,NaN,2023-05-28,2023-02-28,2022-11-28,384,295,203
3,2463,0.519277,1,187769220,238891837,GRISELDA GUTIERREZ CADENA,1993-09-09,GUCG930909MDFTDR05,TENANITLA,18,AMPLIACIÓN POTRERILLO,LA MAGDALENA CONTRERAS,CIUDAD DE MÉXICO,taconwine@gmail.com,5573803423,UNIÓN LIBRE,10600.0,12,120.0,2022-05-09,NaN,2063.49,Mensual,NaN,2022-08-10,2022-09-09,4,1974.61,SI,0.0,1974.61,0.0,NaN,28,2022-09,NaN,2023-05-31,264,1,1,1,1,1,2022-09-09,1974.61,2022-09-09,0.0,NaN,2023-05-28,2023-02-28,2022-11-28,384,295,203
4,2463,0.519277,1,187769220,238891837,GRISELDA GUTIERREZ CADENA,1993-09-09,GUCG930909MDFTDR05,TENANITLA,18,AMPLIACIÓN POTRERILLO,LA MAGDALENA CONTRERAS,CIUDAD DE MÉXICO,taconwine@gmail.com,5573803423,UNIÓN LIBRE,10600.0,12,120.0,2022-05-09,NaN,2063.49,Mensual,NaN,2022-09-10,2022-10-10,5,1680.00,SI,0.0,1680.00,0.0,NaN,28,2022-10,NaN,2023-05-31,233,1,1,1,1,1,2022-10-10,1680.00,2022-10-10,0.0,NaN,2023-05-28,2023-02-28,2022-11-28,384,295,203


In [32]:
data_fact_con_score['NoCredito'].nunique()

13529

In [33]:
data_listas = data_fact_con_score.sort_values(['di_fecha_ini','di_fecha_ven']).groupby(by=['NoCredito'],as_index = False).aggregate(
    {   'pk': lambda x: list(x),
        'di_fecha_ini' : lambda x: list(x),
        'di_fecha_ven': lambda x: list(x),
        'Cuota': lambda x: list(x),
        'Impago': lambda x: list(x),
        'Impagado': lambda x: list(x),
        'periodo_impagado':lambda x: list(x),
        'Recuperado': lambda x: list(x),
        'fecha_recuperacion': lambda x: list(x),
        'importe_moroso': lambda x: list(x),
        'periodo_moroso': lambda x: list(x),
        'importe_recuperado_moroso': lambda x: list(x),
        'periodo_recuperado_moroso': lambda x: list(x),
        'dias_en_default': lambda x: list(x),
        'Default_90_dias': lambda x: list(x)}
)

In [34]:
data_listas['NoCredito'].nunique()

13529

In [35]:
print(data_listas.shape)
data_listas.head()

(13529, 16)


,NoCredito,pk,di_fecha_ini,di_fecha_ven,Cuota,Impago,Impagado,periodo_impagado,Recuperado,fecha_recuperacion,importe_moroso,periodo_moroso,importe_recuperado_moroso,periodo_recuperado_moroso,dias_en_default,Default_90_dias
0,112470,[4904],[2023-01-27],[2023-02-17],[6041.0],[NO],[0.0],[],[0.0],[nan],[0.0],[],[0.0],[],[0],[0]
1,212141,"[5487, 5487, 5487, 5487, 5487]","[2022-08-31, 2022-10-05, 2022-11-05, 2022-12-0...","[2022-10-04, 2022-11-04, 2022-12-05, 2023-01-0...","[2472.0, 2472.0, 2480.0, 2472.0, 2472.0]","[NO, NO, NO, NO, NO]","[0.0, 0.0, 0.0, 0.0, 0.0]","[, , , , ]","[0.0, 0.0, 0.0, 0.0, 0.0]","[nan, nan, nan, nan, nan]","[0.0, 0.0, 0.0, 0.0, 0.0]","[, , , , ]","[0.0, 0.0, 0.0, 0.0, 0.0]","[, , , , ]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]"
2,279368,"[2506, 2506]","[2022-12-28, 2023-01-24]","[2023-01-23, 2023-02-22]","[2873.0, 2873.0]","[NO, NO]","[0.0, 0.0]","[, ]","[0.0, 0.0]","[nan, nan]","[0.0, 0.0]","[, ]","[0.0, 0.0]","[, ]","[0, 0]","[0, 0]"
3,356307,[11006],[2023-01-10],[2023-02-10],[9148.14],[NO],[0.0],[],[0.0],[nan],[0.0],[],[0.0],[],[0],[0]
4,597891,"[7800, 7800, 7800, 7800, 7800, 7800]","[2022-08-26, 2022-09-27, 2022-10-27, 2022-11-2...","[2022-09-26, 2022-10-26, 2022-11-28, 2022-12-2...","[2552.3, 2555.11, 2552.97, 2553.0, 2553.0, 255...","[SI, SI, NO, NO, NO, NO]","[80.30000000000018, 770.1100000000001, 0.0, 0....","[2022-09-26, 2022-10-26, , , , ]","[80.30000000000018, 770.1100000000001, 0.0, 0....","[2022-09-27, 2022-10-28, nan, nan, nan, nan]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[, , , , , ]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[, , , , , ]","[1, 2, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0]"


In [36]:
data_cred = data_fact_con_score[['NoCredito','FechaOtorgamiento','FechaCierre',
                                   'proba','Default_90_dias_cont','Default_90_dias_pers',
                                   'Tenure_mayo_23','Tenure_feb_23','Tenure_nov_22']].drop_duplicates()

In [37]:
data_cred.shape

(13530, 9)

In [38]:
data_listas = data_cred.merge(data_listas,how='inner', on=['NoCredito'])
print(data_listas.shape)
data_listas.head()

(13530, 24)


,NoCredito,FechaOtorgamiento,FechaCierre,proba,Default_90_dias_cont,Default_90_dias_pers,Tenure_mayo_23,Tenure_feb_23,Tenure_nov_22,pk,di_fecha_ini,di_fecha_ven,Cuota,Impago,Impagado,periodo_impagado,Recuperado,fecha_recuperacion,importe_moroso,periodo_moroso,importe_recuperado_moroso,periodo_recuperado_moroso,dias_en_default,Default_90_dias
0,238891837,2022-05-09,NaN,0.519277,1,1,384,295,203,"[2463, 2463, 2463, 2463, 2463, 2463, 2463, 246...","[2022-05-09, 2022-06-10, 2022-07-12, 2022-08-1...","[2022-06-09, 2022-07-11, 2022-08-09, 2022-09-0...","[2116.26, 2009.23, 2035.11, 1974.61, 1680.0, 1...","[SI, SI, SI, SI, SI, SI, SI, SI, SI]","[2116.26, 2009.23, 2035.11, 1974.61, 1680.0, 1...","[2022-06-09, 2022-07-11, 2022-08-09, 2022-09-0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[nan, nan, nan, nan, nan, nan, nan, nan, nan]","[2116.26, 2009.23, 2035.11, 1974.61, 1680.0, 1...","[2022-06-09, 2022-07-11, 2022-08-09, 2022-09-0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[nan, nan, nan, nan, nan, nan, nan, nan, nan]","[356, 324, 295, 264, 233, 203, 173, 142, 111]","[1, 1, 1, 1, 1, 1, 1, 1, 1]"
1,214756696,2022-10-27,NaN,0.483736,0,0,213,124,32,"[11575, 11575, 11575, 11575]","[2022-10-27, 2022-11-29, 2022-12-28, 2023-01-28]","[2022-11-28, 2022-12-27, 2023-01-27, 2023-02-27]","[3296.0, 3298.96, 3296.0, 3296.0]","[SI, NO, NO, NO]","[3292.0, 0.0, 0.0, 0.0]","[2022-11-28, , , ]","[3292.0, 0.0, 0.0, 0.0]","[2022-11-29, nan, nan, nan]","[0.0, 0.0, 0.0, 0.0]","[, , , ]","[0.0, 0.0, 0.0, 0.0]","[, , , ]","[1, 0, 0, 0]","[0, 0, 0, 0]"
2,81198989,2022-04-25,NaN,0.442827,1,1,398,309,217,"[13031, 13031, 13031, 13031, 13031, 13031, 130...","[2022-04-25, 2022-05-26, 2022-06-28, 2022-07-2...","[2022-05-25, 2022-06-27, 2022-07-25, 2022-08-2...","[1547.0, 1546.01, 1546.0, 1546.0, 1556.04, 158...","[NO, NO, NO, NO, SI, SI, SI, SI, SI, SI]","[0.0, 0.0, 0.0, 0.0, 1556.04, 1560.73999999999...","[, , , , 2022-09-26, 2022-10-25, 2022-11-25, 2...","[0.0, 0.0, 0.0, 0.0, 1556.04, 1560.73999999999...","[nan, nan, nan, nan, 2022-09-28, 2022-10-31, 2...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2350....","[, , , , , , , , 2023-01-25, 2023-02-27]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[, , , , , , , , nan, nan]","[0, 0, 0, 0, 2, 6, 6, 4, 126, 93]","[0, 0, 0, 0, 0, 0, 0, 0, 1, 1]"
3,394556928,2022-07-22,NaN,0.438077,0,0,310,221,129,"[1340, 1340, 1340, 1340, 1340, 1340]","[2022-07-22, 2022-09-03, 2022-10-04, 2022-11-0...","[2022-09-02, 2022-10-03, 2022-11-03, 2022-12-0...","[3297.08, 3307.76, 3305.98, 3298.51, 3297.0, 3...","[SI, SI, SI, NO, NO, NO]","[3297.08, 10.800000000000182, 8.0, 0.0, 0.0, 0.0]","[2022-09-02, 2022-10-03, 2022-11-03, , , ]","[3297.08, 10.800000000000182, 8.0, 0.0, 0.0, 0.0]","[2022-09-05, 2022-10-11, 2022-11-04, nan, nan,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[, , , , , ]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[, , , , , ]","[3, 8, 1, 0, 0, 0]","[0, 0, 0, 0, 0, 0]"
4,169788967,2022-07-29,NaN,0.426874,0,0,303,214,122,"[13203, 13203, 13203, 13203, 13203, 13203, 13203]","[2022-07-29, 2022-08-30, 2022-09-30, 2022-11-0...","[2022-08-29, 2022-09-29, 2022-10-31, 2022-11-2...","[2786.0, 2769.0, 2786.0, 3756.0, 2996.07, 3357...","[NO, NO, NO, NO, SI, SI, SI]","[0.0, 0.0, 0.0, 0.0, 2996.07, 1457.73999999999...","[, , , , 2022-12-29, 2023-01-30, 2023-02-28]","[0.0, 0.0, 0.0, 0.0, 2996.07, 1457.73999999999...","[nan, nan, nan, nan, 2023-02-25, 2023-04-11, 2...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[, , , , , , ]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[, , , , , , ]","[0, 0, 0, 0, 58, 71, 42]","[0, 0, 0, 0, 0, 0, 0]"


In [39]:
data_listas['Tenure_mayo_23'] = pd.to_numeric(data_listas['Tenure_mayo_23'])
data_listas['Tenure_feb_23'] = pd.to_numeric(data_listas['Tenure_feb_23'])
data_listas['Tenure_nov_22'] = pd.to_numeric(data_listas['Tenure_nov_22'])

# 2. BUSINESS METRICS

# EMPIEZA LA PLANTILLA DE CALCULADORA

#### Son las celdas C9 y C10

In [110]:
print(data_listas[((data_listas['Tenure_mayo_23'])<180)&
                  ((data_listas['FechaCierre']>='2023-05-28')|(data_listas['FechaCierre'].isnull()))]['NoCredito'].nunique())

print(data_listas[((data_listas['Tenure_mayo_23'])>=180)&
                  ((data_listas['FechaCierre']>='2023-05-28')|(data_listas['FechaCierre'].isnull()))]['NoCredito'].nunique())

3820
7729


In [117]:
data_listas[((data_listas['FechaCierre']<'2023-02-01')&(data_listas['FechaCierre'].notnull()))]

,NoCredito,FechaOtorgamiento,FechaCierre,proba,Default_90_dias_cont,Default_90_dias_pers,Tenure_mayo_23,Tenure_feb_23,Tenure_nov_22,pk,di_fecha_ini,di_fecha_ven,Cuota,Impago,Impagado,periodo_impagado,Recuperado,fecha_recuperacion,importe_moroso,periodo_moroso,importe_recuperado_moroso,periodo_recuperado_moroso,dias_en_default,Default_90_dias
9,923651827,2022-06-01,2022-11-12 00:03:18,0.388023,0,1,164,164,164,"[10926, 10926, 10926, 10926, 10926, 10926]","[2022-06-01, 2022-07-02, 2022-08-02, 2022-09-0...","[2022-07-01, 2022-08-01, 2022-09-01, 2022-10-0...","[2477.0, 2508.35, 2541.33, 2505.88, 2477.0, 24...","[NO, SI, SI, NO, NO, NO]","[0.0, 2508.35, 2402.66, 0.0, 0.0, 0.0]","[, 2022-08-01, 2022-09-01, , , ]","[0.0, 2508.35, 2402.66, 0.0, 0.0, 0.0]","[nan, 2022-08-04, 2022-09-06, nan, nan, nan]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[, , , , , ]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[, , , , , ]","[0, 3, 5, 0, 0, 0]","[0, 0, 0, 0, 0, 0]"
37,941938848,2022-06-13,2022-11-17 00:09:06,0.342054,0,0,157,157,157,"[284, 284, 284, 284, 284, 284, 284, 284]","[2022-06-13, 2022-07-14, 2022-08-16, 2022-09-1...","[2022-07-13, 2022-08-15, 2022-09-13, 2022-10-1...","[2819.02, 2835.6, 2842.85, 2831.12, 2816.23, 1...","[SI, SI, SI, SI, SI, NO, NO, NO]","[2819.02, 2751.2, 2725.7, 2676.24, 2632.46, 0....","[2022-07-13, 2022-08-15, 2022-09-13, 2022-10-1...","[2819.02, 2751.2, 2725.7, 2676.24, 2632.46, 0....","[2022-07-14, 2022-08-19, 2022-09-15, 2022-10-1...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[, , , , , , , ]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[, , , , , , , ]","[1, 4, 2, 5, 2, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0]"
40,607806159,2022-04-26,2022-07-26 00:08:57,0.337286,0,0,91,91,91,"[8119, 8119, 8119, 8119, 8119, 8119]","[2022-04-26, 2022-05-27, 2022-06-28, 2022-07-2...","[2022-05-26, 2022-06-27, 2022-07-26, 2022-08-2...","[3500.0, 3500.0, 12199.36, 0.0, 0.0, 0.0]","[NO, NO, NO, NO, NO, NO]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[, , , , , ]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[nan, nan, nan, nan, nan, nan]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[, , , , , ]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[, , , , , ]","[0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0]"
50,854939668,2022-06-25,2022-11-20 00:02:43,0.331823,0,0,148,148,148,"[2437, 2437, 2437, 2437, 2437, 2437, 2437]","[2022-06-25, 2022-08-03, 2022-09-03, 2022-10-0...","[2022-08-02, 2022-09-02, 2022-10-03, 2022-11-0...","[3322.0, 3321.0, 3334.55, 3448.73, 9465.3, 0.0...","[NO, NO, SI, SI, NO, NO, NO]","[0.0, 0.0, 3334.55, 115.73000000000002, 0.0, 0...","[, , 2022-10-03, 2022-11-03, , , ]","[0.0, 0.0, 3334.55, 115.73000000000002, 0.0, 0...","[nan, nan, 2022-10-05, 2022-11-11, nan, nan, nan]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[, , , , , , ]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[, , , , , , ]","[0, 0, 2, 8, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0]"
57,78725322,2022-08-27,2022-11-03 14:29:52,0.324449,0,0,68,68,68,"[10324, 10324, 10324, 10324, 10324]","[2022-08-27, 2022-10-04, 2022-11-04, 2022-12-0...","[2022-10-03, 2022-11-03, 2022-12-02, 2023-01-0...","[5955.0, 25460.41, 0.0, 0.0, 0.0]","[NO, NO, NO, NO, NO]","[0.0, 0.0, 0.0, 0.0, 0.0]","[, , , , ]","[0.0, 0.0, 0.0, 0.0, 0.0]","[nan, nan, nan, nan, nan]","[0.0, 0.0, 0.0, 0.0, 0.0]","[, , , , ]","[0.0, 0.0, 0.0, 0.0, 0.0]","[, , , , ]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13175,364307105,2022-08-26,2022-10-23 00:11:05,0.038383,0,0,58,58,58,"[11209, 11209, 11209, 11209, 11209]","[2022-08-26, 2022-10-06, 2022-11-08, 2022-12-0...","[2022-10-05, 2022-11-07, 2022-12-05, 2023-01-0...","[2571.0, 10414.75, 0.0, 0.0, 0.0]","[NO, NO, NO, NO, NO]","[0.0, 0.0, 0.0, 0.0, 0.0]","[, , , , ]","[0.0, 0.0, 0.0, 0.0, 0.0]","[nan, nan, nan, nan, nan]","[0.0, 0.0, 0.0, 0.0, 0.0]","[, , , , ]","[0.0, 0.0, 0.0, 0.0, 0.0]","[, , , , ]","[0, 0, 0, 0, 0]","[0, 0, 0, 0, 0]"
13179,819112822,2022-08-31,2022-12-02 13:05:27,0.038331,0,0,93,93,89,"[9547, 9547, 9547, 9547, 9547]","[2022-08-31, 202

In [112]:
3820+7729+1980

13529

In [41]:
business_df = data_listas.copy()

In [42]:
business_df.rename(columns={'FechaOtorgamiento':'Mes Alta','FechaCierre':'Mes Baja','importe_moroso':'Importe_moroso','fecha_recuperacion':'FECHA_RECOBRO','periodo_moroso':'periodo_moroso',
                            'importe_recuperado_moroso':'IMPORTE_RECOBRADO_moroso', 'periodo_recuperado_moroso':'FECHA_RECOBRO_moroso', 'Impagado':'Importe_impagado',
                            'Recuperado':'IMPORTE_RECOBRADO','di_fecha_ven':'PERIOD', 'Cuota':'IMPORT'},inplace=True)

In [43]:
business_df.head()

,NoCredito,Mes Alta,Mes Baja,proba,Default_90_dias_cont,Default_90_dias_pers,Tenure_mayo_23,Tenure_feb_23,Tenure_nov_22,pk,di_fecha_ini,PERIOD,IMPORT,Impago,Importe_impagado,periodo_impagado,IMPORTE_RECOBRADO,FECHA_RECOBRO,Importe_moroso,periodo_moroso,IMPORTE_RECOBRADO_moroso,FECHA_RECOBRO_moroso,dias_en_default,Default_90_dias
0,238891837,2022-05-09,NaN,0.519277,1,1,384,295,203,"[2463, 2463, 2463, 2463, 2463, 2463, 2463, 246...","[2022-05-09, 2022-06-10, 2022-07-12, 2022-08-1...","[2022-06-09, 2022-07-11, 2022-08-09, 2022-09-0...","[2116.26, 2009.23, 2035.11, 1974.61, 1680.0, 1...","[SI, SI, SI, SI, SI, SI, SI, SI, SI]","[2116.26, 2009.23, 2035.11, 1974.61, 1680.0, 1...","[2022-06-09, 2022-07-11, 2022-08-09, 2022-09-0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[nan, nan, nan, nan, nan, nan, nan, nan, nan]","[2116.26, 2009.23, 2035.11, 1974.61, 1680.0, 1...","[2022-06-09, 2022-07-11, 2022-08-09, 2022-09-0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[nan, nan, nan, nan, nan, nan, nan, nan, nan]","[356, 324, 295, 264, 233, 203, 173, 142, 111]","[1, 1, 1, 1, 1, 1, 1, 1, 1]"
1,214756696,2022-10-27,NaN,0.483736,0,0,213,124,32,"[11575, 11575, 11575, 11575]","[2022-10-27, 2022-11-29, 2022-12-28, 2023-01-28]","[2022-11-28, 2022-12-27, 2023-01-27, 2023-02-27]","[3296.0, 3298.96, 3296.0, 3296.0]","[SI, NO, NO, NO]","[3292.0, 0.0, 0.0, 0.0]","[2022-11-28, , , ]","[3292.0, 0.0, 0.0, 0.0]","[2022-11-29, nan, nan, nan]","[0.0, 0.0, 0.0, 0.0]","[, , , ]","[0.0, 0.0, 0.0, 0.0]","[, , , ]","[1, 0, 0, 0]","[0, 0, 0, 0]"
2,81198989,2022-04-25,NaN,0.442827,1,1,398,309,217,"[13031, 13031, 13031, 13031, 13031, 13031, 130...","[2022-04-25, 2022-05-26, 2022-06-28, 2022-07-2...","[2022-05-25, 2022-06-27, 2022-07-25, 2022-08-2...","[1547.0, 1546.01, 1546.0, 1546.0, 1556.04, 158...","[NO, NO, NO, NO, SI, SI, SI, SI, SI, SI]","[0.0, 0.0, 0.0, 0.0, 1556.04, 1560.73999999999...","[, , , , 2022-09-26, 2022-10-25, 2022-11-25, 2...","[0.0, 0.0, 0.0, 0.0, 1556.04, 1560.73999999999...","[nan, nan, nan, nan, 2022-09-28, 2022-10-31, 2...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2350....","[, , , , , , , , 2023-01-25, 2023-02-27]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[, , , , , , , , nan, nan]","[0, 0, 0, 0, 2, 6, 6, 4, 126, 93]","[0, 0, 0, 0, 0, 0, 0, 0, 1, 1]"
3,394556928,2022-07-22,NaN,0.438077,0,0,310,221,129,"[1340, 1340, 1340, 1340, 1340, 1340]","[2022-07-22, 2022-09-03, 2022-10-04, 2022-11-0...","[2022-09-02, 2022-10-03, 2022-11-03, 2022-12-0...","[3297.08, 3307.76, 3305.98, 3298.51, 3297.0, 3...","[SI, SI, SI, NO, NO, NO]","[3297.08, 10.800000000000182, 8.0, 0.0, 0.0, 0.0]","[2022-09-02, 2022-10-03, 2022-11-03, , , ]","[3297.08, 10.800000000000182, 8.0, 0.0, 0.0, 0.0]","[2022-09-05, 2022-10-11, 2022-11-04, nan, nan,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[, , , , , ]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[, , , , , ]","[3, 8, 1, 0, 0, 0]","[0, 0, 0, 0, 0, 0]"
4,169788967,2022-07-29,NaN,0.426874,0,0,303,214,122,"[13203, 13203, 13203, 13203, 13203, 13203, 13203]","[2022-07-29, 2022-08-30, 2022-09-30, 2022-11-0...","[2022-08-29, 2022-09-29, 2022-10-31, 2022-11-2...","[2786.0, 2769.0, 2786.0, 3756.0, 2996.07, 3357...","[NO, NO, NO, NO, SI, SI, SI]","[0.0, 0.0, 0.0, 0.0, 2996.07, 1457.73999999999...","[, , , , 2022-12-29, 2023-01-30, 2023-02-28]","[0.0, 0.0, 0.0, 0.0, 2996.07, 1457.73999999999...","[nan, nan, nan, nan, 2023-02-25, 2023-04-11, 2...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[, , , , , , ]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[, , , , , , ]","[0, 0, 0, 0, 58, 71, 42]","[0, 0, 0, 0, 0, 0, 0]"


### Para calcular las celdas C58-60 correr una vez y luego otra vez desde el princpio C70-72

In [44]:
data_listas.shape

(13530, 24)

In [45]:
business_df_ori = business_df.copy()
from_6_to_18 = True # last month in file: 2022-12. Last month in training model 2022-08 cause 4-month tenure portfolio dropped
init_month = '2022-05-20'
end_month = '2022-11-28'

if from_6_to_18:
    business_df = business_df.loc[(business_df['Mes Alta']<=end_month)&((business_df['Mes Baja']>=init_month)|(business_df['Mes Baja'].isnull()))]
    print(business_df.shape)
    suffix = ' +120 dias filt'
    
    l_fac_m = []
    l_fac_a = []
    
    l_def_m = []
    l_rec_m = []
    l_def_a = []
    l_rec_a = []
    
    l_def_m_120 = []
    l_rec_m_120 = []
    l_def_a_120 = []
    l_rec_a_120 = []
    for i, row in business_df.iterrows():
        # ----- FACTURACION -----
        if isinstance(row['PERIOD'], list):
            #print(row['periodo_impagado'])
            #print(row['FECHA_RECOBRO'], row['IMPORTE_RECOBRADO'])
            #print(len(row['periodo_impagado']), len(row['FECHA_RECOBRO']), len(row['Importe_impagado']), len(row['IMPORTE_RECOBRADO']))
            
            lst_fac_m = []
            lst_fac_a = []
            for fac_m, fac_a in zip(row['PERIOD'], row['IMPORT']):
                if (fac_m>=init_month)&(fac_m<=end_month): # Para filtrar nos fijamos en la fecha de impago
                    lst_fac_m.append(fac_m)
                    lst_fac_a.append(fac_a)
                        
            l_fac_m.append(lst_fac_m)
            l_fac_a.append(lst_fac_a)
            
        else:
            l_fac_m.append(np.nan)
            l_fac_a.append(np.nan)
        
        # ----- IMPAGO +0 días -----
        if isinstance(row['periodo_impagado'], list):
            #print(row['periodo_impagado'])
            #print(row['FECHA_RECOBRO'], row['IMPORTE_RECOBRADO'])
            #print(len(row['periodo_impagado']), len(row['FECHA_RECOBRO']), len(row['Importe_impagado']), len(row['IMPORTE_RECOBRADO']))
            
            lst_def_m = []
            lst_rec_m = []
            lst_def_a = []
            lst_rec_a = []
            for def_m, rec_m, def_a, rec_a in zip(row['periodo_impagado'], row['FECHA_RECOBRO'], row['Importe_impagado'], row['IMPORTE_RECOBRADO']):
                if (def_m>=init_month)&(def_m<=end_month): # Para filtrar nos fijamos en la fecha de impago
                    lst_def_m.append(def_m)
                    lst_rec_m.append(rec_m)
                    lst_def_a.append(def_a)
                    lst_rec_a.append(rec_a)
                        
            l_def_m.append(lst_def_m)
            l_rec_m.append(lst_rec_m)
            l_def_a.append(lst_def_a)
            l_rec_a.append(lst_rec_a)
            
        else:
            l_def_m.append(np.nan)
            l_rec_m.append(np.nan)
            l_def_a.append(np.nan)
            l_rec_a.append(np.nan)
        
        # ----- IMPAGO +120 días -----
        if isinstance(row['periodo_moroso'], list):
            #print(row['periodo_impagado'])
            #print(row['FECHA_RECOBRO'], row['IMPORTE_RECOBRADO'])
            #print(len(row['periodo_impagado']), len(row['FECHA_RECOBRO']), len(row['Importe_impagado']), len(row['IMPORTE_RECOBRADO']))
            
            lst_def_m_120 = []
            lst_rec_m_120 = []
            lst_def_a_120 = []
            lst_rec_a_120 = []
            for def_m_120, rec_m_120, def_a_120, rec_a_120 in zip(row['periodo_moroso'], row['FECHA_RECOBRO_moroso'], row['Importe_moroso'], row['IMPORTE_RECOBRADO_moroso']):
                if (def_m_120>=init_month)&(def_m_120<=end_month): # Para filtrar nos fijamos en la fecha de impago
                    lst_def_m_120.append(def_m_120)
                    lst_rec_m_120.append(rec_m_120)
                    lst_def_a_120.append(def_a_120)
                    lst_rec_a_120.append(rec_a_120)
                        
            l_def_m_120.append(lst_def_m_120)
            l_rec_m_120.append(lst_rec_m_120)
            l_def_a_120.append(lst_def_a_120)
            l_rec_a_120.append(lst_rec_a_120)
            
        else:
            l_def_m_120.append(np.nan)
            l_rec_m_120.append(np.nan)
            l_def_a_120.append(np.nan)
            l_rec_a_120.append(np.nan)
            
    business_df['Meses facturacion filt'] = l_fac_m
    business_df['Importes facturacion filt'] = l_fac_a
    
    business_df['Meses Impago filt'] = l_def_m
    business_df['Meses Recuperación filt'] = l_rec_m
    business_df['Importes impago filt'] = l_def_a
    business_df['Importes recuperacion filt'] = l_rec_a
    
    business_df['Meses Impago +120 dias filt'] = l_def_m_120
    business_df['Meses Recuperación +120 dias filt'] = l_rec_m_120
    business_df['Importes impago +120 dias filt'] = l_def_a_120
    business_df['Importes recuperacion +120 dias filt'] = l_rec_a_120
    
    business_df['Importe facturacion filt'] = business_df['Importes facturacion filt'].apply(lambda x: sum(x) if isinstance(x, list) else x)
    business_df['Cuotas facturacion filt'] = business_df['Importes facturacion filt'].apply(lambda x: len(x) if isinstance(x, list) else 0)
    
    business_df['Importe impagado filt'] = business_df['Importes impago filt'].apply(lambda x: sum(x) if isinstance(x, list) else x)
    business_df['Importe recuperado filt'] = business_df['Importes recuperacion filt'].apply(lambda x: sum(x) if isinstance(x, list) else x)
    business_df['Cuotas impagadas filt'] = business_df['Importes impago filt'].apply(lambda x: len(x) if isinstance(x, list) else 0)
    
    business_df['Importe impagado +120 dias filt'] = business_df['Importes impago +120 dias filt'].apply(lambda x: sum(x) if isinstance(x, list) else x)
    business_df['Importe recuperado +120 dias filt'] = business_df['Importes recuperacion +120 dias filt'].apply(lambda x: sum(x) if isinstance(x, list) else x)
    business_df['Cuotas impagadas +120 dias filt'] = business_df['Importes impago +120 dias filt'].apply(lambda x: len(x) if isinstance(x, list) else 0)
    
else:
    suffix = ''

business_df.head(2)

(9503, 24)


,NoCredito,Mes Alta,Mes Baja,proba,Default_90_dias_cont,Default_90_dias_pers,Tenure_mayo_23,Tenure_feb_23,Tenure_nov_22,pk,di_fecha_ini,PERIOD,IMPORT,Impago,Importe_impagado,periodo_impagado,IMPORTE_RECOBRADO,FECHA_RECOBRO,Importe_moroso,periodo_moroso,IMPORTE_RECOBRADO_moroso,FECHA_RECOBRO_moroso,dias_en_default,Default_90_dias,Meses facturacion filt,Importes facturacion filt,Meses Impago filt,Meses Recuperación filt,Importes impago filt,Importes recuperacion filt,Meses Impago +120 dias filt,Meses Recuperación +120 dias filt,Importes impago +120 dias filt,Importes recuperacion +120 dias filt,Importe facturacion filt,Cuotas facturacion filt,Importe impagado filt,Importe recuperado filt,Cuotas impagadas filt,Importe impagado +120 dias filt,Importe recuperado +120 dias filt,Cuotas impagadas +120 dias filt
0,238891837,2022-05-09,NaN,0.519277,1,1,384,295,203,"[2463, 2463, 2463, 2463, 2463, 2463, 2463, 246...","[2022-05-09, 2022-06-10, 2022-07-12, 2022-08-1...","[2022-06-09, 2022-07-11, 2022-08-09, 2022-09-0...","[2116.26, 2009.23, 2035.11, 1974.61, 1680.0, 1...","[SI, SI, SI, SI, SI, SI, SI, SI, SI]","[2116.26, 2009.23, 2035.11, 1974.61, 1680.0, 1...","[2022-06-09, 2022-07-11, 2022-08-09, 2022-09-0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[nan, nan, nan, nan, nan, nan, nan, nan, nan]","[2116.26, 2009.23, 2035.11, 1974.61, 1680.0, 1...","[2022-06-09, 2022-07-11, 2022-08-09, 2022-09-0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[nan, nan, nan, nan, nan, nan, nan, nan, nan]","[356, 324, 295, 264, 233, 203, 173, 142, 111]","[1, 1, 1, 1, 1, 1, 1, 1, 1]","[2022-06-09, 2022-07-11, 2022-08-09, 2022-09-0...","[2116.26, 2009.23, 2035.11, 1974.61, 1680.0, 1...","[2022-06-09, 2022-07-11, 2022-08-09, 2022-09-0...","[nan, nan, nan, nan, nan, nan]","[2116.26, 2009.23, 2035.11, 1974.61, 1680.0, 1...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[2022-06-09, 2022-07-11, 2022-08-09, 2022-09-0...","[nan, nan, nan, nan, nan, nan]","[2116.26, 2009.23, 2035.11, 1974.61, 1680.0, 1...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",11495.21,6,11495.21,0.0,6,11495.21,0.0,6
1,214756696,2022-10-27,NaN,0.483736,0,0,213,124,32,"[11575, 11575, 11575, 11575]","[2022-10-27, 2022-11-29, 2022-12-28, 2023-01-28]","[2022-11-28, 2022-12-27, 2023-01-27, 2023-02-27]","[3296.0, 3298.96, 3296.0, 3296.0]","[SI, NO, NO, NO]","[3292.0, 0.0, 0.0, 0.0]","[2022-11-28, , , ]","[3292.0, 0.0, 0.0, 0.0]","[2022-11-29, nan, nan, nan]","[0.0, 0.0, 0.0, 0.0]","[, , , ]","[0.0, 0.0, 0.0, 0.0]","[, , , ]","[1, 0, 0, 0]","[0, 0, 0, 0]",[2022-11-28],[3296.0],[2022-11-28],[2022-11-29],[3292.0],[3292.0],[],[],[],[],3296.00,1,3292.00,3292.0,1,0.00,0.0,0


# CALCULO DE LAS CELDAS C70,C71 y C72

In [46]:
#Importe en mora en periodo estudiado
print(business_df['Importe impagado +120 dias filt'].sum())
#Importe impagado de los clientes morosos
print(business_df[business_df['Importe impagado +120 dias filt']>0]['Importe impagado filt'].sum())
#Importe anual de los clientes morosos
print(business_df['Importe facturacion filt'].sum())

4183865.86
5216149.93
82421417.16


In [47]:
#LGD
print(100-100*(business_df['Importe recuperado +120 dias filt'].sum())/(business_df['Importe impagado +120 dias filt'].sum()))

94.09020512909083


# CALCULO DE LAS CELDAS C58,C59 y C60 

In [95]:
#Importe en mora en periodo estudiado
print(business_df['Importe impagado +120 dias filt'].sum())
#Importe impagado de los clientes morosos
print(business_df[business_df['Importe impagado +120 dias filt']>0]['Importe impagado filt'].sum())
#Importe anual de los clientes morosos
print(business_df['Importe facturacion filt'].sum())

15555939.79
17996602.75
169099083.05


In [96]:
#LGD
print(100-100*(business_df['Importe recuperado +120 dias filt'].sum())/(business_df['Importe impagado +120 dias filt'].sum()))

94.36496565406159


In [97]:
#Importe recuperado moroso en el periodo de un año estudiado
print(business_df['Importe recuperado +120 dias filt'].sum())

876582.5499999999


# TICKET MEDIO

In [48]:
#max_insurance_amount = 999999 #300
mean_estimated_ticket = np.round(business_df['Importe impagado'+suffix].sum()/business_df['Cuotas impagadas'+suffix].sum(), 2)
mean_estimated_ticket = np.round(business_df['Importe facturacion filt'].sum()/business_df['Cuotas facturacion filt'].sum(), 8)
lgd = 1-business_df.loc[business_df['Default_90_dias_cont']==1, 'Importe recuperado'+suffix].sum()/business_df.loc[business_df['Default_90_dias_cont']==1, 'Importe impagado'+suffix].sum()
print('LGD: {0:.4f}\nTicket medio: {1:.8f}'.format(lgd, mean_estimated_ticket))
term = 1
business_df['term'] = term

#business_df['Importe impagado_limit'] = np.where(
#    business_df['Importe impagado'+suffix]>max_insurance_amount,
#    max_insurance_amount,
#    business_df['Importe impagado'+suffix]
#)
#business_df['Importe recuperado_limit'] = np.where(
#    business_df['Importe recuperado'+suffix]>max_insurance_amount,
#    max_insurance_amount,
#    business_df['Importe recuperado'+suffix]
#)
#business_df['installment'] = np.where(
#    business_df['Importe impagado_limit'].isnull(),
#    0,
#    business_df['Importe impagado_limit']
#)

business_df['installment'] = business_df['Importe impagado'+suffix]

business_df['principal'] = business_df['installment']
#mean_estimated_ticket = np.round(business_df['Importe impagado_limit'].sum()/business_df['Cuotas impagadas'+suffix].sum(), 2)
#lgd = 1-business_df.loc[business_df['real_value']==1, 'Importe recuperado_limit'].sum()/business_df.loc[business_df['real_value']==1, 'Importe impagado_limit'].sum()
#print('LGD: {0:.4f}\nTicket medio: {1:.2f}'.format(lgd, mean_estimated_ticket))
business_df[['proba', 'Default_90_dias_cont', 'principal', 'term', 'installment']].head()

LGD: 0.9409
Ticket medio: 4139.07583789


,proba,Default_90_dias_cont,principal,term,installment
0,0.519277,1,11495.21,1,11495.21
1,0.483736,0,0.00,1,0.00
2,0.442827,1,0.00,1,0.00
3,0.438077,0,0.00,1,0.00
4,0.426874,0,0.00,1,0.00


### CALCULADORA ZRS

In [49]:
suffix = ' +120 dias filt'

Calculamos los totales

In [50]:
def get_macro_numbers(df, invoice_col, default_col, recovery_col, suffix=' +120 dias filt'):
    print('Total invoice amount of portfolio: {0:.2f}€'.format(df[invoice_col].sum()))
    print('Total default amount of portfolio: {0:.2f}€'.format(df[default_col+suffix].sum()))
    print('PD: {0:.2f}%'.format(df[default_col+suffix].sum()/df[invoice_col].sum()*100))
    print('Total recovered amount of portfolio: {0:.2f}€'.format(df[recovery_col+suffix].sum()))
    print('Total pending amount of portfolio: {0:.2f}€'.format(df[default_col+suffix].sum()-df[recovery_col+suffix].sum()))
    print('PD of pending amount: {0:.2f}%'.format((df[default_col+suffix].sum()-df[recovery_col+suffix].sum())/df[invoice_col].sum()*100))

In [51]:
get_macro_numbers(
    df=business_df, 
    invoice_col='Importe facturacion filt',
    default_col='Importe impagado', 
    recovery_col='Importe recuperado', 
    suffix=' +120 dias filt'
)

Total invoice amount of portfolio: 82421417.16€
Total default amount of portfolio: 4183865.86€
PD: 5.08%
Total recovered amount of portfolio: 247257.89€
Total pending amount of portfolio: 3936607.97€
PD of pending amount: 4.78%


#### Numeros del periodo de analisis

Función general que calcula todo llamando a otras funciones más delimitadas

In [52]:
def get_numbers_over_studied_period(df, next_month, end_date_col, tenure_col, invoice_col, non_payment_col, default_col, recovery_non_payment_col, recovery_default_col, tenure_days=180, show=True):
    
    ### ----- NÚMERO DE CLIENTES ----- ###
    dic, lst_n_cli = get_n_cli_numbers(df, next_month, end_date_col, tenure_col, non_payment_col, default_col, tenure_days, show)
    ### ----------------------------------
    
    
    ### ----- FACTURACIÓN ----- ###
    lst_fact_cli = get_invoice_numbers(df, dic, next_month, end_date_col, tenure_col, invoice_col, tenure_days, show)
    ### ----------------------------------
    
    
    ### ----- IMPAGO Y RECUPERACIÓN IMPAGO ----- ###
    lst_impa_cli, lst_rec_impa_cli = get_non_payment_numbers(df, dic, next_month, end_date_col, tenure_col, non_payment_col, recovery_non_payment_col, tenure_days, show)
    ### ----------------------------------
    
    
    ### ----- MORA Y RECUPERACIÓN MORA----- ###
    lst_mora_cli, lst_rec_mora_cli = get_default_numbers(df, dic, next_month, end_date_col, tenure_col, default_col, recovery_default_col, tenure_days, show)
    ### ----------------------------------
    
    
    ### ----- BUILD DATAFRAME -----
    final_periodo_df = build_output_dataframe(lst_n_cli, lst_fact_cli, lst_impa_cli, lst_rec_impa_cli, lst_mora_cli, lst_rec_mora_cli, month_period='Enero2022')
    
    return final_periodo_df, dic

Funciones delimitadas para cálculos parciales, de clientes, facturación, etc..

In [53]:
def get_n_cli_numbers(df, next_month, end_date_col, tenure_col, non_payment_col, default_col, tenure_days, show):
    print('----- NÚMERO DE CLIENTES -----')
    if show:
        # Total clientes que estuvieron activos en algún momento del periodo 2020-06 a 2021-05
        print('\nTotal clientes activos en el periodo: {0:.0f}\n'.format(df.shape[0]))
    
    # Total clientes vivos al final del periodo con +/- 6 meses de antigüedad
    n_cli = df.loc[((df[end_date_col].isnull())|(df[end_date_col]>=next_month))].shape[0]
    n_cli_old = df.loc[((df[end_date_col].isnull())|(df[end_date_col]>=next_month))&(df[tenure_col]>tenure_days)].shape[0]
    n_cli_new = df.loc[((df[end_date_col].isnull())|(df[end_date_col]>=next_month))&(df[tenure_col]<=tenure_days)].shape[0]
    
    if show:
        print('Total clientes vivos al final del periodo: {0:.0f}'.format(n_cli))
        print('Total clientes vivos al final del periodo con +6 meses tenure: {0:.0f}'.format(n_cli_old))
        print('Total clientes vivos al final del periodo con -6 meses tenure: {0:.0f}'.format(n_cli_new))
        print('Check sobre la suma: {0}\n'.format(n_cli == n_cli_old+n_cli_new))
    
    idx_old_p = df.loc[((df[end_date_col].isnull())|(df[end_date_col]>=next_month))&(df[tenure_col]>tenure_days)&(df[non_payment_col]<=0)].index.tolist()
    idx_old_d = df.loc[((df[end_date_col].isnull())|(df[end_date_col]>=next_month))&(df[tenure_col]>tenure_days)&(df[non_payment_col]>0)&(df[default_col]<=0)].index.tolist()
    idx_old_m = df.loc[((df[end_date_col].isnull())|(df[end_date_col]>=next_month))&(df[tenure_col]>tenure_days)&(df[default_col]>0)].index.tolist()
    
    # Pagadores, Impagadores, Morosos con +6 meses de antigüedad
    n_cli_old_p = df.loc[idx_old_p].shape[0]
    n_cli_old_d = df.loc[idx_old_d].shape[0]
    n_cli_old_m = df.loc[idx_old_m].shape[0]
    if show:
        print('Pagadores con +6 meses: {0:.0f}'.format(n_cli_old_p))
        print('Impagadores con +6 meses: {0:.0f}'.format(n_cli_old_d))
        print('Morosos con +6 meses: {0:.0f}'.format(n_cli_old_m))
        print('Check sobre la suma: {0}\n'.format(n_cli_old == n_cli_old_p+n_cli_old_d+n_cli_old_m))
    
    idx_new_p = df.loc[((df[end_date_col].isnull())|(df[end_date_col]>=next_month))&(df[tenure_col]<=tenure_days)&(df[non_payment_col]<=0)].index.tolist()
    idx_new_d = df.loc[((df[end_date_col].isnull())|(df[end_date_col]>=next_month))&(df[tenure_col]<=tenure_days)&(df[non_payment_col]>0)&(df[default_col]<=0)].index.tolist()
    idx_new_m = df.loc[((df[end_date_col].isnull())|(df[end_date_col]>=next_month))&(df[tenure_col]<=tenure_days)&(df[default_col]>0)].index.tolist()
    
    # Pagadores, Impagadores, Morosos con -6 meses de antigüedad
    n_cli_new_p = df.loc[idx_new_p].shape[0]
    n_cli_new_d = df.loc[idx_new_d].shape[0]
    n_cli_new_m = df.loc[idx_new_m].shape[0]
    if show:
        print('Pagadores con -6 meses: {0:.0f}'.format(n_cli_new_p))
        print('Impagadores con -6 meses: {0:.0f}'.format(n_cli_new_d))
        print('Morosos con -6 meses: {0:.0f}'.format(n_cli_new_m))
        print('Check sobre la suma: {0}\n'.format(n_cli_new == n_cli_new_p+n_cli_new_d+n_cli_new_m))
    
    lst_n_cli = [n_cli_old_p, n_cli_old_d, n_cli_old_m, n_cli_new_p, n_cli_new_d, n_cli_new_m]
    dic = {
        'idx_old_p': idx_old_p,
        'idx_old_d': idx_old_d,
        'idx_old_m': idx_old_m,
        'idx_new_p': idx_new_p,
        'idx_new_d': idx_new_d,
        'idx_new_m': idx_new_m
    }
    print('-----------------------------\n\n')
    return dic, lst_n_cli

def get_invoice_numbers(df, dic, next_month, end_date_col, tenure_col, invoice_col, tenure_days, show):
    print('----- FACTURACIÓN -----')
    
    if show:
        # Total clientes que estuvieron activos en algún momento del periodo 2020-06 a 2021-05
        print('\nTotal facturación clientes activos en el periodo: {0:.0f}€'.format(df[invoice_col].sum()))
    
    lst_fact_cli = get_col_numbers(df, invoice_col, dic, next_month, end_date_col, tenure_col, tenure_days, show, name_to_print='facturación')
    
    print('-----------------------------\n\n')
    return lst_fact_cli

def get_non_payment_numbers(df, dic, next_month, end_date_col, tenure_col, non_payment_col, recovery_non_payment_col, tenure_days, show):
    print('----- IMPAGO -----')
    
    if show:    
        # Total clientes que estuvieron activos en algún momento del periodo 2020-06 a 2021-05
        print('\nTotal impago clientes activos en el periodo: {0:.0f}€'.format(df[non_payment_col].sum()))
        print('Total recuperado impago clientes activos en el periodo: {0:.0f}€'.format(df[recovery_non_payment_col].sum()))
        print('Total pendiente impago clientes activos en el periodo: {0:.0f}€'.format(df[non_payment_col].sum()-df[recovery_non_payment_col].sum()))
        print('LGD impago clientes activos en el periodo: {0:.2f}%'.format((df[non_payment_col].sum()-df[recovery_non_payment_col].sum())/df[non_payment_col].sum()*100))
    
    lst_impa_cli = get_col_numbers(df, non_payment_col, dic, next_month, end_date_col, tenure_col, tenure_days, show, name_to_print='impago')
    print('-----------------------------\n\n')
    
    print('----- RECUPERACIÓN IMPAGO -----')
    lst_rec_impa_cli = get_col_numbers(df, recovery_non_payment_col, dic, next_month, end_date_col, tenure_col, tenure_days, show, name_to_print='recuperado impago')
    print('-----------------------------\n\n')
    
    return lst_impa_cli, lst_rec_impa_cli

def get_default_numbers(df, dic, next_month, end_date_col, tenure_col, default_col, recovery_default_col, tenure_days, show):
    print('----- MORA -----')
    
    if show:    
        # Total clientes que estuvieron activos en algún momento del periodo 2020-06 a 2021-05
        print('\nTotal mora clientes activos en el periodo: {0:.0f}€'.format(df[default_col].sum()))
        print('Total recuperado mora clientes activos en el periodo: {0:.0f}€'.format(df[recovery_default_col].sum()))
        print('Total pendiente mora clientes activos en el periodo: {0:.0f}€'.format(df[default_col].sum()-df[recovery_default_col].sum()))
        print('LGD mora clientes activos en el periodo: {0:.2f}%'.format((df[default_col].sum()-df[recovery_default_col].sum())/df[default_col].sum()*100))
    
    lst_impa_cli = get_col_numbers(df, default_col, dic, next_month, end_date_col, tenure_col, tenure_days, show, name_to_print='mora')
    print('-----------------------------\n\n')
    
    print('----- RECUPERACIÓN MORA -----')
    lst_rec_impa_cli = get_col_numbers(df, recovery_default_col, dic, next_month, end_date_col, tenure_col, tenure_days, show, name_to_print='recuperado mora')
    print('-----------------------------\n\n')
    
    return lst_impa_cli, lst_rec_impa_cli

Función genérica para obtener los números en función de la columna que se le pase como parámetro

In [54]:
def get_col_numbers(df, colname, dic, next_month, end_date_col, tenure_col, tenure_days, show, name_to_print):
    
    # Total clientes vivos al final del periodo con +/- 6 meses de antigüedad
    var_cli = df.loc[((df[end_date_col].isnull())|(df[end_date_col]>=next_month)), colname].sum()
    var_cli_old = df.loc[((df[end_date_col].isnull())|(df[end_date_col]>=next_month))&(df[tenure_col]>tenure_days), colname].sum()
    var_cli_new = df.loc[((df[end_date_col].isnull())|(df[end_date_col]>=next_month))&(df[tenure_col]<=tenure_days), colname].sum()
    
    if show:
        print('\nTotal {0} clientes vivos al final del periodo: {1:.0f}€'.format(name_to_print, var_cli))
        print('Total {0} clientes vivos al final del periodo con +6 meses tenure: {1:.0f}€'.format(name_to_print, var_cli_old))
        print('Total {0} clientes vivos al final del periodo con -6 meses tenure: {1:.0f}€'.format(name_to_print, var_cli_new))
        print('Check sobre la suma: {0}\n'.format(abs(var_cli - (var_cli_old+var_cli_new)) <= 10**-6))
    
    # Pagadores, Impagadores, Morosos con +6 meses de antigüedad
    var_cli_old_p = df.loc[dic['idx_old_p'], colname].sum()
    var_cli_old_d = df.loc[dic['idx_old_d'], colname].sum()
    var_cli_old_m = df.loc[dic['idx_old_m'], colname].sum()
    if show:
        print('Pagadores con +6 meses: {0:.0f}€'.format(var_cli_old_p))
        print('Impagadores con +6 meses: {0:.0f}€'.format(var_cli_old_d))
        print('Morosos con +6 meses: {0:.0f}€'.format(var_cli_old_m))
        print('Check sobre la suma: {0}\n'.format(abs(var_cli_old - (var_cli_old_p+var_cli_old_d+var_cli_old_m)) <= 10**-6))
    
    # Pagadores, Impagadores, Morosos con -6 meses de antigüedad
    var_cli_new_p = df.loc[dic['idx_new_p'], colname].sum()
    var_cli_new_d = df.loc[dic['idx_new_d'], colname].sum()
    var_cli_new_m = df.loc[dic['idx_new_m'], colname].sum()
    if show:
        print('Pagadores con -6 meses: {0:.0f}€'.format(var_cli_new_p))
        print('Impagadores con -6 meses: {0:.0f}€'.format(var_cli_new_d))
        print('Morosos con -6 meses: {0:.0f}€'.format(var_cli_new_m))
        print('Check sobre la suma: {0}\n'.format(abs(var_cli_new - (var_cli_new_p+var_cli_new_d+var_cli_new_m)) <= 10**-6))
        
    lst_var_cli = [var_cli_old_p, var_cli_old_d, var_cli_old_m, var_cli_new_p, var_cli_new_d, var_cli_new_m]
    return lst_var_cli

Función que construye el dataframe para el delivery

In [55]:
def build_output_dataframe(lst_n_cli, lst_fact_cli, lst_impa_cli, lst_rec_impa_cli, lst_mora_cli, lst_rec_mora_cli, month_period='Junio 2021'):
    final_periodo_df = pd.DataFrame(
        index=[
            'Pagadores +6 meses', 'Impagadores +6 meses', 'Morosos +6 meses', 
            'Pagadores -6 meses', 'Impagadores -6 meses', 'Morosos -6 meses'
        ],
        columns=[
            '#', '$ Pagado', '$ Impagado', '$ Impagado recuperado', '$ Moroso', '$ Moroso recuperado', '$ Total'
        ]
    )
    final_periodo_df['#'] = lst_n_cli
    final_periodo_df['$ Impagado'] = lst_impa_cli
    final_periodo_df['$ Impagado recuperado'] = lst_rec_impa_cli
    final_periodo_df['$ Moroso'] = lst_mora_cli
    final_periodo_df['$ Moroso recuperado'] = lst_rec_mora_cli
    final_periodo_df['$ Total'] = lst_fact_cli
    
    final_periodo_df['$ Impagado'] = final_periodo_df['$ Impagado'] - final_periodo_df['$ Moroso']
    final_periodo_df['$ Impagado recuperado'] = final_periodo_df['$ Impagado recuperado'] - final_periodo_df['$ Moroso recuperado']
    final_periodo_df['$ Pagado'] = final_periodo_df['$ Total'] - final_periodo_df['$ Impagado'] - final_periodo_df['$ Moroso']
    final_periodo_df['PD impago'] = final_periodo_df['$ Impagado']/final_periodo_df['$ Total']
    final_periodo_df['PD mora'] = final_periodo_df['$ Moroso']/final_periodo_df['$ Total']
    final_periodo_df['LGD'] = 1 - final_periodo_df['$ Moroso recuperado']/final_periodo_df['$ Moroso']
    
    cols_df = pd.MultiIndex.from_product([[month_period], final_periodo_df])
    final_periodo_df.columns = cols_df
    return final_periodo_df

Ejecución de la función principal

In [56]:
business_df.columns

Index(['NoCredito', 'Mes Alta', 'Mes Baja', 'proba', 'Default_90_dias_cont',
       'Default_90_dias_pers', 'Tenure_mayo_23', 'Tenure_feb_23', 'Tenure_nov_22', 'pk',
       'di_fecha_ini', 'PERIOD', 'IMPORT', 'Impago', 'Importe_impagado', 'periodo_impagado',
       'IMPORTE_RECOBRADO', 'FECHA_RECOBRO', 'Importe_moroso', 'periodo_moroso',
       'IMPORTE_RECOBRADO_moroso', 'FECHA_RECOBRO_moroso', 'dias_en_default', 'Default_90_dias',
       'Meses facturacion filt', 'Importes facturacion filt', 'Meses Impago filt',
       'Meses Recuperación filt', 'Importes impago filt', 'Importes recuperacion filt',
       'Meses Impago +120 dias filt', 'Meses Recuperación +120 dias filt',
       'Importes impago +120 dias filt', 'Importes recuperacion +120 dias filt',
       'Importe facturacion filt', 'Cuotas facturacion filt', 'Importe impagado filt',
       'Importe recuperado filt', 'Cuotas impagadas filt', 'Importe impagado +120 dias filt',
       'Importe recuperado +120 dias filt', 'Cuotas imp

In [57]:
business_df.head(1)

,NoCredito,Mes Alta,Mes Baja,proba,Default_90_dias_cont,Default_90_dias_pers,Tenure_mayo_23,Tenure_feb_23,Tenure_nov_22,pk,di_fecha_ini,PERIOD,IMPORT,Impago,Importe_impagado,periodo_impagado,IMPORTE_RECOBRADO,FECHA_RECOBRO,Importe_moroso,periodo_moroso,IMPORTE_RECOBRADO_moroso,FECHA_RECOBRO_moroso,dias_en_default,Default_90_dias,Meses facturacion filt,Importes facturacion filt,Meses Impago filt,Meses Recuperación filt,Importes impago filt,Importes recuperacion filt,Meses Impago +120 dias filt,Meses Recuperación +120 dias filt,Importes impago +120 dias filt,Importes recuperacion +120 dias filt,Importe facturacion filt,Cuotas facturacion filt,Importe impagado filt,Importe recuperado filt,Cuotas impagadas filt,Importe impagado +120 dias filt,Importe recuperado +120 dias filt,Cuotas impagadas +120 dias filt,term,installment,principal
0,238891837,2022-05-09,NaN,0.519277,1,1,384,295,203,"[2463, 2463, 2463, 2463, 2463, 2463, 2463, 246...","[2022-05-09, 2022-06-10, 2022-07-12, 2022-08-1...","[2022-06-09, 2022-07-11, 2022-08-09, 2022-09-0...","[2116.26, 2009.23, 2035.11, 1974.61, 1680.0, 1...","[SI, SI, SI, SI, SI, SI, SI, SI, SI]","[2116.26, 2009.23, 2035.11, 1974.61, 1680.0, 1...","[2022-06-09, 2022-07-11, 2022-08-09, 2022-09-0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[nan, nan, nan, nan, nan, nan, nan, nan, nan]","[2116.26, 2009.23, 2035.11, 1974.61, 1680.0, 1...","[2022-06-09, 2022-07-11, 2022-08-09, 2022-09-0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[nan, nan, nan, nan, nan, nan, nan, nan, nan]","[356, 324, 295, 264, 233, 203, 173, 142, 111]","[1, 1, 1, 1, 1, 1, 1, 1, 1]","[2022-06-09, 2022-07-11, 2022-08-09, 2022-09-0...","[2116.26, 2009.23, 2035.11, 1974.61, 1680.0, 1...","[2022-06-09, 2022-07-11, 2022-08-09, 2022-09-0...","[nan, nan, nan, nan, nan, nan]","[2116.26, 2009.23, 2035.11, 1974.61, 1680.0, 1...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[2022-06-09, 2022-07-11, 2022-08-09, 2022-09-0...","[nan, nan, nan, nan, nan, nan]","[2116.26, 2009.23, 2035.11, 1974.61, 1680.0, 1...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",11495.21,6,11495.21,0.0,6,11495.21,0.0,6,1,11495.21,11495.21


In [58]:
business_df['Tenure_nov_22'] = pd.to_numeric(business_df['Tenure_nov_22'])

In [59]:
#Next month es el inicio del postperiodo
next_month='2022-11-28'
business_df[(business_df['Importe impagado filt']<=0)&(business_df['Importe impagado +120 dias filt']>0)&(business_df['Tenure_nov_22']<=180)&
           ((business_df['Mes Baja'].isnull())|(business_df['Mes Baja']>=next_month))]['Importe recuperado filt'].sum()

0.0

In [60]:
business_df.head(1)

,NoCredito,Mes Alta,Mes Baja,proba,Default_90_dias_cont,Default_90_dias_pers,Tenure_mayo_23,Tenure_feb_23,Tenure_nov_22,pk,di_fecha_ini,PERIOD,IMPORT,Impago,Importe_impagado,periodo_impagado,IMPORTE_RECOBRADO,FECHA_RECOBRO,Importe_moroso,periodo_moroso,IMPORTE_RECOBRADO_moroso,FECHA_RECOBRO_moroso,dias_en_default,Default_90_dias,Meses facturacion filt,Importes facturacion filt,Meses Impago filt,Meses Recuperación filt,Importes impago filt,Importes recuperacion filt,Meses Impago +120 dias filt,Meses Recuperación +120 dias filt,Importes impago +120 dias filt,Importes recuperacion +120 dias filt,Importe facturacion filt,Cuotas facturacion filt,Importe impagado filt,Importe recuperado filt,Cuotas impagadas filt,Importe impagado +120 dias filt,Importe recuperado +120 dias filt,Cuotas impagadas +120 dias filt,term,installment,principal
0,238891837,2022-05-09,NaN,0.519277,1,1,384,295,203,"[2463, 2463, 2463, 2463, 2463, 2463, 2463, 246...","[2022-05-09, 2022-06-10, 2022-07-12, 2022-08-1...","[2022-06-09, 2022-07-11, 2022-08-09, 2022-09-0...","[2116.26, 2009.23, 2035.11, 1974.61, 1680.0, 1...","[SI, SI, SI, SI, SI, SI, SI, SI, SI]","[2116.26, 2009.23, 2035.11, 1974.61, 1680.0, 1...","[2022-06-09, 2022-07-11, 2022-08-09, 2022-09-0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[nan, nan, nan, nan, nan, nan, nan, nan, nan]","[2116.26, 2009.23, 2035.11, 1974.61, 1680.0, 1...","[2022-06-09, 2022-07-11, 2022-08-09, 2022-09-0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[nan, nan, nan, nan, nan, nan, nan, nan, nan]","[356, 324, 295, 264, 233, 203, 173, 142, 111]","[1, 1, 1, 1, 1, 1, 1, 1, 1]","[2022-06-09, 2022-07-11, 2022-08-09, 2022-09-0...","[2116.26, 2009.23, 2035.11, 1974.61, 1680.0, 1...","[2022-06-09, 2022-07-11, 2022-08-09, 2022-09-0...","[nan, nan, nan, nan, nan, nan]","[2116.26, 2009.23, 2035.11, 1974.61, 1680.0, 1...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[2022-06-09, 2022-07-11, 2022-08-09, 2022-09-0...","[nan, nan, nan, nan, nan, nan]","[2116.26, 2009.23, 2035.11, 1974.61, 1680.0, 1...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",11495.21,6,11495.21,0.0,6,11495.21,0.0,6,1,11495.21,11495.21


In [61]:
data, dic = get_numbers_over_studied_period(
    df=business_df, 
    next_month='2022-11-28', 
    end_date_col='Mes Baja', 
    tenure_col='Tenure_nov_22',
    invoice_col='Importe facturacion filt',
    non_payment_col='Importe impagado filt', 
    default_col='Importe impagado +120 dias filt',
    recovery_non_payment_col='Importe recuperado filt',
    recovery_default_col='Importe recuperado +120 dias filt',
    show=True
)
data

----- NÚMERO DE CLIENTES -----

Total clientes activos en el periodo: 9503

Total clientes vivos al final del periodo: 9349
Total clientes vivos al final del periodo con +6 meses tenure: 195
Total clientes vivos al final del periodo con -6 meses tenure: 9154
Check sobre la suma: True

Pagadores con +6 meses: 57
Impagadores con +6 meses: 87
Morosos con +6 meses: 51
Check sobre la suma: True

Pagadores con -6 meses: 6123
Impagadores con -6 meses: 2578
Morosos con -6 meses: 453
Check sobre la suma: True

-----------------------------


----- FACTURACIÓN -----

Total facturación clientes activos en el periodo: 82421417€

Total facturación clientes vivos al final del periodo: 78525840€
Total facturación clientes vivos al final del periodo con +6 meses tenure: 3626735€
Total facturación clientes vivos al final del periodo con -6 meses tenure: 74899105€
Check sobre la suma: True

Pagadores con +6 meses: 990124€
Impagadores con +6 meses: 1670284€
Morosos con +6 meses: 966328€
Check sobre la su

Enero2022                                                              \
                             #     $ Pagado   $ Impagado $ Impagado recuperado    $ Moroso   
Pagadores +6 meses          57    990124.07         0.00                  0.00        0.00   
Impagadores +6 meses        87   1201811.75    468471.83             468485.01        0.00   
Morosos +6 meses            51    255677.06    101425.55             101426.67   609225.10   
Pagadores -6 meses        6123  41617670.67         0.00                  0.00        0.00   
Impagadores -6 meses      2578  15661564.92  11588287.30           11588755.38        0.00   
Morosos -6 meses           453   1556229.32    912665.76             912678.24  3562686.78   

                                                                                     
                     $ Moroso recuperado      $ Total PD impago   PD mora       LGD  
Pagadores +6 meses                  0.00    990124.07  0.000000  0.000000       NaN  
Impagadores +6 meses                0.00   1670283.58  0.280474  0.000000       NaN  
Morosos +6 meses                24222.74    966327.71  0.104960  0.630454  0.960240  
Pagadores -6 meses                  0.00  41617670.67  0.000000  0.000000       NaN  
Impagadores -6 meses                0.00  27249852.22  0.425261  0.000000       NaN  
Morosos -6 meses               211081.40   6031581.86  0.151314  0.590672  0.940752

#### NÚMEROS POST PERIODO DE ANÁLISIS

In [62]:
business_df_periodo = business_df.copy()
from_6_to_18 = True # last month in file: 2021-11. Last month in training model 2021-05 cause 6-month tenure portfolio dropped
init_month = '2022-05-20'
end_month = '2022-11-28'
post_month = '2023-02-28'

if from_6_to_18:
    business_df = business_df.loc[(business_df['Mes Alta']<=end_month)&((business_df['Mes Baja']>=init_month)|(business_df['Mes Baja'].isnull()))]
    print(business_df.shape)
    suffix = ' +120 dias filt'
    
    l_fac_m = []
    l_fac_a = []
    
    l_def_m = []
    l_rec_m = []
    l_def_a = []
    l_rec_a = []
    
    l_def_m_120 = []
    l_rec_m_120 = []
    l_def_a_120 = []
    l_rec_a_120 = []
    for i, row in business_df.iterrows():
        # ----- FACTURACION -----
        if isinstance(row['PERIOD'], list):
            #print(row['periodo_impagado'])
            #print(row['FECHA_RECOBRO'], row['IMPORTE_RECOBRADO'])
            #print(len(row['periodo_impagado']), len(row['FECHA_RECOBRO']), len(row['Importe_impagado']), len(row['IMPORTE_RECOBRADO']))
            
            lst_fac_m = []
            lst_fac_a = []
            for fac_m, fac_a in zip(row['PERIOD'], row['IMPORT']):
                if (fac_m>=init_month)&(fac_m<=post_month): # Para filtrar nos fijamos en la fecha de impago
                    lst_fac_m.append(fac_m)
                    lst_fac_a.append(fac_a)
                        
            l_fac_m.append(lst_fac_m)
            l_fac_a.append(lst_fac_a)
            
        else:
            l_fac_m.append(np.nan)
            l_fac_a.append(np.nan)
        
        # ----- IMPAGO +0 días -----
        if isinstance(row['periodo_impagado'], list):
            #print(row['periodo_impagado'])
            #print(row['FECHA_RECOBRO'], row['IMPORTE_RECOBRADO'])
            #print(len(row['periodo_impagado']), len(row['FECHA_RECOBRO']), len(row['Importe_impagado']), len(row['IMPORTE_RECOBRADO']))
            
            lst_def_m = []
            lst_rec_m = []
            lst_def_a = []
            lst_rec_a = []
            for def_m, rec_m, def_a, rec_a in zip(row['periodo_impagado'], row['FECHA_RECOBRO'], row['Importe_impagado'], row['IMPORTE_RECOBRADO']):
                if (def_m>=init_month)&(def_m<=post_month): # Para filtrar nos fijamos en la fecha de impago
                    lst_def_m.append(def_m)
                    lst_rec_m.append(rec_m)
                    lst_def_a.append(def_a)
                    lst_rec_a.append(rec_a)
                        
            l_def_m.append(lst_def_m)
            l_rec_m.append(lst_rec_m)
            l_def_a.append(lst_def_a)
            l_rec_a.append(lst_rec_a)
            
        else:
            l_def_m.append(np.nan)
            l_rec_m.append(np.nan)
            l_def_a.append(np.nan)
            l_rec_a.append(np.nan)
        
        # ----- IMPAGO +120 días -----
        if isinstance(row['periodo_moroso'], list):
            #print(row['periodo_impagado'])
            #print(row['FECHA_RECOBRO'], row['IMPORTE_RECOBRADO'])
            #print(len(row['periodo_impagado']), len(row['FECHA_RECOBRO']), len(row['Importe_impagado']), len(row['IMPORTE_RECOBRADO']))
            
            lst_def_m_120 = []
            lst_rec_m_120 = []
            lst_def_a_120 = []
            lst_rec_a_120 = []
            for def_m_120, rec_m_120, def_a_120, rec_a_120 in zip(row['periodo_moroso'], row['FECHA_RECOBRO_moroso'], row['Importe_moroso'], row['IMPORTE_RECOBRADO_moroso']):
                if (def_m_120>=init_month)&(def_m_120<=post_month): # Para filtrar nos fijamos en la fecha de impago
                    lst_def_m_120.append(def_m_120)
                    lst_rec_m_120.append(rec_m_120)
                    lst_def_a_120.append(def_a_120)
                    lst_rec_a_120.append(rec_a_120)
                        
            l_def_m_120.append(lst_def_m_120)
            l_rec_m_120.append(lst_rec_m_120)
            l_def_a_120.append(lst_def_a_120)
            l_rec_a_120.append(lst_rec_a_120)
            
        else:
            l_def_m_120.append(np.nan)
            l_rec_m_120.append(np.nan)
            l_def_a_120.append(np.nan)
            l_rec_a_120.append(np.nan)
            
    business_df['Meses facturacion filt post'] = l_fac_m
    business_df['Importes facturacion filt post'] = l_fac_a
    
    business_df['Meses Impago filt post'] = l_def_m
    business_df['Meses Recuperación filt post'] = l_rec_m
    business_df['Importes impago filt post'] = l_def_a
    business_df['Importes recuperacion filt post'] = l_rec_a
    
    business_df['Meses Impago +120 dias filt post'] = l_def_m_120
    business_df['Meses Recuperación +120 dias filt post'] = l_rec_m_120
    business_df['Importes impago +120 dias filt post'] = l_def_a_120
    business_df['Importes recuperacion +120 dias filt post'] = l_rec_a_120
    
    business_df['Importe facturacion filt post'] = business_df['Importes facturacion filt post'].apply(lambda x: sum(x) if isinstance(x, list) else x)
    business_df['Cuotas facturacion filt post'] = business_df['Importes facturacion filt post'].apply(lambda x: len(x) if isinstance(x, list) else 0)
    
    business_df['Importe impagado filt post'] = business_df['Importes impago filt post'].apply(lambda x: sum(x) if isinstance(x, list) else x)
    business_df['Importe recuperado filt post'] = business_df['Importes recuperacion filt post'].apply(lambda x: sum(x) if isinstance(x, list) else x)
    business_df['Cuotas impagadas filt post'] = business_df['Importes impago filt post'].apply(lambda x: len(x) if isinstance(x, list) else 0)
    
    business_df['Importe impagado +120 dias filt post'] = business_df['Importes impago +120 dias filt post'].apply(lambda x: sum(x) if isinstance(x, list) else x)
    business_df['Importe recuperado +120 dias filt post'] = business_df['Importes recuperacion +120 dias filt post'].apply(lambda x: sum(x) if isinstance(x, list) else x)
    business_df['Cuotas impagadas +120 dias filt post'] = business_df['Importes impago +120 dias filt post'].apply(lambda x: len(x) if isinstance(x, list) else 0)
    
else:
    suffix = ''

print(business_df.shape[0])
business_df.head(2)

(9503, 45)
9503


,NoCredito,Mes Alta,Mes Baja,proba,Default_90_dias_cont,Default_90_dias_pers,Tenure_mayo_23,Tenure_feb_23,Tenure_nov_22,pk,di_fecha_ini,PERIOD,IMPORT,Impago,Importe_impagado,periodo_impagado,IMPORTE_RECOBRADO,FECHA_RECOBRO,Importe_moroso,periodo_moroso,IMPORTE_RECOBRADO_moroso,FECHA_RECOBRO_moroso,dias_en_default,Default_90_dias,Meses facturacion filt,Importes facturacion filt,Meses Impago filt,Meses Recuperación filt,Importes impago filt,Importes recuperacion filt,Meses Impago +120 dias filt,Meses Recuperación +120 dias filt,Importes impago +120 dias filt,Importes recuperacion +120 dias filt,Importe facturacion filt,Cuotas facturacion filt,Importe impagado filt,Importe recuperado filt,Cuotas impagadas filt,Importe impagado +120 dias filt,Importe recuperado +120 dias filt,Cuotas impagadas +120 dias filt,term,installment,principal,Meses facturacion filt post,Importes facturacion filt post,Meses Impago filt post,Meses Recuperación filt post,Importes impago filt post,Importes recuperacion filt post,Meses Impago +120 dias filt post,Meses Recuperación +120 dias filt post,Importes impago +120 dias filt post,Importes recuperacion +120 dias filt post,Importe facturacion filt post,Cuotas facturacion filt post,Importe impagado filt post,Importe recuperado filt post,Cuotas impagadas filt post,Importe impagado +120 dias filt post,Importe recuperado +120 dias filt post,Cuotas impagadas +120 dias filt post
0,238891837,2022-05-09,NaN,0.519277,1,1,384,295,203,"[2463, 2463, 2463, 2463, 2463, 2463, 2463, 246...","[2022-05-09, 2022-06-10, 2022-07-12, 2022-08-1...","[2022-06-09, 2022-07-11, 2022-08-09, 2022-09-0...","[2116.26, 2009.23, 2035.11, 1974.61, 1680.0, 1...","[SI, SI, SI, SI, SI, SI, SI, SI, SI]","[2116.26, 2009.23, 2035.11, 1974.61, 1680.0, 1...","[2022-06-09, 2022-07-11, 2022-08-09, 2022-09-0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[nan, nan, nan, nan, nan, nan, nan, nan, nan]","[2116.26, 2009.23, 2035.11, 1974.61, 1680.0, 1...","[2022-06-09, 2022-07-11, 2022-08-09, 2022-09-0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[nan, nan, nan, nan, nan, nan, nan, nan, nan]","[356, 324, 295, 264, 233, 203, 173, 142, 111]","[1, 1, 1, 1, 1, 1, 1, 1, 1]","[2022-06-09, 2022-07-11, 2022-08-09, 2022-09-0...","[2116.26, 2009.23, 2035.11, 1974.61, 1680.0, 1...","[2022-06-09, 2022-07-11, 2022-08-09, 2022-09-0...","[nan, nan, nan, nan, nan, nan]","[2116.26, 2009.23, 2035.11, 1974.61, 1680.0, 1...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[2022-06-09, 2022-07-11, 2022-08-09, 2022-09-0...","[nan, nan, nan, nan, nan, nan]","[2116.26, 2009.23, 2035.11, 1974.61, 1680.0, 1...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",11495.21,6,11495.21,0.0,6,11495.21,0.0,6,1,11495.21,11495.21,"[2022-06-09, 2022-07-11, 2022-08-09, 2022-09-0...","[2116.26, 2009.23, 2035.11, 1974.61, 1680.0, 1...","[2022-06-09, 2022-07-11, 2022-08-09, 2022-09-0...","[nan, nan, nan, nan, nan, nan, nan, nan, nan]","[2116.26, 2009.23, 2035.11, 1974.61, 1680.0, 1...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[2022-06-09, 2022-07-11, 2022-08-09, 2022-09-0...","[nan, nan, nan, nan, nan, nan, nan, nan, nan]","[2116.26, 2009.23, 2035.11, 1974.61, 1680.0, 1...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",16535.21,9,16535.21,0.0,9,16535.21,0.0,9
1,214756696,2022-10-27,NaN,0.483736,0,0,213,124,32,"[11575, 11575, 11575, 11575]","[2022-10-27, 2022-11-29, 2022-12-28, 2023-01-28]","[2022-11-28, 2022-12-27, 2023-01-27, 2023-02-27]","[3296.0, 3298.96, 3296.0, 3296.0]","[SI, NO, NO, NO]","[3292.0, 0.0, 0.0, 0.0]","[2022-11-28, , , ]","[3292.0, 0.0, 0.0, 0.0]","[2022-11-29, nan, nan, nan]","[0.0, 0.0, 0.0, 0.0]","[, , , ]","[0.0, 0.0, 0.0, 0.0]","[, , , ]","[1, 0, 0, 0]","[0, 0, 0, 0]",[2022-11-28],[3296.0],[2022-11-28],[2022-11-29],[3292.0],[3292.0],[],[],[],[],3296.00,1,3292.00,3292.0,1,0.00,0.0,0,1,0.00,0.00,"[2022-11-28, 2022-12-27, 2023-01-27, 2023-02-27]","[3296.0, 3298.96, 3296.0, 3296.0]",[2022-11-28],[2022-11-29],[3292.0],[3292.0],[],[],[],[],13186.96,4,3292.00,3292.0,1,0.00,0.0,0


In [79]:
idx_old_p = dic['idx_old_p']
idx_old_d = dic['idx_old_d']
idx_old_m = dic['idx_old_m']
idx_new_p = dic['idx_new_p']
idx_new_d = dic['idx_new_d']
idx_new_m = dic['idx_new_m']

n_cli_old_p = data.loc['Pagadores +6 meses', ('Noviembre 2022','#')]
n_cli_new_p = data.loc['Pagadores -6 meses', ('Noviembre 2022','#')]
n_cli_old_d = data.loc['Impagadores +6 meses', ('Noviembre 2022','#')]
n_cli_new_d = data.loc['Impagadores -6 meses', ('Noviembre 2022','#')]
n_cli_old_m = data.loc['Morosos +6 meses', ('Noviembre 2022','#')]
n_cli_new_m = data.loc['Morosos -6 meses', ('Noviembre 2022','#')]

KeyError: ('Noviembre 2022', '#')

In [80]:
next_month = '2022-11'

# Para idx_old_p sacamos número de pagadores, impagadores y morosos
n_cli_old_p_vivos = business_df.loc[idx_old_p].loc[((business_df['Mes Baja'].isnull())|(business_df['Mes Baja']>=next_month))].shape[0]
n_cli_old_p_muertos = business_df.loc[idx_old_p].loc[(business_df['Mes Baja']<next_month)].shape[0]
print('Pagadores con +6 meses periodo anterior: {0:.0f}'.format(n_cli_old_p))
print('Pagadores con +6 meses aún activos: {0:.0f}'.format(n_cli_old_p_vivos))
print('Pagadores con +6 meses inactivos: {0:.0f}'.format(n_cli_old_p_muertos))
print('Check sobre la suma: {0}\n'.format(n_cli_old_p == n_cli_old_p_vivos+n_cli_old_p_muertos))
n_cli_old_p_p = business_df.loc[idx_old_p].loc[(business_df['Importe impagado filt post']<=0)].shape[0]
n_cli_old_p_d = business_df.loc[idx_old_p].loc[(business_df['Importe impagado filt post']>0)&(business_df['Importe impagado +120 dias filt post']<=0)].shape[0]
n_cli_old_p_m = business_df.loc[idx_old_p].loc[(business_df['Importe impagado +120 dias filt post']>0)].shape[0]
print('Pagadores con +6 meses son pagadores: {0:.0f}'.format(n_cli_old_p_p))
print('Pagadores con +6 meses son impagadores: {0:.0f}'.format(n_cli_old_p_d))
print('Pagadores con +6 meses son morosos: {0:.0f}'.format(n_cli_old_p_m))
print('Check sobre la suma: {0}\n'.format(n_cli_old_p == n_cli_old_p_p+n_cli_old_p_d+n_cli_old_p_m))

# Para idx_new_p sacamos número de pagadores, impagadores y morosos
n_cli_new_p_vivos = business_df.loc[idx_new_p].loc[((business_df['Mes Baja'].isnull())|(business_df['Mes Baja']>=next_month))].shape[0]
n_cli_new_p_muertos = business_df.loc[idx_new_p].loc[(business_df['Mes Baja']<next_month)].shape[0]
print('Pagadores con -6 meses periodo anterior: {0:.0f}'.format(n_cli_new_p))
print('Pagadores con -6 meses aún activos: {0:.0f}'.format(n_cli_new_p_vivos))
print('Pagadores con -6 meses inactivos: {0:.0f}'.format(n_cli_new_p_muertos))
print('Check sobre la suma: {0}\n'.format(n_cli_new_p == n_cli_new_p_vivos+n_cli_new_p_muertos))
n_cli_new_p_p = business_df.loc[idx_new_p].loc[(business_df['Importe impagado filt post']<=0)].shape[0]
n_cli_new_p_d = business_df.loc[idx_new_p].loc[(business_df['Importe impagado filt post']>0)&(business_df['Importe impagado +120 dias filt post']<=0)].shape[0]
n_cli_new_p_m = business_df.loc[idx_new_p].loc[(business_df['Importe impagado +120 dias filt post']>0)].shape[0]
print('Pagadores con -6 meses son pagadores: {0:.0f}'.format(n_cli_new_p_p))
print('Pagadores con -6 meses son impagadores: {0:.0f}'.format(n_cli_new_p_d))
print('Pagadores con -6 meses son morosos: {0:.0f}'.format(n_cli_new_p_m))
print('Check sobre la suma: {0}\n'.format(n_cli_new_p == n_cli_new_p_p+n_cli_new_p_d+n_cli_new_p_m))


# Para idx_old_d sacamos número de pagadores, impagadores y morosos
n_cli_old_d_vivos = business_df.loc[idx_old_d].loc[((business_df['Mes Baja'].isnull())|(business_df['Mes Baja']>=next_month))].shape[0]
n_cli_old_d_muertos = business_df.loc[idx_old_d].loc[(business_df['Mes Baja']<next_month)].shape[0]
print('Impagadores con +6 meses periodo anterior: {0:.0f}'.format(n_cli_old_d))
print('Impagadores con +6 meses aún activos: {0:.0f}'.format(n_cli_old_d_vivos))
print('Impagadores con +6 meses inactivos: {0:.0f}'.format(n_cli_old_d_muertos))
print('Check sobre la suma: {0}\n'.format(n_cli_old_d == n_cli_old_d_vivos+n_cli_old_d_muertos))
n_cli_old_d_p = business_df.loc[idx_old_d].loc[(business_df['Importe impagado filt post']<=0)].shape[0]
n_cli_old_d_d = business_df.loc[idx_old_d].loc[(business_df['Importe impagado filt post']>0)&(business_df['Importe impagado +120 dias filt post']<=0)].shape[0]
n_cli_old_d_m = business_df.loc[idx_old_d].loc[(business_df['Importe impagado +120 dias filt post']>0)].shape[0]
print('Impagadores con +6 meses son pagadores: {0:.0f}'.format(n_cli_old_d_p))
print('Impagadores con +6 meses son impagadores: {0:.0f}'.format(n_cli_old_d_d))
print('Impagadores con +6 meses son morosos: {0:.0f}'.format(n_cli_old_d_m))
print('Check sobre la suma: {0}\n'.format(n_cli_old_d == n_cli_old_d_p+n_cli_old_d_d+n_cli_old_d_m))

# Para idx_old_d sacamos número de pagadores, impagadores y morosos
n_cli_new_d_vivos = business_df.loc[idx_new_d].loc[((business_df['Mes Baja'].isnull())|(business_df['Mes Baja']>=next_month))].shape[0]
n_cli_new_d_muertos = business_df.loc[idx_new_d].loc[(business_df['Mes Baja']<next_month)].shape[0]
print('Impagadores con -6 meses periodo anterior: {0:.0f}'.format(n_cli_new_d))
print('Impagadores con -6 meses aún activos: {0:.0f}'.format(n_cli_new_d_vivos))
print('Impagadores con -6 meses inactivos: {0:.0f}'.format(n_cli_new_d_muertos))
print('Check sobre la suma: {0}\n'.format(n_cli_new_d == n_cli_new_d_vivos+n_cli_new_d_muertos))
n_cli_new_d_p = business_df.loc[idx_new_d].loc[(business_df['Importe impagado filt post']<=0)].shape[0]
n_cli_new_d_d = business_df.loc[idx_new_d].loc[(business_df['Importe impagado filt post']>0)&(business_df['Importe impagado +120 dias filt post']<=0)].shape[0]
n_cli_new_d_m = business_df.loc[idx_new_d].loc[(business_df['Importe impagado +120 dias filt post']>0)].shape[0]
print('Impagadores con -6 meses son pagadores: {0:.0f}'.format(n_cli_new_d_p))
print('Impagadores con -6 meses son impagadores: {0:.0f}'.format(n_cli_new_d_d))
print('Impagadores con -6 meses son morosos: {0:.0f}'.format(n_cli_new_d_m))
print('Check sobre la suma: {0}\n'.format(n_cli_new_d == n_cli_new_d_p+n_cli_new_d_d+n_cli_new_d_m))


# Para idx_old_m sacamos número de pagadores, impagadores y morosos
n_cli_old_m_vivos = business_df.loc[idx_old_m].loc[((business_df['Mes Baja'].isnull())|(business_df['Mes Baja']>=next_month))].shape[0]
n_cli_old_m_muertos = business_df.loc[idx_old_m].loc[(business_df['Mes Baja']<next_month)].shape[0]
print('Morosos con +6 meses periodo anterior: {0:.0f}'.format(n_cli_old_m))
print('Morosos con +6 meses aún activos: {0:.0f}'.format(n_cli_old_m_vivos))
print('Morosos con +6 meses inactivos: {0:.0f}'.format(n_cli_old_m_muertos))
print('Check sobre la suma: {0}\n'.format(n_cli_old_m == n_cli_old_m_vivos+n_cli_old_m_muertos))
n_cli_old_m_p = business_df.loc[idx_old_m].loc[(business_df['Importe impagado filt post']<=0)].shape[0]
n_cli_old_m_d = business_df.loc[idx_old_m].loc[(business_df['Importe impagado filt post']>0)&(business_df['Importe impagado +120 dias filt post']<=0)].shape[0]
n_cli_old_m_m = business_df.loc[idx_old_m].loc[(business_df['Importe impagado +120 dias filt post']>0)].shape[0]
print('Morosos con +6 meses son pagadores: {0:.0f}'.format(n_cli_old_m_p))
print('Morosos con +6 meses son impagadores: {0:.0f}'.format(n_cli_old_m_d))
print('Morosos con +6 meses son morosos: {0:.0f}'.format(n_cli_old_m_m))
print('Check sobre la suma: {0}\n'.format(n_cli_old_m == n_cli_old_m_p+n_cli_old_m_d+n_cli_old_m_m))

n_cli_new_m_vivos = business_df.loc[idx_new_m].loc[((business_df['Mes Baja'].isnull())|(business_df['Mes Baja']>=next_month))].shape[0]
n_cli_new_m_muertos = business_df.loc[idx_new_m].loc[(business_df['Mes Baja']<next_month)].shape[0]
print('Morosos con -6 meses periodo anterior: {0:.0f}'.format(n_cli_new_m))
print('Morosos con -6 meses aún activos: {0:.0f}'.format(n_cli_new_m_vivos))
print('Morosos con -6 meses inactivos: {0:.0f}'.format(n_cli_new_m_muertos))
print('Check sobre la suma: {0}\n'.format(n_cli_new_m == n_cli_new_m_vivos+n_cli_new_m_muertos))
n_cli_new_m_p = business_df.loc[idx_new_m].loc[(business_df['Importe impagado filt post']<=0)].shape[0]
n_cli_new_m_d = business_df.loc[idx_new_m].loc[(business_df['Importe impagado filt post']>0)&(business_df['Importe impagado +120 dias filt post']<=0)].shape[0]
n_cli_new_m_m = business_df.loc[idx_new_m].loc[(business_df['Importe impagado +120 dias filt post']>0)].shape[0]
print('Morosos con -6 meses son pagadores: {0:.0f}'.format(n_cli_new_m_p))
print('Morosos con -6 meses son impagadores: {0:.0f}'.format(n_cli_new_m_d))
print('Morosos con -6 meses son morosos: {0:.0f}'.format(n_cli_new_m_m))
print('Check sobre la suma: {0}\n'.format(n_cli_new_m == n_cli_new_m_p+n_cli_new_m_d+n_cli_new_m_m))


lst_n_cli_p = [n_cli_old_p_p, n_cli_old_d_p, n_cli_old_m_p, n_cli_new_p_p, n_cli_new_d_p, n_cli_new_m_p]
print(lst_n_cli_p, '\n')

lst_n_cli_d = [n_cli_old_p_d, n_cli_old_d_d, n_cli_old_m_d, n_cli_new_p_d, n_cli_new_d_d, n_cli_new_m_d]
print(lst_n_cli_d, '\n')

lst_n_cli_m = [n_cli_old_p_m, n_cli_old_d_m, n_cli_old_m_m, n_cli_new_p_m, n_cli_new_d_m, n_cli_new_m_m]
print(lst_n_cli_m, '\n')

Pagadores con +6 meses periodo anterior: 57
Pagadores con +6 meses aún activos: 57
Pagadores con +6 meses inactivos: 0
Check sobre la suma: True

Pagadores con +6 meses son pagadores: 45
Pagadores con +6 meses son impagadores: 11
Pagadores con +6 meses son morosos: 1
Check sobre la suma: True

Pagadores con -6 meses periodo anterior: 6123
Pagadores con -6 meses aún activos: 6123
Pagadores con -6 meses inactivos: 0
Check sobre la suma: True

Pagadores con -6 meses son pagadores: 3361
Pagadores con -6 meses son impagadores: 2439
Pagadores con -6 meses son morosos: 323
Check sobre la suma: True

Impagadores con +6 meses periodo anterior: 87
Impagadores con +6 meses aún activos: 87
Impagadores con +6 meses inactivos: 0
Check sobre la suma: True

Impagadores con +6 meses son pagadores: 0
Impagadores con +6 meses son impagadores: 71
Impagadores con +6 meses son morosos: 16
Check sobre la suma: True

Impagadores con -6 meses periodo anterior: 2578
Impagadores con -6 meses aún activos: 2578
Im

#### Facturación

In [81]:
#next_month = '2022-09'

# Para idx_old_p sacamos total facturado de pagadores, impagadores y morosos. NOS DA IGUAL SI ESTÁN VIVOS O NO, ANALIZAMOS TODO LO OCURRIDO PARA LOS CLIENTES VIVOS AL FINAL DEL PERIODO LOS 12 MESES ANTERIORES Y DE AHÍ EN ADELANTE
fact_cli_old_p = business_df.loc[idx_old_p, 'Importe facturacion filt post'].sum() - business_df.loc[idx_old_p, 'Importe facturacion filt'].sum()
n_cli_old_p_p = business_df.loc[idx_old_p].loc[(business_df['Importe impagado filt post']<=0), 'Importe facturacion filt post'].sum() - \
                business_df.loc[idx_old_p].loc[(business_df['Importe impagado filt post']<=0), 'Importe facturacion filt'].sum()
n_cli_old_p_d = business_df.loc[idx_old_p].loc[(business_df['Importe impagado filt post']>0)&(business_df['Importe impagado +120 dias filt post']<=0), 'Importe facturacion filt post'].sum() - \
                business_df.loc[idx_old_p].loc[(business_df['Importe impagado filt post']>0)&(business_df['Importe impagado +120 dias filt post']<=0), 'Importe facturacion filt'].sum()
n_cli_old_p_m = business_df.loc[idx_old_p].loc[(business_df['Importe impagado +120 dias filt post']>0), 'Importe facturacion filt post'].sum() - \
                business_df.loc[idx_old_p].loc[(business_df['Importe impagado +120 dias filt post']>0), 'Importe facturacion filt'].sum()
print('Pagadores con +6 meses son pagadores: {0:.0f}€'.format(n_cli_old_p_p))
print('Pagadores con +6 meses son impagadores: {0:.0f}€'.format(n_cli_old_p_d))
print('Pagadores con +6 meses son morosos: {0:.0f}€'.format(n_cli_old_p_m))
print('Check sobre la suma: {0}\n'.format(abs(fact_cli_old_p - (n_cli_old_p_p+n_cli_old_p_d+n_cli_old_p_m)) < 10**-6))

# Para idx_new_p sacamos total facturado de pagadores, impagadores y morosos. NOS DA IGUAL SI ESTÁN VIVOS O NO, ANALIZAMOS TODO LO OCURRIDO PARA LOS CLIENTES VIVOS AL FINAL DEL PERIODO LOS 12 MESES ANTERIORES Y DE AHÍ EN ADELANTE
fact_cli_new_p = business_df.loc[idx_new_p, 'Importe facturacion filt post'].sum() - business_df.loc[idx_new_p, 'Importe facturacion filt'].sum()
n_cli_new_p_p = business_df.loc[idx_new_p].loc[(business_df['Importe impagado filt post']<=0), 'Importe facturacion filt post'].sum() - \
                business_df.loc[idx_new_p].loc[(business_df['Importe impagado filt post']<=0), 'Importe facturacion filt'].sum()
n_cli_new_p_d = business_df.loc[idx_new_p].loc[(business_df['Importe impagado filt post']>0)&(business_df['Importe impagado +120 dias filt post']<=0), 'Importe facturacion filt post'].sum() - \
                business_df.loc[idx_new_p].loc[(business_df['Importe impagado filt post']>0)&(business_df['Importe impagado +120 dias filt post']<=0), 'Importe facturacion filt'].sum()
n_cli_new_p_m = business_df.loc[idx_new_p].loc[(business_df['Importe impagado +120 dias filt post']>0), 'Importe facturacion filt post'].sum() - \
                business_df.loc[idx_new_p].loc[(business_df['Importe impagado +120 dias filt post']>0), 'Importe facturacion filt'].sum()
print('Pagadores con +6 meses son pagadores: {0:.0f}€'.format(n_cli_new_p_p))
print('Pagadores con +6 meses son impagadores: {0:.0f}€'.format(n_cli_old_p_d))
print('Pagadores con +6 meses son morosos: {0:.0f}€'.format(n_cli_new_p_m))
print('Check sobre la suma: {0}\n'.format(abs(fact_cli_new_p - (n_cli_new_p_p+n_cli_new_p_d+n_cli_new_p_m)) < 10**-6))


# Para idx_old_d sacamos total facturado de pagadores, impagadores y morosos. NOS DA IGUAL SI ESTÁN VIVOS O NO, ANALIZAMOS TODO LO OCURRIDO PARA LOS CLIENTES VIVOS AL FINAL DEL PERIODO LOS 12 MESES ANTERIORES Y DE AHÍ EN ADELANTE
fact_cli_old_d = business_df.loc[idx_old_d, 'Importe facturacion filt post'].sum() - \
                business_df.loc[idx_old_d, 'Importe facturacion filt'].sum()
n_cli_old_d_p = business_df.loc[idx_old_d].loc[(business_df['Importe impagado filt post']<=0), 'Importe facturacion filt post'].sum() - \
                business_df.loc[idx_old_d].loc[(business_df['Importe impagado filt post']<=0), 'Importe facturacion filt'].sum()
n_cli_old_d_d = business_df.loc[idx_old_d].loc[(business_df['Importe impagado filt post']>0)&(business_df['Importe impagado +120 dias filt post']<=0), 'Importe facturacion filt post'].sum() - \
                business_df.loc[idx_old_d].loc[(business_df['Importe impagado filt post']>0)&(business_df['Importe impagado +120 dias filt post']<=0), 'Importe facturacion filt'].sum()
n_cli_old_d_m = business_df.loc[idx_old_d].loc[(business_df['Importe impagado +120 dias filt post']>0), 'Importe facturacion filt post'].sum() - \
                business_df.loc[idx_old_d].loc[(business_df['Importe impagado +120 dias filt post']>0), 'Importe facturacion filt'].sum()
print('Impagadores con +6 meses son pagadores: {0:.0f}€'.format(n_cli_old_d_p))
print('Impagadores con +6 meses son impagadores: {0:.0f}€'.format(n_cli_old_d_d))
print('Impagadores con +6 meses son morosos: {0:.0f}€'.format(n_cli_old_d_m))
print('Check sobre la suma: {0}\n'.format(abs(fact_cli_old_d - (n_cli_old_d_p+n_cli_old_d_d+n_cli_old_d_m)) < 10**-6))

# Para idx_new_d sacamos total facturado de pagadores, impagadores y morosos. NOS DA IGUAL SI ESTÁN VIVOS O NO, ANALIZAMOS TODO LO OCURRIDO PARA LOS CLIENTES VIVOS AL FINAL DEL PERIODO LOS 12 MESES ANTERIORES Y DE AHÍ EN ADELANTE
fact_cli_new_d = business_df.loc[idx_new_d, 'Importe facturacion filt post'].sum() - business_df.loc[idx_new_d, 'Importe facturacion filt'].sum()
n_cli_new_d_p = business_df.loc[idx_new_d].loc[(business_df['Importe impagado filt post']<=0), 'Importe facturacion filt post'].sum() - \
                business_df.loc[idx_new_d].loc[(business_df['Importe impagado filt post']<=0), 'Importe facturacion filt'].sum()
n_cli_new_d_d = business_df.loc[idx_new_d].loc[(business_df['Importe impagado filt post']>0)&(business_df['Importe impagado +120 dias filt post']<=0), 'Importe facturacion filt post'].sum() - \
                business_df.loc[idx_new_d].loc[(business_df['Importe impagado filt post']>0)&(business_df['Importe impagado +120 dias filt post']<=0), 'Importe facturacion filt'].sum()
n_cli_new_d_m = business_df.loc[idx_new_d].loc[(business_df['Importe impagado +120 dias filt post']>0), 'Importe facturacion filt post'].sum() - \
                business_df.loc[idx_new_d].loc[(business_df['Importe impagado +120 dias filt post']>0), 'Importe facturacion filt'].sum()
print('Impagadores con +6 meses son pagadores: {0:.0f}€'.format(n_cli_new_d_p))
print('Impagadores con +6 meses son impagadores: {0:.0f}€'.format(n_cli_new_d_d))
print('Impagadores con +6 meses son morosos: {0:.0f}€'.format(n_cli_new_d_m))
print('Check sobre la suma: {0}\n'.format(abs(fact_cli_new_d - (n_cli_new_d_p+n_cli_new_d_d+n_cli_new_d_m)) < 10**-6))


# Para idx_old_m sacamos total facturado de pagadores, impagadores y morosos. NOS DA IGUAL SI ESTÁN VIVOS O NO, ANALIZAMOS TODO LO OCURRIDO PARA LOS CLIENTES VIVOS AL FINAL DEL PERIODO LOS 12 MESES ANTERIORES Y DE AHÍ EN ADELANTE
fact_cli_old_m = business_df.loc[idx_old_m, 'Importe facturacion filt post'].sum() - business_df.loc[idx_old_m, 'Importe facturacion filt'].sum()
n_cli_old_m_p = business_df.loc[idx_old_m].loc[(business_df['Importe impagado filt post']<=0), 'Importe facturacion filt post'].sum() - \
                business_df.loc[idx_old_m].loc[(business_df['Importe impagado filt post']<=0), 'Importe facturacion filt'].sum()
n_cli_old_m_d = business_df.loc[idx_old_m].loc[(business_df['Importe impagado filt post']>0)&(business_df['Importe impagado +120 dias filt post']<=0), 'Importe facturacion filt post'].sum() - \
                business_df.loc[idx_old_m].loc[(business_df['Importe impagado filt post']>0)&(business_df['Importe impagado +120 dias filt post']<=0), 'Importe facturacion filt'].sum()
n_cli_old_m_m = business_df.loc[idx_old_m].loc[(business_df['Importe impagado +120 dias filt post']>0), 'Importe facturacion filt post'].sum() - \
                business_df.loc[idx_old_m].loc[(business_df['Importe impagado +120 dias filt post']>0), 'Importe facturacion filt'].sum()
print('Morosos con +6 meses son pagadores: {0:.0f}€'.format(n_cli_old_m_p))
print('Morosos con +6 meses son impagadores: {0:.0f}€'.format(n_cli_old_m_d))
print('Morosos con +6 meses son morosos: {0:.0f}€'.format(n_cli_old_m_m))
print('Check sobre la suma: {0}\n'.format(abs(fact_cli_old_m - (n_cli_old_m_p+n_cli_old_m_d+n_cli_old_m_m)) < 10**-6))

# Para idx_new_m sacamos total facturado de pagadores, impagadores y morosos. NOS DA IGUAL SI ESTÁN VIVOS O NO, ANALIZAMOS TODO LO OCURRIDO PARA LOS CLIENTES VIVOS AL FINAL DEL PERIODO LOS 12 MESES ANTERIORES Y DE AHÍ EN ADELANTE
fact_cli_new_m = business_df.loc[idx_new_m, 'Importe facturacion filt post'].sum() - business_df.loc[idx_new_m, 'Importe facturacion filt'].sum()
n_cli_new_m_p = business_df.loc[idx_new_m].loc[(business_df['Importe impagado filt post']<=0), 'Importe facturacion filt post'].sum() - \
                business_df.loc[idx_new_m].loc[(business_df['Importe impagado filt post']<=0), 'Importe facturacion filt'].sum()
n_cli_new_m_d = business_df.loc[idx_new_m].loc[(business_df['Importe impagado filt post']>0)&(business_df['Importe impagado +120 dias filt post']<=0), 'Importe facturacion filt post'].sum() - \
                business_df.loc[idx_new_m].loc[(business_df['Importe impagado filt post']>0)&(business_df['Importe impagado +120 dias filt post']<=0), 'Importe facturacion filt'].sum()
n_cli_new_m_m = business_df.loc[idx_new_m].loc[(business_df['Importe impagado +120 dias filt post']>0), 'Importe facturacion filt post'].sum() - \
                business_df.loc[idx_new_m].loc[(business_df['Importe impagado +120 dias filt post']>0), 'Importe facturacion filt'].sum()
print('Morosos con +6 meses son pagadores: {0:.0f}€'.format(n_cli_new_m_p))
print('Morosos con +6 meses son impagadores: {0:.0f}€'.format(n_cli_new_m_d))
print('Morosos con +6 meses son morosos: {0:.0f}€'.format(n_cli_new_m_m))
print('Check sobre la suma: {0}\n'.format(abs(fact_cli_new_m - (n_cli_new_m_p+n_cli_new_m_d+n_cli_new_m_m)) < 10**-6))

lst_fact_cli_p = [n_cli_old_p_p, n_cli_old_d_p, n_cli_old_m_p, n_cli_new_p_p, n_cli_new_d_p, n_cli_new_m_p]
print(lst_fact_cli_p, '\n')

lst_fact_cli_d = [n_cli_old_p_d, n_cli_old_d_d, n_cli_old_m_d, n_cli_new_p_d, n_cli_new_d_d, n_cli_new_m_d]
print(lst_fact_cli_d, '\n')

lst_fact_cli_m = [n_cli_old_p_m, n_cli_old_d_m, n_cli_old_m_m, n_cli_new_p_m, n_cli_new_d_m, n_cli_new_m_m]
print(lst_fact_cli_m, '\n')

Pagadores con +6 meses son pagadores: 420939€
Pagadores con +6 meses son impagadores: 81291€
Pagadores con +6 meses son morosos: 12298€
Check sobre la suma: True

Pagadores con +6 meses son pagadores: 46174514€
Pagadores con +6 meses son impagadores: 81291€
Pagadores con +6 meses son morosos: 4523736€
Check sobre la suma: True

Impagadores con +6 meses son pagadores: 0€
Impagadores con +6 meses son impagadores: 703495€
Impagadores con +6 meses son morosos: 191512€
Check sobre la suma: True

Impagadores con +6 meses son pagadores: 0€
Impagadores con +6 meses son impagadores: 26304971€
Impagadores con +6 meses son morosos: 8275141€
Check sobre la suma: True

Morosos con +6 meses son pagadores: 0€
Morosos con +6 meses son impagadores: 0€
Morosos con +6 meses son morosos: 380017€
Check sobre la suma: True

Morosos con +6 meses son pagadores: 0€
Morosos con +6 meses son impagadores: 0€
Morosos con +6 meses son morosos: 5741982€
Check sobre la suma: True

[420938.9099999999, 0.0, 0.0, 461745

#### Impago

In [82]:
#next_month = '2022-09'

# Para idx_old_p sacamos total facturado de pagadores, impagadores y morosos. NOS DA IGUAL SI ESTÁN VIVOS O NO, ANALIZAMOS TODO LO OCURRIDO PARA LOS CLIENTES VIVOS AL FINAL DEL PERIODO LOS 12 MESES ANTERIORES Y DE AHÍ EN ADELANTE
impa_cli_old_p = business_df.loc[idx_old_p, 'Importe impagado filt post'].sum() - business_df.loc[idx_old_p, 'Importe impagado filt'].sum()
n_cli_old_p_p = business_df.loc[idx_old_p].loc[(business_df['Importe impagado filt post']<=0), 'Importe impagado filt post'].sum() - \
                business_df.loc[idx_old_p].loc[(business_df['Importe impagado filt post']<=0), 'Importe impagado filt'].sum()
n_cli_old_p_d = business_df.loc[idx_old_p].loc[(business_df['Importe impagado filt post']>0)&(business_df['Importe impagado +120 dias filt post']<=0), 'Importe impagado filt post'].sum() - \
                business_df.loc[idx_old_p].loc[(business_df['Importe impagado filt post']>0)&(business_df['Importe impagado +120 dias filt post']<=0), 'Importe impagado filt'].sum()
n_cli_old_p_m = business_df.loc[idx_old_p].loc[(business_df['Importe impagado +120 dias filt post']>0), 'Importe impagado filt post'].sum() - \
                business_df.loc[idx_old_p].loc[(business_df['Importe impagado +120 dias filt post']>0), 'Importe impagado filt'].sum()
print('Pagadores con +6 meses son pagadores: {0:.0f}€'.format(n_cli_old_p_p))
print('Pagadores con +6 meses son impagadores: {0:.0f}€'.format(n_cli_old_p_d))
print('Pagadores con +6 meses son morosos: {0:.0f}€'.format(n_cli_old_p_m))
print('Check sobre la suma: {0}\n'.format(abs(impa_cli_old_p - (n_cli_old_p_p+n_cli_old_p_d+n_cli_old_p_m)) < 10**-6))

# Para idx_new_p sacamos total facturado de pagadores, impagadores y morosos. NOS DA IGUAL SI ESTÁN VIVOS O NO, ANALIZAMOS TODO LO OCURRIDO PARA LOS CLIENTES VIVOS AL FINAL DEL PERIODO LOS 12 MESES ANTERIORES Y DE AHÍ EN ADELANTE
impa_cli_new_p = business_df.loc[idx_new_p, 'Importe impagado filt post'].sum() - business_df.loc[idx_new_p, 'Importe impagado filt'].sum()
n_cli_new_p_p = business_df.loc[idx_new_p].loc[(business_df['Importe impagado filt post']<=0), 'Importe impagado filt post'].sum() - \
                business_df.loc[idx_new_p].loc[(business_df['Importe impagado filt post']<=0), 'Importe impagado filt'].sum()
n_cli_new_p_d = business_df.loc[idx_new_p].loc[(business_df['Importe impagado filt post']>0)&(business_df['Importe impagado +120 dias filt post']<=0), 'Importe impagado filt post'].sum() - \
                business_df.loc[idx_new_p].loc[(business_df['Importe impagado filt post']>0)&(business_df['Importe impagado +120 dias filt post']<=0), 'Importe impagado filt'].sum()
n_cli_new_p_m = business_df.loc[idx_new_p].loc[(business_df['Importe impagado +120 dias filt post']>0), 'Importe impagado filt post'].sum() - \
                business_df.loc[idx_new_p].loc[(business_df['Importe impagado +120 dias filt post']>0), 'Importe impagado filt'].sum()
print('Pagadores con +6 meses son pagadores: {0:.0f}€'.format(n_cli_new_p_p))
print('Pagadores con +6 meses son impagadores: {0:.0f}€'.format(n_cli_old_p_d))
print('Pagadores con +6 meses son morosos: {0:.0f}€'.format(n_cli_new_p_m))
print('Check sobre la suma: {0}\n'.format(abs(impa_cli_new_p - (n_cli_new_p_p+n_cli_new_p_d+n_cli_new_p_m)) < 10**-6))


# Para idx_old_d sacamos total facturado de pagadores, impagadores y morosos. NOS DA IGUAL SI ESTÁN VIVOS O NO, ANALIZAMOS TODO LO OCURRIDO PARA LOS CLIENTES VIVOS AL FINAL DEL PERIODO LOS 12 MESES ANTERIORES Y DE AHÍ EN ADELANTE
impa_cli_old_d = business_df.loc[idx_old_d, 'Importe impagado filt post'].sum() - business_df.loc[idx_old_d, 'Importe impagado filt'].sum()
n_cli_old_d_p = business_df.loc[idx_old_d].loc[(business_df['Importe impagado filt post']<=0), 'Importe impagado filt post'].sum() - \
                business_df.loc[idx_old_d].loc[(business_df['Importe impagado filt post']<=0), 'Importe impagado filt'].sum()
n_cli_old_d_d = business_df.loc[idx_old_d].loc[(business_df['Importe impagado filt post']>0)&(business_df['Importe impagado +120 dias filt post']<=0), 'Importe impagado filt post'].sum() - \
                business_df.loc[idx_old_d].loc[(business_df['Importe impagado filt post']>0)&(business_df['Importe impagado +120 dias filt post']<=0), 'Importe impagado filt'].sum()
n_cli_old_d_m = business_df.loc[idx_old_d].loc[(business_df['Importe impagado +120 dias filt post']>0), 'Importe impagado filt post'].sum() - \
                business_df.loc[idx_old_d].loc[(business_df['Importe impagado +120 dias filt post']>0), 'Importe impagado filt'].sum()
print('Impagadores con +6 meses son pagadores: {0:.0f}€'.format(n_cli_old_d_p))
print('Impagadores con +6 meses son impagadores: {0:.0f}€'.format(n_cli_old_d_d))
print('Impagadores con +6 meses son morosos: {0:.0f}€'.format(n_cli_old_d_m))
print('Check sobre la suma: {0}\n'.format(abs(impa_cli_old_d - (n_cli_old_d_p+n_cli_old_d_d+n_cli_old_d_m)) < 10**-6))

# Para idx_new_d sacamos total facturado de pagadores, impagadores y morosos. NOS DA IGUAL SI ESTÁN VIVOS O NO, ANALIZAMOS TODO LO OCURRIDO PARA LOS CLIENTES VIVOS AL FINAL DEL PERIODO LOS 12 MESES ANTERIORES Y DE AHÍ EN ADELANTE
impa_cli_new_d = business_df.loc[idx_new_d, 'Importe impagado filt post'].sum() - business_df.loc[idx_new_d, 'Importe impagado filt'].sum()
n_cli_new_d_p = business_df.loc[idx_new_d].loc[(business_df['Importe impagado filt post']<=0), 'Importe impagado filt post'].sum() - \
                business_df.loc[idx_new_d].loc[(business_df['Importe impagado filt post']<=0), 'Importe impagado filt'].sum()
n_cli_new_d_d = business_df.loc[idx_new_d].loc[(business_df['Importe impagado filt post']>0)&(business_df['Importe impagado +120 dias filt post']<=0), 'Importe impagado filt post'].sum() - \
                business_df.loc[idx_new_d].loc[(business_df['Importe impagado filt post']>0)&(business_df['Importe impagado +120 dias filt post']<=0), 'Importe impagado filt'].sum()
n_cli_new_d_m = business_df.loc[idx_new_d].loc[(business_df['Importe impagado +120 dias filt post']>0), 'Importe impagado filt post'].sum() - \
                business_df.loc[idx_new_d].loc[(business_df['Importe impagado +120 dias filt post']>0), 'Importe impagado filt'].sum()
print('Impagadores con +6 meses son pagadores: {0:.0f}€'.format(n_cli_new_d_p))
print('Impagadores con +6 meses son impagadores: {0:.0f}€'.format(n_cli_new_d_d))
print('Impagadores con +6 meses son morosos: {0:.0f}€'.format(n_cli_new_d_m))
print('Check sobre la suma: {0}\n'.format(abs(impa_cli_new_d - (n_cli_new_d_p+n_cli_new_d_d+n_cli_new_d_m)) < 10**-6))


# Para idx_old_m sacamos total facturado de pagadores, impagadores y morosos. NOS DA IGUAL SI ESTÁN VIVOS O NO, ANALIZAMOS TODO LO OCURRIDO PARA LOS CLIENTES VIVOS AL FINAL DEL PERIODO LOS 12 MESES ANTERIORES Y DE AHÍ EN ADELANTE
impa_cli_old_m = business_df.loc[idx_old_m, 'Importe impagado filt post'].sum() - business_df.loc[idx_old_m, 'Importe impagado filt'].sum()
n_cli_old_m_p = business_df.loc[idx_old_m].loc[(business_df['Importe impagado filt post']<=0), 'Importe impagado filt post'].sum() - \
                business_df.loc[idx_old_m].loc[(business_df['Importe impagado filt post']<=0), 'Importe impagado filt'].sum()
n_cli_old_m_d = business_df.loc[idx_old_m].loc[(business_df['Importe impagado filt post']>0)&(business_df['Importe impagado +120 dias filt post']<=0), 'Importe impagado filt post'].sum() - \
                business_df.loc[idx_old_m].loc[(business_df['Importe impagado filt post']>0)&(business_df['Importe impagado +120 dias filt post']<=0), 'Importe impagado filt'].sum()
n_cli_old_m_m = business_df.loc[idx_old_m].loc[(business_df['Importe impagado +120 dias filt post']>0), 'Importe impagado filt post'].sum() -\
                business_df.loc[idx_old_m].loc[(business_df['Importe impagado +120 dias filt post']>0), 'Importe impagado filt'].sum()
print('Morosos con +6 meses son pagadores: {0:.0f}€'.format(n_cli_old_m_p))
print('Morosos con +6 meses son impagadores: {0:.0f}€'.format(n_cli_old_m_d))
print('Morosos con +6 meses son morosos: {0:.0f}€'.format(n_cli_old_m_m))
print('Check sobre la suma: {0}\n'.format(abs(impa_cli_old_m - (n_cli_old_m_p+n_cli_old_m_d+n_cli_old_m_m)) < 10**-6))

# Para idx_new_m sacamos total facturado de pagadores, impagadores y morosos. NOS DA IGUAL SI ESTÁN VIVOS O NO, ANALIZAMOS TODO LO OCURRIDO PARA LOS CLIENTES VIVOS AL FINAL DEL PERIODO LOS 12 MESES ANTERIORES Y DE AHÍ EN ADELANTE
impa_cli_new_m = business_df.loc[idx_new_m, 'Importe impagado filt post'].sum() - business_df.loc[idx_new_m, 'Importe impagado filt'].sum()
n_cli_new_m_p = business_df.loc[idx_new_m].loc[(business_df['Importe impagado filt post']<=0), 'Importe impagado filt post'].sum() - \
                business_df.loc[idx_new_m].loc[(business_df['Importe impagado filt post']<=0), 'Importe impagado filt'].sum()
n_cli_new_m_d = business_df.loc[idx_new_m].loc[(business_df['Importe impagado filt post']>0)&(business_df['Importe impagado +120 dias filt post']<=0), 'Importe impagado filt post'].sum() - \
                business_df.loc[idx_new_m].loc[(business_df['Importe impagado filt post']>0)&(business_df['Importe impagado +120 dias filt post']<=0), 'Importe impagado filt'].sum()
n_cli_new_m_m = business_df.loc[idx_new_m].loc[(business_df['Importe impagado +120 dias filt post']>0), 'Importe impagado filt post'].sum() - \
                business_df.loc[idx_new_m].loc[(business_df['Importe impagado +120 dias filt post']>0), 'Importe impagado filt'].sum()
print('Morosos con +6 meses son pagadores: {0:.0f}€'.format(n_cli_new_m_p))
print('Morosos con +6 meses son impagadores: {0:.0f}€'.format(n_cli_new_m_d))
print('Morosos con +6 meses son morosos: {0:.0f}€'.format(n_cli_new_m_m))
print('Check sobre la suma: {0}\n'.format(abs(impa_cli_new_m - (n_cli_new_m_p+n_cli_new_m_d+n_cli_new_m_m)) < 10**-6))

lst_impa_cli_p = [n_cli_old_p_p, n_cli_old_d_p, n_cli_old_m_p, n_cli_new_p_p, n_cli_new_d_p, n_cli_new_m_p]
print(lst_impa_cli_p, '\n')

lst_impa_cli_d = [n_cli_old_p_d, n_cli_old_d_d, n_cli_old_m_d, n_cli_new_p_d, n_cli_new_d_d, n_cli_new_m_d]
print(lst_impa_cli_d, '\n')

lst_impa_cli_m = [n_cli_old_p_m,n_cli_old_d_m, n_cli_old_m_m, n_cli_new_p_m, n_cli_new_d_m, n_cli_new_m_m]
print(lst_impa_cli_m, '\n')

Pagadores con +6 meses son pagadores: 0€
Pagadores con +6 meses son impagadores: 29612€
Pagadores con +6 meses son morosos: 12298€
Check sobre la suma: True

Pagadores con +6 meses son pagadores: 0€
Pagadores con +6 meses son impagadores: 29612€
Pagadores con +6 meses son morosos: 3563193€
Check sobre la suma: True

Impagadores con +6 meses son pagadores: 0€
Impagadores con +6 meses son impagadores: 161819€
Impagadores con +6 meses son morosos: 151871€
Check sobre la suma: True

Impagadores con +6 meses son pagadores: 0€
Impagadores con +6 meses son impagadores: 7015571€
Impagadores con +6 meses son morosos: 6495556€
Check sobre la suma: True

Morosos con +6 meses son pagadores: 0€
Morosos con +6 meses son impagadores: 0€
Morosos con +6 meses son morosos: 368979€
Check sobre la suma: True

Morosos con +6 meses son pagadores: 0€
Morosos con +6 meses son impagadores: 0€
Morosos con +6 meses son morosos: 5734488€
Check sobre la suma: True

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0] 

[29612.14, 16181

In [83]:
#next_month = '2022-09'

# Para idx_old_p sacamos total facturado de pagadores, impagadores y morosos. NOS DA IGUAL SI ESTÁN VIVOS O NO, ANALIZAMOS TODO LO OCURRIDO PARA LOS CLIENTES VIVOS AL FINAL DEL PERIODO LOS 12 MESES ANTERIORES Y DE AHÍ EN ADELANTE
rec_impa_cli_old_p = business_df.loc[idx_old_p, 'Importe recuperado filt post'].sum() - business_df.loc[idx_old_p, 'Importe recuperado filt'].sum()
n_cli_old_p_p = business_df.loc[idx_old_p].loc[(business_df['Importe impagado filt post']<=0), 'Importe recuperado filt post'].sum() - \
                business_df.loc[idx_old_p].loc[(business_df['Importe impagado filt post']<=0), 'Importe recuperado filt'].sum()
n_cli_old_p_d = business_df.loc[idx_old_p].loc[(business_df['Importe impagado filt post']>0)&(business_df['Importe impagado +120 dias filt post']<=0), 'Importe recuperado filt post'].sum() - \
                business_df.loc[idx_old_p].loc[(business_df['Importe impagado filt post']>0)&(business_df['Importe impagado +120 dias filt post']<=0), 'Importe recuperado filt'].sum()
n_cli_old_p_m = business_df.loc[idx_old_p].loc[(business_df['Importe impagado +120 dias filt post']>0), 'Importe recuperado filt post'].sum() - \
                business_df.loc[idx_old_p].loc[(business_df['Importe impagado +120 dias filt post']>0), 'Importe recuperado filt'].sum()
print('Pagadores con +6 meses son pagadores: {0:.0f}€'.format(n_cli_old_p_p))
print('Pagadores con +6 meses son impagadores: {0:.0f}€'.format(n_cli_old_p_d))
print('Pagadores con +6 meses son morosos: {0:.0f}€'.format(n_cli_old_p_m))
print('Check sobre la suma: {0}\n'.format(abs(rec_impa_cli_old_p - (n_cli_old_p_p+n_cli_old_p_d+n_cli_old_p_m)) < 10**-6))

# Para idx_new_p sacamos total facturado de pagadores, impagadores y morosos. NOS DA IGUAL SI ESTÁN VIVOS O NO, ANALIZAMOS TODO LO OCURRIDO PARA LOS CLIENTES VIVOS AL FINAL DEL PERIODO LOS 12 MESES ANTERIORES Y DE AHÍ EN ADELANTE
rec_impa_cli_new_p = business_df.loc[idx_new_p, 'Importe recuperado filt post'].sum() - business_df.loc[idx_new_p, 'Importe recuperado filt'].sum()
n_cli_new_p_p = business_df.loc[idx_new_p].loc[(business_df['Importe impagado filt post']<=0), 'Importe recuperado filt post'].sum() - \
                business_df.loc[idx_new_p].loc[(business_df['Importe impagado filt post']<=0), 'Importe recuperado filt'].sum()
n_cli_new_p_d = business_df.loc[idx_new_p].loc[(business_df['Importe impagado filt post']>0)&(business_df['Importe impagado +120 dias filt post']<=0), 'Importe recuperado filt post'].sum() - \
                business_df.loc[idx_new_p].loc[(business_df['Importe impagado filt post']>0)&(business_df['Importe impagado +120 dias filt post']<=0), 'Importe recuperado filt'].sum()
n_cli_new_p_m = business_df.loc[idx_new_p].loc[(business_df['Importe impagado +120 dias filt post']>0), 'Importe recuperado filt post'].sum() - \
                business_df.loc[idx_new_p].loc[(business_df['Importe impagado +120 dias filt post']>0), 'Importe recuperado filt'].sum()
print('Pagadores con +6 meses son pagadores: {0:.0f}€'.format(n_cli_new_p_p))
print('Pagadores con +6 meses son impagadores: {0:.0f}€'.format(n_cli_old_p_d))
print('Pagadores con +6 meses son morosos: {0:.0f}€'.format(n_cli_new_p_m))
print('Check sobre la suma: {0}\n'.format(abs(rec_impa_cli_new_p - (n_cli_new_p_p+n_cli_new_p_d+n_cli_new_p_m)) < 10**-6))


# Para idx_old_d sacamos total facturado de pagadores, impagadores y morosos. NOS DA IGUAL SI ESTÁN VIVOS O NO, ANALIZAMOS TODO LO OCURRIDO PARA LOS CLIENTES VIVOS AL FINAL DEL PERIODO LOS 12 MESES ANTERIORES Y DE AHÍ EN ADELANTE
rec_impa_cli_old_d = business_df.loc[idx_old_d, 'Importe recuperado filt post'].sum() - business_df.loc[idx_old_d, 'Importe recuperado filt'].sum()
n_cli_old_d_p = business_df.loc[idx_old_d].loc[(business_df['Importe impagado filt post']<=0), 'Importe recuperado filt post'].sum() - \
                business_df.loc[idx_old_d].loc[(business_df['Importe impagado filt post']<=0), 'Importe recuperado filt'].sum()
n_cli_old_d_d = business_df.loc[idx_old_d].loc[(business_df['Importe impagado filt post']>0)&(business_df['Importe impagado +120 dias filt post']<=0), 'Importe recuperado filt post'].sum() - \
                business_df.loc[idx_old_d].loc[(business_df['Importe impagado filt post']>0)&(business_df['Importe impagado +120 dias filt post']<=0), 'Importe recuperado filt'].sum()
n_cli_old_d_m = business_df.loc[idx_old_d].loc[(business_df['Importe impagado +120 dias filt post']>0), 'Importe recuperado filt post'].sum() - \
                business_df.loc[idx_old_d].loc[(business_df['Importe impagado +120 dias filt post']>0), 'Importe recuperado filt'].sum()
print('Impagadores con +6 meses son pagadores: {0:.0f}€'.format(n_cli_old_d_p))
print('Impagadores con +6 meses son impagadores: {0:.0f}€'.format(n_cli_old_d_d))
print('Impagadores con +6 meses son morosos: {0:.0f}€'.format(n_cli_old_d_m))
print('Check sobre la suma: {0}\n'.format(abs(rec_impa_cli_old_d - (n_cli_old_d_p+n_cli_old_d_d+n_cli_old_d_m)) < 10**-6))

# Para idx_new_d sacamos total facturado de pagadores, impagadores y morosos. NOS DA IGUAL SI ESTÁN VIVOS O NO, ANALIZAMOS TODO LO OCURRIDO PARA LOS CLIENTES VIVOS AL FINAL DEL PERIODO LOS 12 MESES ANTERIORES Y DE AHÍ EN ADELANTE
rec_impa_cli_new_d = business_df.loc[idx_new_d, 'Importe recuperado filt post'].sum() - business_df.loc[idx_new_d, 'Importe recuperado filt'].sum()
n_cli_new_d_p = business_df.loc[idx_new_d].loc[(business_df['Importe impagado filt post']<=0), 'Importe recuperado filt post'].sum() - \
                business_df.loc[idx_new_d].loc[(business_df['Importe impagado filt post']<=0), 'Importe recuperado filt'].sum()
n_cli_new_d_d = business_df.loc[idx_new_d].loc[(business_df['Importe impagado filt post']>0)&(business_df['Importe impagado +120 dias filt post']<=0), 'Importe recuperado filt post'].sum() - \
                business_df.loc[idx_new_d].loc[(business_df['Importe impagado filt post']>0)&(business_df['Importe impagado +120 dias filt post']<=0), 'Importe recuperado filt'].sum()
n_cli_new_d_m = business_df.loc[idx_new_d].loc[(business_df['Importe impagado +120 dias filt post']>0), 'Importe recuperado filt post'].sum() - \
                business_df.loc[idx_new_d].loc[(business_df['Importe impagado +120 dias filt post']>0), 'Importe recuperado filt'].sum()
print('Impagadores con +6 meses son pagadores: {0:.0f}€'.format(n_cli_new_d_p))
print('Impagadores con +6 meses son impagadores: {0:.0f}€'.format(n_cli_new_d_d))
print('Impagadores con +6 meses son morosos: {0:.0f}€'.format(n_cli_new_d_m))
print('Check sobre la suma: {0}\n'.format(abs(rec_impa_cli_new_d - (n_cli_new_d_p+n_cli_new_d_d+n_cli_new_d_m)) < 10**-6))


# Para idx_old_m sacamos total facturado de pagadores, impagadores y morosos. NOS DA IGUAL SI ESTÁN VIVOS O NO, ANALIZAMOS TODO LO OCURRIDO PARA LOS CLIENTES VIVOS AL FINAL DEL PERIODO LOS 12 MESES ANTERIORES Y DE AHÍ EN ADELANTE
rec_impa_cli_old_m = business_df.loc[idx_old_m, 'Importe recuperado filt post'].sum() - business_df.loc[idx_old_m, 'Importe recuperado filt'].sum()
n_cli_old_m_p = business_df.loc[idx_old_m].loc[(business_df['Importe impagado filt post']<=0), 'Importe recuperado filt post'].sum() - \
                business_df.loc[idx_old_m].loc[(business_df['Importe impagado filt post']<=0), 'Importe recuperado filt'].sum()
n_cli_old_m_d = business_df.loc[idx_old_m].loc[(business_df['Importe impagado filt post']>0)&(business_df['Importe impagado +120 dias filt post']<=0), 'Importe recuperado filt post'].sum() - \
                business_df.loc[idx_old_m].loc[(business_df['Importe impagado filt post']>0)&(business_df['Importe impagado +120 dias filt post']<=0), 'Importe recuperado filt'].sum()
n_cli_old_m_m = business_df.loc[idx_old_m].loc[(business_df['Importe impagado +120 dias filt post']>0), 'Importe recuperado filt post'].sum() - \
                business_df.loc[idx_old_m].loc[(business_df['Importe impagado +120 dias filt post']>0), 'Importe recuperado filt'].sum()
print('Morosos con +6 meses son pagadores: {0:.0f}€'.format(n_cli_old_m_p))
print('Morosos con +6 meses son impagadores: {0:.0f}€'.format(n_cli_old_m_d))
print('Morosos con +6 meses son morosos: {0:.0f}€'.format(n_cli_old_m_m))
print('Check sobre la suma: {0}\n'.format(abs(rec_impa_cli_old_m - (n_cli_old_m_p+n_cli_old_m_d+n_cli_old_m_m)) < 10**-6))

# Para idx_new_m sacamos total facturado de pagadores, impagadores y morosos. NOS DA IGUAL SI ESTÁN VIVOS O NO, ANALIZAMOS TODO LO OCURRIDO PARA LOS CLIENTES VIVOS AL FINAL DEL PERIODO LOS 12 MESES ANTERIORES Y DE AHÍ EN ADELANTE
rec_impa_cli_new_m = business_df.loc[idx_new_m, 'Importe recuperado filt post'].sum() - business_df.loc[idx_new_m, 'Importe recuperado filt'].sum()
n_cli_new_m_p = business_df.loc[idx_new_m].loc[(business_df['Importe impagado filt post']<=0), 'Importe recuperado filt post'].sum() - \
                business_df.loc[idx_new_m].loc[(business_df['Importe impagado filt post']<=0), 'Importe recuperado filt'].sum()
n_cli_new_m_d = business_df.loc[idx_new_m].loc[(business_df['Importe impagado filt post']>0)&(business_df['Importe impagado +120 dias filt post']<=0), 'Importe recuperado filt post'].sum() - \
                business_df.loc[idx_new_m].loc[(business_df['Importe impagado filt post']>0)&(business_df['Importe impagado +120 dias filt post']<=0), 'Importe recuperado filt'].sum()
n_cli_new_m_m = business_df.loc[idx_new_m].loc[(business_df['Importe impagado +120 dias filt post']>0), 'Importe recuperado filt post'].sum() - \
                business_df.loc[idx_new_m].loc[(business_df['Importe impagado +120 dias filt post']>0), 'Importe recuperado filt'].sum()
print('Morosos con +6 meses son pagadores: {0:.0f}€'.format(n_cli_new_m_p))
print('Morosos con +6 meses son impagadores: {0:.0f}€'.format(n_cli_new_m_d))
print('Morosos con +6 meses son morosos: {0:.0f}€'.format(n_cli_new_m_m))
print('Check sobre la suma: {0}\n'.format(abs(rec_impa_cli_new_m - (n_cli_new_m_p+n_cli_new_m_d+n_cli_new_m_m)) < 10**-6))

lst_rec_impa_cli_p = [n_cli_old_p_p, n_cli_old_d_p, n_cli_old_m_p, n_cli_new_p_p, n_cli_new_d_p, n_cli_new_m_p]
print(lst_rec_impa_cli_p, '\n')

lst_rec_impa_cli_d = [n_cli_old_p_d, n_cli_old_d_d, n_cli_old_m_d, n_cli_new_p_d, n_cli_new_d_d, n_cli_new_m_d]
print(lst_rec_impa_cli_d, '\n')

lst_rec_impa_cli_m = [n_cli_old_p_m, n_cli_old_d_m, n_cli_old_m_m, n_cli_new_p_m, n_cli_new_d_m, n_cli_new_m_m]
print(lst_rec_impa_cli_m, '\n')

Pagadores con +6 meses son pagadores: 0€
Pagadores con +6 meses son impagadores: 29612€
Pagadores con +6 meses son morosos: 2244€
Check sobre la suma: True

Pagadores con +6 meses son pagadores: 0€
Pagadores con +6 meses son impagadores: 29612€
Pagadores con +6 meses son morosos: 1004989€
Check sobre la suma: True

Impagadores con +6 meses son pagadores: 0€
Impagadores con +6 meses son impagadores: 161864€
Impagadores con +6 meses son morosos: 42420€
Check sobre la suma: True

Impagadores con +6 meses son pagadores: 0€
Impagadores con +6 meses son impagadores: 7022662€
Impagadores con +6 meses son morosos: 1426750€
Check sobre la suma: True

Morosos con +6 meses son pagadores: 0€
Morosos con +6 meses son impagadores: 0€
Morosos con +6 meses son morosos: 30822€
Check sobre la suma: True

Morosos con +6 meses son pagadores: 0€
Morosos con +6 meses son impagadores: 0€
Morosos con +6 meses son morosos: 32053€
Check sobre la suma: True

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0] 

[29612.14, 161864.179

#### Mora

In [84]:
#next_month = '2022-09'

# Para idx_old_p sacamos total facturado de pagadores, impagadores y morosos. NOS DA IGUAL SI ESTÁN VIVOS O NO, ANALIZAMOS TODO LO OCURRIDO PARA LOS CLIENTES VIVOS AL FINAL DEL PERIODO LOS 12 MESES ANTERIORES Y DE AHÍ EN ADELANTE
mora_cli_old_p = business_df.loc[idx_old_p, 'Importe impagado +120 dias filt post'].sum() - business_df.loc[idx_old_p, 'Importe impagado +120 dias filt'].sum()
n_cli_old_p_p = business_df.loc[idx_old_p].loc[(business_df['Importe impagado filt post']<=0), 'Importe impagado +120 dias filt post'].sum() - \
                business_df.loc[idx_old_p].loc[(business_df['Importe impagado filt post']<=0), 'Importe impagado +120 dias filt'].sum()
n_cli_old_p_d = business_df.loc[idx_old_p].loc[(business_df['Importe impagado filt post']>0)&(business_df['Importe impagado +120 dias filt post']<=0), 'Importe impagado +120 dias filt post'].sum() - \
                business_df.loc[idx_old_p].loc[(business_df['Importe impagado filt post']>0)&(business_df['Importe impagado +120 dias filt post']<=0), 'Importe impagado +120 dias filt'].sum()
n_cli_old_p_m = business_df.loc[idx_old_p].loc[(business_df['Importe impagado +120 dias filt post']>0), 'Importe impagado +120 dias filt post'].sum() - \
                business_df.loc[idx_old_p].loc[(business_df['Importe impagado +120 dias filt post']>0), 'Importe impagado +120 dias filt'].sum()
print('Pagadores con +6 meses son pagadores: {0:.0f}€'.format(n_cli_old_p_p))
print('Pagadores con +6 meses son impagadores: {0:.0f}€'.format(n_cli_old_p_d))
print('Pagadores con +6 meses son morosos: {0:.0f}€'.format(n_cli_old_p_m))
print('Check sobre la suma: {0}\n'.format(abs(mora_cli_old_p - (n_cli_old_p_p+n_cli_old_p_d+n_cli_old_p_m)) < 10**-6))

# Para idx_new_p sacamos total facturado de pagadores, impagadores y morosos. NOS DA IGUAL SI ESTÁN VIVOS O NO, ANALIZAMOS TODO LO OCURRIDO PARA LOS CLIENTES VIVOS AL FINAL DEL PERIODO LOS 12 MESES ANTERIORES Y DE AHÍ EN ADELANTE
mora_cli_new_p = business_df.loc[idx_new_p, 'Importe impagado +120 dias filt post'].sum() - business_df.loc[idx_new_p, 'Importe impagado +120 dias filt'].sum()
n_cli_new_p_p = business_df.loc[idx_new_p].loc[(business_df['Importe impagado filt post']<=0), 'Importe impagado +120 dias filt post'].sum() - \
                business_df.loc[idx_new_p].loc[(business_df['Importe impagado filt post']<=0), 'Importe impagado +120 dias filt'].sum()
n_cli_new_p_d = business_df.loc[idx_new_p].loc[(business_df['Importe impagado filt post']>0)&(business_df['Importe impagado +120 dias filt post']<=0), 'Importe impagado +120 dias filt post'].sum() - \
                business_df.loc[idx_new_p].loc[(business_df['Importe impagado filt post']>0)&(business_df['Importe impagado +120 dias filt post']<=0), 'Importe impagado +120 dias filt'].sum()
n_cli_new_p_m = business_df.loc[idx_new_p].loc[(business_df['Importe impagado +120 dias filt post']>0), 'Importe impagado +120 dias filt post'].sum() - \
                business_df.loc[idx_new_p].loc[(business_df['Importe impagado +120 dias filt post']>0), 'Importe impagado +120 dias filt'].sum()
print('Pagadores con +6 meses son pagadores: {0:.0f}€'.format(n_cli_new_p_p))
print('Pagadores con +6 meses son impagadores: {0:.0f}€'.format(n_cli_old_p_d))
print('Pagadores con +6 meses son morosos: {0:.0f}€'.format(n_cli_new_p_m))
print('Check sobre la suma: {0}\n'.format(abs(mora_cli_new_p - (n_cli_new_p_p+n_cli_new_p_d+n_cli_new_p_m)) < 10**-6))


# Para idx_old_d sacamos total facturado de pagadores, impagadores y morosos. NOS DA IGUAL SI ESTÁN VIVOS O NO, ANALIZAMOS TODO LO OCURRIDO PARA LOS CLIENTES VIVOS AL FINAL DEL PERIODO LOS 12 MESES ANTERIORES Y DE AHÍ EN ADELANTE
mora_cli_old_d = business_df.loc[idx_old_d, 'Importe impagado +120 dias filt post'].sum() - business_df.loc[idx_old_d, 'Importe impagado +120 dias filt'].sum()
n_cli_old_d_p = business_df.loc[idx_old_d].loc[(business_df['Importe impagado filt post']<=0), 'Importe impagado +120 dias filt post'].sum() - \
                business_df.loc[idx_old_d].loc[(business_df['Importe impagado filt post']<=0), 'Importe impagado +120 dias filt'].sum()
n_cli_old_d_d = business_df.loc[idx_old_d].loc[(business_df['Importe impagado filt post']>0)&(business_df['Importe impagado +120 dias filt post']<=0), 'Importe impagado +120 dias filt post'].sum() - \
                business_df.loc[idx_old_d].loc[(business_df['Importe impagado filt post']>0)&(business_df['Importe impagado +120 dias filt post']<=0), 'Importe impagado +120 dias filt'].sum()
n_cli_old_d_m = business_df.loc[idx_old_d].loc[(business_df['Importe impagado +120 dias filt post']>0), 'Importe impagado +120 dias filt post'].sum() - \
                business_df.loc[idx_old_d].loc[(business_df['Importe impagado +120 dias filt post']>0), 'Importe impagado +120 dias filt'].sum()
print('Impagadores con +6 meses son pagadores: {0:.0f}€'.format(n_cli_old_d_p))
print('Impagadores con +6 meses son impagadores: {0:.0f}€'.format(n_cli_old_d_d))
print('Impagadores con +6 meses son morosos: {0:.0f}€'.format(n_cli_old_d_m))
print('Check sobre la suma: {0}\n'.format(abs(mora_cli_old_d - (n_cli_old_d_p+n_cli_old_d_d+n_cli_old_d_m)) < 10**-6))

# Para idx_new_d sacamos total facturado de pagadores, impagadores y morosos. NOS DA IGUAL SI ESTÁN VIVOS O NO, ANALIZAMOS TODO LO OCURRIDO PARA LOS CLIENTES VIVOS AL FINAL DEL PERIODO LOS 12 MESES ANTERIORES Y DE AHÍ EN ADELANTE
mora_cli_new_d = business_df.loc[idx_new_d, 'Importe impagado +120 dias filt post'].sum() - business_df.loc[idx_new_d, 'Importe impagado +120 dias filt'].sum()
n_cli_new_d_p = business_df.loc[idx_new_d].loc[(business_df['Importe impagado filt post']<=0), 'Importe impagado +120 dias filt post'].sum() - \
                business_df.loc[idx_new_d].loc[(business_df['Importe impagado filt post']<=0), 'Importe impagado +120 dias filt'].sum()
n_cli_new_d_d = business_df.loc[idx_new_d].loc[(business_df['Importe impagado filt post']>0)&(business_df['Importe impagado +120 dias filt post']<=0), 'Importe impagado +120 dias filt post'].sum() - \
                business_df.loc[idx_new_d].loc[(business_df['Importe impagado filt post']>0)&(business_df['Importe impagado +120 dias filt post']<=0), 'Importe impagado +120 dias filt'].sum()
n_cli_new_d_m = business_df.loc[idx_new_d].loc[(business_df['Importe impagado +120 dias filt post']>0), 'Importe impagado +120 dias filt post'].sum() - \
                business_df.loc[idx_new_d].loc[(business_df['Importe impagado +120 dias filt post']>0), 'Importe impagado +120 dias filt'].sum()
print('Impagadores con +6 meses son pagadores: {0:.0f}€'.format(n_cli_new_d_p))
print('Impagadores con +6 meses son impagadores: {0:.0f}€'.format(n_cli_new_d_d))
print('Impagadores con +6 meses son morosos: {0:.0f}€'.format(n_cli_new_d_m))
print('Check sobre la suma: {0}\n'.format(abs(mora_cli_new_d - (n_cli_new_d_p+n_cli_new_d_d+n_cli_new_d_m)) < 10**-6))


# Para idx_old_m sacamos total facturado de pagadores, impagadores y morosos. NOS DA IGUAL SI ESTÁN VIVOS O NO, ANALIZAMOS TODO LO OCURRIDO PARA LOS CLIENTES VIVOS AL FINAL DEL PERIODO LOS 12 MESES ANTERIORES Y DE AHÍ EN ADELANTE
mora_cli_old_m = business_df.loc[idx_old_m, 'Importe impagado +120 dias filt post'].sum() - business_df.loc[idx_old_m, 'Importe impagado +120 dias filt'].sum()
n_cli_old_m_p = business_df.loc[idx_old_m].loc[(business_df['Importe impagado filt post']<=0), 'Importe impagado +120 dias filt post'].sum() - \
                business_df.loc[idx_old_m].loc[(business_df['Importe impagado filt post']<=0), 'Importe impagado +120 dias filt'].sum()
n_cli_old_m_d = business_df.loc[idx_old_m].loc[(business_df['Importe impagado filt post']>0)&(business_df['Importe impagado +120 dias filt post']<=0), 'Importe impagado +120 dias filt post'].sum() - \
                business_df.loc[idx_old_m].loc[(business_df['Importe impagado filt post']>0)&(business_df['Importe impagado +120 dias filt post']<=0), 'Importe impagado +120 dias filt'].sum()
n_cli_old_m_m = business_df.loc[idx_old_m].loc[(business_df['Importe impagado +120 dias filt post']>0), 'Importe impagado +120 dias filt post'].sum() - \
                business_df.loc[idx_old_m].loc[(business_df['Importe impagado +120 dias filt post']>0), 'Importe impagado +120 dias filt'].sum()
print('Morosos con +6 meses son pagadores: {0:.0f}€'.format(n_cli_old_m_p))
print('Morosos con +6 meses son impagadores: {0:.0f}€'.format(n_cli_old_m_d))
print('Morosos con +6 meses son morosos: {0:.0f}€'.format(n_cli_old_m_m))
print('Check sobre la suma: {0}\n'.format(abs(mora_cli_old_m - (n_cli_old_m_p+n_cli_old_m_d+n_cli_old_m_m)) < 10**-6))

# Para idx_new_m sacamos total facturado de pagadores, impagadores y morosos. NOS DA IGUAL SI ESTÁN VIVOS O NO, ANALIZAMOS TODO LO OCURRIDO PARA LOS CLIENTES VIVOS AL FINAL DEL PERIODO LOS 12 MESES ANTERIORES Y DE AHÍ EN ADELANTE
mora_cli_new_m = business_df.loc[idx_new_m, 'Importe impagado +120 dias filt post'].sum() - business_df.loc[idx_new_m, 'Importe impagado +120 dias filt'].sum()
n_cli_new_m_p = business_df.loc[idx_new_m].loc[(business_df['Importe impagado filt post']<=0), 'Importe impagado +120 dias filt post'].sum() - \
                business_df.loc[idx_new_m].loc[(business_df['Importe impagado filt post']<=0), 'Importe impagado +120 dias filt'].sum()
n_cli_new_m_d = business_df.loc[idx_new_m].loc[(business_df['Importe impagado filt post']>0)&(business_df['Importe impagado +120 dias filt post']<=0), 'Importe impagado +120 dias filt post'].sum() - \
                business_df.loc[idx_new_m].loc[(business_df['Importe impagado filt post']>0)&(business_df['Importe impagado +120 dias filt post']<=0), 'Importe impagado +120 dias filt'].sum()
n_cli_new_m_m = business_df.loc[idx_new_m].loc[(business_df['Importe impagado +120 dias filt post']>0), 'Importe impagado +120 dias filt post'].sum() - \
                business_df.loc[idx_new_m].loc[(business_df['Importe impagado +120 dias filt post']>0), 'Importe impagado +120 dias filt'].sum()
print('Morosos con +6 meses son pagadores: {0:.0f}€'.format(n_cli_new_m_p))
print('Morosos con +6 meses son impagadores: {0:.0f}€'.format(n_cli_new_m_d))
print('Morosos con +6 meses son morosos: {0:.0f}€'.format(n_cli_new_m_m))
print('Check sobre la suma: {0}\n'.format(abs(mora_cli_new_m - (n_cli_new_m_p+n_cli_new_m_d+n_cli_new_m_m)) < 10**-6))

lst_mora_cli_p = [n_cli_old_p_p, n_cli_old_d_p, n_cli_old_m_p, n_cli_new_p_p, n_cli_new_d_p, n_cli_new_m_p]
print(lst_mora_cli_p, '\n')

lst_mora_cli_d = [n_cli_old_p_d, n_cli_old_d_d, n_cli_old_m_d, n_cli_new_p_d, n_cli_new_d_d, n_cli_new_m_d]
print(lst_mora_cli_d, '\n')

lst_mora_cli_m = [n_cli_old_p_m, n_cli_old_d_m, n_cli_old_m_m, n_cli_new_p_m, n_cli_new_d_m, n_cli_new_m_m]
print(lst_mora_cli_m, '\n')

Pagadores con +6 meses son pagadores: 0€
Pagadores con +6 meses son impagadores: 0€
Pagadores con +6 meses son morosos: 12298€
Check sobre la suma: True

Pagadores con +6 meses son pagadores: 0€
Pagadores con +6 meses son impagadores: 0€
Pagadores con +6 meses son morosos: 2766741€
Check sobre la suma: True

Impagadores con +6 meses son pagadores: 0€
Impagadores con +6 meses son impagadores: 0€
Impagadores con +6 meses son morosos: 134580€
Check sobre la suma: True

Impagadores con +6 meses son pagadores: 0€
Impagadores con +6 meses son impagadores: 0€
Impagadores con +6 meses son morosos: 5547465€
Check sobre la suma: True

Morosos con +6 meses son pagadores: 0€
Morosos con +6 meses son impagadores: 0€
Morosos con +6 meses son morosos: 344326€
Check sobre la suma: True

Morosos con +6 meses son pagadores: 0€
Morosos con +6 meses son impagadores: 0€
Morosos con +6 meses son morosos: 5717049€
Check sobre la suma: True

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0] 

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0] 

[1

In [85]:
#next_month = '2022-09'

# Para idx_old_p sacamos total facturado de pagadores, impagadores y morosos. NOS DA IGUAL SI ESTÁN VIVOS O NO, ANALIZAMOS TODO LO OCURRIDO PARA LOS CLIENTES VIVOS AL FINAL DEL PERIODO LOS 12 MESES ANTERIORES Y DE AHÍ EN ADELANTE
rec_mora_cli_old_p = business_df.loc[idx_old_p, 'Importe recuperado +120 dias filt post'].sum() - business_df.loc[idx_old_p, 'Importe recuperado +120 dias filt'].sum()
n_cli_old_p_p = business_df.loc[idx_old_p].loc[(business_df['Importe impagado filt post']<=0), 'Importe recuperado +120 dias filt post'].sum() - \
                business_df.loc[idx_old_p].loc[(business_df['Importe impagado filt post']<=0), 'Importe recuperado +120 dias filt'].sum()
n_cli_old_p_d = business_df.loc[idx_old_p].loc[(business_df['Importe impagado filt post']>0)&(business_df['Importe impagado +120 dias filt post']<=0), 'Importe recuperado +120 dias filt post'].sum() - \
                business_df.loc[idx_old_p].loc[(business_df['Importe impagado filt post']>0)&(business_df['Importe impagado +120 dias filt post']<=0), 'Importe recuperado +120 dias filt'].sum()
n_cli_old_p_m = business_df.loc[idx_old_p].loc[(business_df['Importe impagado +120 dias filt post']>0), 'Importe recuperado +120 dias filt post'].sum() - \
                business_df.loc[idx_old_p].loc[(business_df['Importe impagado +120 dias filt post']>0), 'Importe recuperado +120 dias filt'].sum()
print('Pagadores con +6 meses son pagadores: {0:.0f}€'.format(n_cli_old_p_p))
print('Pagadores con +6 meses son impagadores: {0:.0f}€'.format(n_cli_old_p_d))
print('Pagadores con +6 meses son morosos: {0:.0f}€'.format(n_cli_old_p_m))
print('Check sobre la suma: {0}\n'.format(abs(rec_mora_cli_old_p - (n_cli_old_p_p+n_cli_old_p_d+n_cli_old_p_m)) < 10**-6))

# Para idx_new_p sacamos total facturado de pagadores, impagadores y morosos. NOS DA IGUAL SI ESTÁN VIVOS O NO, ANALIZAMOS TODO LO OCURRIDO PARA LOS CLIENTES VIVOS AL FINAL DEL PERIODO LOS 12 MESES ANTERIORES Y DE AHÍ EN ADELANTE
rec_mora_cli_new_p = business_df.loc[idx_new_p, 'Importe recuperado +120 dias filt post'].sum() - business_df.loc[idx_new_p, 'Importe recuperado +120 dias filt'].sum()
n_cli_new_p_p = business_df.loc[idx_new_p].loc[(business_df['Importe impagado filt post']<=0), 'Importe recuperado +120 dias filt post'].sum() - \
                business_df.loc[idx_new_p].loc[(business_df['Importe impagado filt post']<=0), 'Importe recuperado +120 dias filt'].sum()
n_cli_new_p_d = business_df.loc[idx_new_p].loc[(business_df['Importe impagado filt post']>0)&(business_df['Importe impagado +120 dias filt post']<=0), 'Importe recuperado +120 dias filt post'].sum() - \
                business_df.loc[idx_new_p].loc[(business_df['Importe impagado filt post']>0)&(business_df['Importe impagado +120 dias filt post']<=0), 'Importe recuperado +120 dias filt'].sum()
n_cli_new_p_m = business_df.loc[idx_new_p].loc[(business_df['Importe impagado +120 dias filt post']>0), 'Importe recuperado +120 dias filt post'].sum() - \
                business_df.loc[idx_new_p].loc[(business_df['Importe impagado +120 dias filt post']>0), 'Importe recuperado +120 dias filt'].sum()
print('Pagadores con +6 meses son pagadores: {0:.0f}€'.format(n_cli_new_p_p))
print('Pagadores con +6 meses son impagadores: {0:.0f}€'.format(n_cli_old_p_d))
print('Pagadores con +6 meses son morosos: {0:.0f}€'.format(n_cli_new_p_m))
print('Check sobre la suma: {0}\n'.format(abs(rec_mora_cli_new_p - (n_cli_new_p_p+n_cli_new_p_d+n_cli_new_p_m)) < 10**-6))


# Para idx_old_d sacamos total facturado de pagadores, impagadores y morosos. NOS DA IGUAL SI ESTÁN VIVOS O NO, ANALIZAMOS TODO LO OCURRIDO PARA LOS CLIENTES VIVOS AL FINAL DEL PERIODO LOS 12 MESES ANTERIORES Y DE AHÍ EN ADELANTE
rec_mora_cli_old_d = business_df.loc[idx_old_d, 'Importe recuperado +120 dias filt post'].sum() - business_df.loc[idx_old_d, 'Importe recuperado +120 dias filt'].sum()
n_cli_old_d_p = business_df.loc[idx_old_d].loc[(business_df['Importe impagado filt post']<=0), 'Importe recuperado +120 dias filt post'].sum() - \
                business_df.loc[idx_old_d].loc[(business_df['Importe impagado filt post']<=0), 'Importe recuperado +120 dias filt'].sum()
n_cli_old_d_d = business_df.loc[idx_old_d].loc[(business_df['Importe impagado filt post']>0)&(business_df['Importe impagado +120 dias filt post']<=0), 'Importe recuperado +120 dias filt post'].sum() - \
                business_df.loc[idx_old_d].loc[(business_df['Importe impagado filt post']>0)&(business_df['Importe impagado +120 dias filt post']<=0), 'Importe recuperado +120 dias filt'].sum()
n_cli_old_d_m = business_df.loc[idx_old_d].loc[(business_df['Importe impagado +120 dias filt post']>0), 'Importe recuperado +120 dias filt post'].sum() - \
                business_df.loc[idx_old_d].loc[(business_df['Importe impagado +120 dias filt post']>0), 'Importe recuperado +120 dias filt'].sum()
print('Impagadores con +6 meses son pagadores: {0:.0f}€'.format(n_cli_old_d_p))
print('Impagadores con +6 meses son impagadores: {0:.0f}€'.format(n_cli_old_d_d))
print('Impagadores con +6 meses son morosos: {0:.0f}€'.format(n_cli_old_d_m))
print('Check sobre la suma: {0}\n'.format(abs(rec_mora_cli_old_d - (n_cli_old_d_p+n_cli_old_d_d+n_cli_old_d_m)) < 10**-6))

# Para idx_new_d sacamos total facturado de pagadores, impagadores y morosos. NOS DA IGUAL SI ESTÁN VIVOS O NO, ANALIZAMOS TODO LO OCURRIDO PARA LOS CLIENTES VIVOS AL FINAL DEL PERIODO LOS 12 MESES ANTERIORES Y DE AHÍ EN ADELANTE
rec_mora_cli_new_d = business_df.loc[idx_new_d, 'Importe recuperado +120 dias filt post'].sum() - business_df.loc[idx_new_d, 'Importe recuperado +120 dias filt'].sum()
n_cli_new_d_p = business_df.loc[idx_new_d].loc[(business_df['Importe impagado filt post']<=0), 'Importe recuperado +120 dias filt post'].sum() - \
                business_df.loc[idx_new_d].loc[(business_df['Importe impagado filt post']<=0), 'Importe recuperado +120 dias filt'].sum()
n_cli_new_d_d = business_df.loc[idx_new_d].loc[(business_df['Importe impagado filt post']>0)&(business_df['Importe impagado +120 dias filt post']<=0), 'Importe recuperado +120 dias filt post'].sum() - \
                business_df.loc[idx_new_d].loc[(business_df['Importe impagado filt post']>0)&(business_df['Importe impagado +120 dias filt post']<=0), 'Importe recuperado +120 dias filt'].sum()
n_cli_new_d_m = business_df.loc[idx_new_d].loc[(business_df['Importe impagado +120 dias filt post']>0), 'Importe recuperado +120 dias filt post'].sum() - \
                business_df.loc[idx_new_d].loc[(business_df['Importe impagado +120 dias filt post']>0), 'Importe recuperado +120 dias filt'].sum()
print('Impagadores con +6 meses son pagadores: {0:.0f}€'.format(n_cli_new_d_p))
print('Impagadores con +6 meses son impagadores: {0:.0f}€'.format(n_cli_new_d_d))
print('Impagadores con +6 meses son morosos: {0:.0f}€'.format(n_cli_new_d_m))
print('Check sobre la suma: {0}\n'.format(abs(rec_mora_cli_new_d - (n_cli_new_d_p+n_cli_new_d_d+n_cli_new_d_m)) < 10**-6))


# Para idx_old_m sacamos total facturado de pagadores, impagadores y morosos. NOS DA IGUAL SI ESTÁN VIVOS O NO, ANALIZAMOS TODO LO OCURRIDO PARA LOS CLIENTES VIVOS AL FINAL DEL PERIODO LOS 12 MESES ANTERIORES Y DE AHÍ EN ADELANTE
rec_mora_cli_old_m = business_df.loc[idx_old_m, 'Importe recuperado +120 dias filt post'].sum() - business_df.loc[idx_old_m, 'Importe recuperado +120 dias filt'].sum()
n_cli_old_m_p = business_df.loc[idx_old_m].loc[(business_df['Importe impagado filt post']<=0), 'Importe recuperado +120 dias filt post'].sum() - \
                business_df.loc[idx_old_m].loc[(business_df['Importe impagado filt post']<=0), 'Importe recuperado +120 dias filt'].sum()
n_cli_old_m_d = business_df.loc[idx_old_m].loc[(business_df['Importe impagado filt post']>0)&(business_df['Importe impagado +120 dias filt post']<=0), 'Importe recuperado +120 dias filt post'].sum() - \
                business_df.loc[idx_old_m].loc[(business_df['Importe impagado filt post']>0)&(business_df['Importe impagado +120 dias filt post']<=0), 'Importe recuperado +120 dias filt'].sum()
n_cli_old_m_m = business_df.loc[idx_old_m].loc[(business_df['Importe impagado +120 dias filt post']>0), 'Importe recuperado +120 dias filt post'].sum() - \
                business_df.loc[idx_old_m].loc[(business_df['Importe impagado +120 dias filt post']>0), 'Importe recuperado +120 dias filt'].sum()
print('Morosos con +6 meses son pagadores: {0:.0f}€'.format(n_cli_old_m_p))
print('Morosos con +6 meses son impagadores: {0:.0f}€'.format(n_cli_old_m_d))
print('Morosos con +6 meses son morosos: {0:.0f}€'.format(n_cli_old_m_m))
print('Check sobre la suma: {0}\n'.format(abs(rec_mora_cli_old_m - (n_cli_old_m_p+n_cli_old_m_d+n_cli_old_m_m)) < 10**-6))

# Para idx_new_m sacamos total facturado de pagadores, impagadores y morosos. NOS DA IGUAL SI ESTÁN VIVOS O NO, ANALIZAMOS TODO LO OCURRIDO PARA LOS CLIENTES VIVOS AL FINAL DEL PERIODO LOS 12 MESES ANTERIORES Y DE AHÍ EN ADELANTE
rec_mora_cli_new_m = business_df.loc[idx_new_m, 'Importe recuperado +120 dias filt post'].sum() - business_df.loc[idx_new_m, 'Importe recuperado +120 dias filt'].sum()
n_cli_new_m_p = business_df.loc[idx_new_m].loc[(business_df['Importe impagado filt post']<=0), 'Importe recuperado +120 dias filt post'].sum() - \
                business_df.loc[idx_new_m].loc[(business_df['Importe impagado filt post']<=0), 'Importe recuperado +120 dias filt'].sum()
n_cli_new_m_d = business_df.loc[idx_new_m].loc[(business_df['Importe impagado filt post']>0)&(business_df['Importe impagado +120 dias filt post']<=0), 'Importe recuperado +120 dias filt post'].sum() - \
                business_df.loc[idx_new_m].loc[(business_df['Importe impagado filt post']>0)&(business_df['Importe impagado +120 dias filt post']<=0), 'Importe recuperado +120 dias filt'].sum()
n_cli_new_m_m = business_df.loc[idx_new_m].loc[(business_df['Importe impagado +120 dias filt post']>0), 'Importe recuperado +120 dias filt post'].sum() - \
                business_df.loc[idx_new_m].loc[(business_df['Importe impagado +120 dias filt post']>0), 'Importe recuperado +120 dias filt'].sum()
print('Morosos con +6 meses son pagadores: {0:.0f}€'.format(n_cli_new_m_p))
print('Morosos con +6 meses son impagadores: {0:.0f}€'.format(n_cli_new_m_d))
print('Morosos con +6 meses son morosos: {0:.0f}€'.format(n_cli_new_m_m))
print('Check sobre la suma: {0}\n'.format(abs(rec_mora_cli_new_m - (n_cli_new_m_p+n_cli_new_m_d+n_cli_new_m_m)) < 10**-6))

lst_rec_mora_cli_p = [n_cli_old_p_p, n_cli_old_d_p, n_cli_old_m_p, n_cli_new_p_p, n_cli_new_d_p, n_cli_new_m_p]
print(lst_rec_mora_cli_p, '\n')

lst_rec_mora_cli_d = [n_cli_old_p_d, n_cli_old_d_d, n_cli_old_m_d, n_cli_new_p_d, n_cli_new_d_d, n_cli_new_m_d]
print(lst_rec_mora_cli_d, '\n')

lst_rec_mora_cli_m = [n_cli_old_p_m, n_cli_old_d_m, n_cli_old_m_m, n_cli_new_p_m, n_cli_new_d_m, n_cli_new_m_m]
print(lst_rec_mora_cli_m, '\n')

Pagadores con +6 meses son pagadores: 0€
Pagadores con +6 meses son impagadores: 0€
Pagadores con +6 meses son morosos: 2244€
Check sobre la suma: True

Pagadores con +6 meses son pagadores: 0€
Pagadores con +6 meses son impagadores: 0€
Pagadores con +6 meses son morosos: 208537€
Check sobre la suma: True

Impagadores con +6 meses son pagadores: 0€
Impagadores con +6 meses son impagadores: 0€
Impagadores con +6 meses son morosos: 25129€
Check sobre la suma: True

Impagadores con +6 meses son pagadores: 0€
Impagadores con +6 meses son impagadores: 0€
Impagadores con +6 meses son morosos: 478649€
Check sobre la suma: True

Morosos con +6 meses son pagadores: 0€
Morosos con +6 meses son impagadores: 0€
Morosos con +6 meses son morosos: 6168€
Check sobre la suma: True

Morosos con +6 meses son pagadores: 0€
Morosos con +6 meses son impagadores: 0€
Morosos con +6 meses son morosos: 14613€
Check sobre la suma: True

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0] 

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0] 

[2244.22, 

Creamos una tabla con toda la información post periodo de analisis

In [86]:
data[('Pagadores febrero 2023', '#')] = lst_n_cli_p
data[('Pagadores febrero 2023', '$ Pagado')] = 0
data[('Pagadores febrero 2023', '$ Impagado')] = lst_impa_cli_p
data[('Pagadores febrero 2023', '$ Impagado recuperado')] = lst_rec_impa_cli_p
data[('Pagadores febrero 2023', '$ Moroso')] = lst_mora_cli_p
data[('Pagadores febrero 2023', '$ Moroso recuperado')] = lst_rec_mora_cli_p
data[('Pagadores febrero 2023', '$ Total')] = lst_fact_cli_p

data[('Pagadores febrero 2023', '$ Impagado')] = data[('Pagadores febrero 2023', '$ Impagado')] - data[('Pagadores febrero 2023', '$ Moroso')]
data[('Pagadores febrero 2023', '$ Impagado recuperado')] = data[('Pagadores febrero 2023', '$ Impagado recuperado')] - data[('Pagadores febrero 2023', '$ Moroso recuperado')]
data[('Pagadores febrero 2023', '$ Pagado')] = data[('Pagadores febrero 2023', '$ Total')] - data[('Pagadores febrero 2023', '$ Impagado')] - data[('Pagadores febrero 2023', '$ Moroso')]

In [87]:
data[('Impagadores febrero 2023', '#')] = lst_n_cli_d
data[('Impagadores febrero 2023', '$ Pagado')] = 0
data[('Impagadores febrero 2023', '$ Impagado')] = lst_impa_cli_d
data[('Impagadores febrero 2023', '$ Impagado recuperado')] = lst_rec_impa_cli_d
data[('Impagadores febrero 2023', '$ Moroso')] = lst_mora_cli_d
data[('Impagadores febrero 2023', '$ Moroso recuperado')] = lst_rec_mora_cli_d
data[('Impagadores febrero 2023', '$ Total')] = lst_fact_cli_d

data[('Impagadores febrero 2023', '$ Impagado')] = data[('Impagadores febrero 2023', '$ Impagado')] - data[('Impagadores febrero 2023', '$ Moroso')]
data[('Impagadores febrero 2023', '$ Impagado recuperado')] = data[('Impagadores febrero 2023', '$ Impagado recuperado')] - data[('Impagadores febrero 2023', '$ Moroso recuperado')]
data[('Impagadores febrero 2023', '$ Pagado')] = data[('Impagadores febrero 2023', '$ Total')] - data[('Impagadores febrero 2023', '$ Impagado')] - data[('Impagadores febrero 2023', '$ Moroso')]

In [88]:
data[('Morosos febrero 2023', '#')] = lst_n_cli_m
data[('Morosos febrero 2023', '$ Pagado')] = 0
data[('Morosos febrero 2023', '$ Impagado')] = lst_impa_cli_m
data[('Morosos febrero 2023', '$ Impagado recuperado')] = lst_rec_impa_cli_m
data[('Morosos febrero 2023', '$ Moroso')] = lst_mora_cli_m
data[('Morosos febrero 2023', '$ Moroso recuperado')] = lst_rec_mora_cli_m
data[('Morosos febrero 2023', '$ Total')] = lst_fact_cli_m

data[('Morosos febrero 2023', '$ Impagado')] = data[('Morosos febrero 2023', '$ Impagado')] - data[('Morosos febrero 2023', '$ Moroso')]
data[('Morosos febrero 2023', '$ Impagado recuperado')] = data[('Morosos febrero 2023', '$ Impagado recuperado')] - data[('Morosos febrero 2023', '$ Moroso recuperado')]
data[('Morosos febrero 2023', '$ Pagado')] = data[('Morosos febrero 2023', '$ Total')] - data[('Morosos febrero 2023', '$ Impagado')] - data[('Morosos febrero 2023', '$ Moroso')]

In [89]:
business_df.columns

Index(['NoCredito', 'Mes Alta', 'Mes Baja', 'proba', 'Default_90_dias_cont',
       'Default_90_dias_pers', 'Tenure_mayo_23', 'Tenure_feb_23', 'Tenure_nov_22', 'pk',
       'di_fecha_ini', 'PERIOD', 'IMPORT', 'Impago', 'Importe_impagado', 'periodo_impagado',
       'IMPORTE_RECOBRADO', 'FECHA_RECOBRO', 'Importe_moroso', 'periodo_moroso',
       'IMPORTE_RECOBRADO_moroso', 'FECHA_RECOBRO_moroso', 'dias_en_default', 'Default_90_dias',
       'Meses facturacion filt', 'Importes facturacion filt', 'Meses Impago filt',
       'Meses Recuperación filt', 'Importes impago filt', 'Importes recuperacion filt',
       'Meses Impago +120 dias filt', 'Meses Recuperación +120 dias filt',
       'Importes impago +120 dias filt', 'Importes recuperacion +120 dias filt',
       'Importe facturacion filt', 'Cuotas facturacion filt', 'Importe impagado filt',
       'Importe recuperado filt', 'Cuotas impagadas filt', 'Importe impagado +120 dias filt',
       'Importe recuperado +120 dias filt', 'Cuotas imp

In [90]:
business_df.head(1)

,NoCredito,Mes Alta,Mes Baja,proba,Default_90_dias_cont,Default_90_dias_pers,Tenure_mayo_23,Tenure_feb_23,Tenure_nov_22,pk,di_fecha_ini,PERIOD,IMPORT,Impago,Importe_impagado,periodo_impagado,IMPORTE_RECOBRADO,FECHA_RECOBRO,Importe_moroso,periodo_moroso,IMPORTE_RECOBRADO_moroso,FECHA_RECOBRO_moroso,dias_en_default,Default_90_dias,Meses facturacion filt,Importes facturacion filt,Meses Impago filt,Meses Recuperación filt,Importes impago filt,Importes recuperacion filt,Meses Impago +120 dias filt,Meses Recuperación +120 dias filt,Importes impago +120 dias filt,Importes recuperacion +120 dias filt,Importe facturacion filt,Cuotas facturacion filt,Importe impagado filt,Importe recuperado filt,Cuotas impagadas filt,Importe impagado +120 dias filt,Importe recuperado +120 dias filt,Cuotas impagadas +120 dias filt,term,installment,principal,Meses facturacion filt post,Importes facturacion filt post,Meses Impago filt post,Meses Recuperación filt post,Importes impago filt post,Importes recuperacion filt post,Meses Impago +120 dias filt post,Meses Recuperación +120 dias filt post,Importes impago +120 dias filt post,Importes recuperacion +120 dias filt post,Importe facturacion filt post,Cuotas facturacion filt post,Importe impagado filt post,Importe recuperado filt post,Cuotas impagadas filt post,Importe impagado +120 dias filt post,Importe recuperado +120 dias filt post,Cuotas impagadas +120 dias filt post
0,238891837,2022-05-09,NaN,0.519277,1,1,384,295,203,"[2463, 2463, 2463, 2463, 2463, 2463, 2463, 246...","[2022-05-09, 2022-06-10, 2022-07-12, 2022-08-1...","[2022-06-09, 2022-07-11, 2022-08-09, 2022-09-0...","[2116.26, 2009.23, 2035.11, 1974.61, 1680.0, 1...","[SI, SI, SI, SI, SI, SI, SI, SI, SI]","[2116.26, 2009.23, 2035.11, 1974.61, 1680.0, 1...","[2022-06-09, 2022-07-11, 2022-08-09, 2022-09-0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[nan, nan, nan, nan, nan, nan, nan, nan, nan]","[2116.26, 2009.23, 2035.11, 1974.61, 1680.0, 1...","[2022-06-09, 2022-07-11, 2022-08-09, 2022-09-0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[nan, nan, nan, nan, nan, nan, nan, nan, nan]","[356, 324, 295, 264, 233, 203, 173, 142, 111]","[1, 1, 1, 1, 1, 1, 1, 1, 1]","[2022-06-09, 2022-07-11, 2022-08-09, 2022-09-0...","[2116.26, 2009.23, 2035.11, 1974.61, 1680.0, 1...","[2022-06-09, 2022-07-11, 2022-08-09, 2022-09-0...","[nan, nan, nan, nan, nan, nan]","[2116.26, 2009.23, 2035.11, 1974.61, 1680.0, 1...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[2022-06-09, 2022-07-11, 2022-08-09, 2022-09-0...","[nan, nan, nan, nan, nan, nan]","[2116.26, 2009.23, 2035.11, 1974.61, 1680.0, 1...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",11495.21,6,11495.21,0.0,6,11495.21,0.0,6,1,11495.21,11495.21,"[2022-06-09, 2022-07-11, 2022-08-09, 2022-09-0...","[2116.26, 2009.23, 2035.11, 1974.61, 1680.0, 1...","[2022-06-09, 2022-07-11, 2022-08-09, 2022-09-0...","[nan, nan, nan, nan, nan, nan, nan, nan, nan]","[2116.26, 2009.23, 2035.11, 1974.61, 1680.0, 1...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[2022-06-09, 2022-07-11, 2022-08-09, 2022-09-0...","[nan, nan, nan, nan, nan, nan, nan, nan, nan]","[2116.26, 2009.23, 2035.11, 1974.61, 1680.0, 1...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",16535.21,9,16535.21,0.0,9,16535.21,0.0,9


In [91]:
next_month='2022-11'
business_df[(business_df['Importe impagado filt']>0)&(business_df['Importe impagado filt post']>0)&(business_df['Tenure_nov_22']<=180)&
            (business_df['Importe impagado +120 dias filt post']<=0)&(business_df['Importe impagado +120 dias filt']<=0)&
           ((business_df['Mes Baja'].isnull())|(business_df['Mes Baja']>=next_month))]['Importe impagado filt post'].sum()-\
business_df[(business_df['Importe impagado filt']>0)&(business_df['Importe impagado filt post']>0)&(business_df['Tenure_nov_22']<=180)&
            (business_df['Importe impagado +120 dias filt post']<=0)&(business_df['Importe impagado +120 dias filt']<=0)&
           ((business_df['Mes Baja'].isnull())|(business_df['Mes Baja']>=next_month))]['Importe impagado filt'].sum()

7015571.229999999

In [92]:
next_month='2022-11'
business_df[(business_df['Importe impagado filt']>0)&(business_df['Importe impagado filt post']>-10000000000)&(business_df['Tenure_nov_22']>180)&
            (business_df['Importe impagado +120 dias filt post']>0)&(business_df['Importe impagado +120 dias filt']<=0)&
           ((business_df['Mes Baja'].isnull())|(business_df['Mes Baja']>=next_month))]['Importe impagado filt post'].sum()-\
business_df[(business_df['Importe impagado filt']>0)&(business_df['Importe impagado filt post']>-10000000000)&(business_df['Tenure_nov_22']>180)&
            (business_df['Importe impagado +120 dias filt post']>0)&(business_df['Importe impagado +120 dias filt']<=0)&
           ((business_df['Mes Baja'].isnull())|(business_df['Mes Baja']>=next_month))]['Importe impagado filt'].sum()

151871.44

In [93]:
next_month='2022-11'
business_df[(business_df['Importe impagado filt']>0)&(business_df['Importe impagado filt post']>-10000000000)&(business_df['Tenure_nov_22']<=180)&
            (business_df['Importe impagado +120 dias filt post']>0)&(business_df['Importe impagado +120 dias filt']>0)&
           ((business_df['Mes Baja'].isnull())|(business_df['Mes Baja']>=next_month))]['Importe impagado filt post'].sum()-\
business_df[(business_df['Importe impagado filt']>0)&(business_df['Importe impagado filt post']>-10000000000)&(business_df['Tenure_nov_22']<=180)&
            (business_df['Importe impagado +120 dias filt post']>0)&(business_df['Importe impagado +120 dias filt']>0)&
           ((business_df['Mes Baja'].isnull())|(business_df['Mes Baja']>=next_month))]['Importe impagado filt'].sum()

5734488.49

In [94]:
data

Enero2022                                                              \
                             #     $ Pagado   $ Impagado $ Impagado recuperado    $ Moroso   
Pagadores +6 meses          57    990124.07         0.00                  0.00        0.00   
Impagadores +6 meses        87   1201811.75    468471.83             468485.01        0.00   
Morosos +6 meses            51    255677.06    101425.55             101426.67   609225.10   
Pagadores -6 meses        6123  41617670.67         0.00                  0.00        0.00   
Impagadores -6 meses      2578  15661564.92  11588287.30           11588755.38        0.00   
Morosos -6 meses           453   1556229.32    912665.76             912678.24  3562686.78   

                                                                                     \
                     $ Moroso recuperado      $ Total PD impago   PD mora       LGD   
Pagadores +6 meses                  0.00    990124.07  0.000000  0.000000       NaN   
Impagadores +6 meses                0.00   1670283.58  0.280474  0.000000       NaN   
Morosos +6 meses                24222.74    966327.71  0.104960  0.630454  0.960240   
Pagadores -6 meses                  0.00  41617670.67  0.000000  0.000000       NaN   
Impagadores -6 meses                0.00  27249852.22  0.425261  0.000000       NaN   
Morosos -6 meses               211081.40   6031581.86  0.151314  0.590672  0.940752   

                     Pagadores febrero 2023                                                \
                                          #     $ Pagado $ Impagado $ Impagado recuperado   
Pagadores +6 meses                       45    420938.91        0.0                   0.0   
Impagadores +6 meses                      0         0.00        0.0                   0.0   
Morosos +6 meses                          0         0.00        0.0                   0.0   
Pagadores -6 meses                     3361  46174514.17        0.0                   0.0   
Impagadores -6 meses                      0         0.00        0.0                   0.0   
Morosos -6 meses                          0         0.00        0.0                   0.0   

                                                               Impagadores febrero 2023  \
                     $ Moroso $ Moroso recuperado      $ Total                        #   
Pagadores +6 meses        0.0                 0.0    420938.91                       11   
Impagadores +6 meses      0.0                 0.0         0.00                       71   
Morosos +6 meses          0.0                 0.0         0.00                        0   
Pagadores -6 meses        0.0                 0.0  46174514.17                     2439   
Impagadores -6 meses      0.0                 0.0         0.00                     1997   
Morosos -6 meses          0.0                 0.0         0.00                        0   

                                                                                                   \
                         $ Pagado   $ Impagado $ Impagado recuperado $ Moroso $ Moroso recuperado   
Pagadores +6 meses       51679.12     29612.14              29612.14      0.0                 0.0   
Impagadores +6 meses    541676.47    161818.74             161864.18      0.0                 0.0   
Morosos +6 meses             0.00         0.00                  0.00      0.0                 0.0   
Pagadores -6 meses    21109336.95  12099724.22           12099950.74      0.0                 0.0   
Impagadores -6 meses  19289399.35   7015571.23            7022661.61      0.0                 0.0   
Morosos -6 meses             0.00         0.00                  0.00      0.0                 0.0   

                                  Morosos febrero 2023                           \
                          $ Total                    #      $ Pagado $ Impagado   
Pagadores +6 meses       81291.26                    1 -3.637979e-12       0.00   
Impagadores +6 meses    703495.21      

In [95]:
business_df.columns

Index(['NoCredito', 'Mes Alta', 'Mes Baja', 'proba', 'Default_90_dias_cont',
       'Default_90_dias_pers', 'Tenure_mayo_23', 'Tenure_feb_23', 'Tenure_nov_22', 'pk',
       'di_fecha_ini', 'PERIOD', 'IMPORT', 'Impago', 'Importe_impagado', 'periodo_impagado',
       'IMPORTE_RECOBRADO', 'FECHA_RECOBRO', 'Importe_moroso', 'periodo_moroso',
       'IMPORTE_RECOBRADO_moroso', 'FECHA_RECOBRO_moroso', 'dias_en_default', 'Default_90_dias',
       'Meses facturacion filt', 'Importes facturacion filt', 'Meses Impago filt',
       'Meses Recuperación filt', 'Importes impago filt', 'Importes recuperacion filt',
       'Meses Impago +120 dias filt', 'Meses Recuperación +120 dias filt',
       'Importes impago +120 dias filt', 'Importes recuperacion +120 dias filt',
       'Importe facturacion filt', 'Cuotas facturacion filt', 'Importe impagado filt',
       'Importe recuperado filt', 'Cuotas impagadas filt', 'Importe impagado +120 dias filt',
       'Importe recuperado +120 dias filt', 'Cuotas imp

In [96]:
data.to_csv('gs://hocelot_mx_analytics/mexico/output/mx-00007-Came/matriz_cambio_de_estado.csv',sep=';',index=False)
data

Enero2022                                                              \
                             #     $ Pagado   $ Impagado $ Impagado recuperado    $ Moroso   
Pagadores +6 meses          57    990124.07         0.00                  0.00        0.00   
Impagadores +6 meses        87   1201811.75    468471.83             468485.01        0.00   
Morosos +6 meses            51    255677.06    101425.55             101426.67   609225.10   
Pagadores -6 meses        6123  41617670.67         0.00                  0.00        0.00   
Impagadores -6 meses      2578  15661564.92  11588287.30           11588755.38        0.00   
Morosos -6 meses           453   1556229.32    912665.76             912678.24  3562686.78   

                                                                                     \
                     $ Moroso recuperado      $ Total PD impago   PD mora       LGD   
Pagadores +6 meses                  0.00    990124.07  0.000000  0.000000       NaN   
Impagadores +6 meses                0.00   1670283.58  0.280474  0.000000       NaN   
Morosos +6 meses                24222.74    966327.71  0.104960  0.630454  0.960240   
Pagadores -6 meses                  0.00  41617670.67  0.000000  0.000000       NaN   
Impagadores -6 meses                0.00  27249852.22  0.425261  0.000000       NaN   
Morosos -6 meses               211081.40   6031581.86  0.151314  0.590672  0.940752   

                     Pagadores febrero 2023                                                \
                                          #     $ Pagado $ Impagado $ Impagado recuperado   
Pagadores +6 meses                       45    420938.91        0.0                   0.0   
Impagadores +6 meses                      0         0.00        0.0                   0.0   
Morosos +6 meses                          0         0.00        0.0                   0.0   
Pagadores -6 meses                     3361  46174514.17        0.0                   0.0   
Impagadores -6 meses                      0         0.00        0.0                   0.0   
Morosos -6 meses                          0         0.00        0.0                   0.0   

                                                               Impagadores febrero 2023  \
                     $ Moroso $ Moroso recuperado      $ Total                        #   
Pagadores +6 meses        0.0                 0.0    420938.91                       11   
Impagadores +6 meses      0.0                 0.0         0.00                       71   
Morosos +6 meses          0.0                 0.0         0.00                        0   
Pagadores -6 meses        0.0                 0.0  46174514.17                     2439   
Impagadores -6 meses      0.0                 0.0         0.00                     1997   
Morosos -6 meses          0.0                 0.0         0.00                        0   

                                                                                                   \
                         $ Pagado   $ Impagado $ Impagado recuperado $ Moroso $ Moroso recuperado   
Pagadores +6 meses       51679.12     29612.14              29612.14      0.0                 0.0   
Impagadores +6 meses    541676.47    161818.74             161864.18      0.0                 0.0   
Morosos +6 meses             0.00         0.00                  0.00      0.0                 0.0   
Pagadores -6 meses    21109336.95  12099724.22           12099950.74      0.0                 0.0   
Impagadores -6 meses  19289399.35   7015571.23            7022661.61      0.0                 0.0   
Morosos -6 meses             0.00         0.00                  0.00      0.0                 0.0   

                                  Morosos febrero 2023                           \
                          $ Total                    #      $ Pagado $ Impagado   
Pagadores +6 meses       81291.26                    1 -3.637979e-12       0.00   
Impagadores +6 meses    703495.21      

In [97]:
business_df

,NoCredito,Mes Alta,Mes Baja,proba,Default_90_dias_cont,Default_90_dias_pers,Tenure_mayo_23,Tenure_feb_23,Tenure_nov_22,pk,di_fecha_ini,PERIOD,IMPORT,Impago,Importe_impagado,periodo_impagado,IMPORTE_RECOBRADO,FECHA_RECOBRO,Importe_moroso,periodo_moroso,IMPORTE_RECOBRADO_moroso,FECHA_RECOBRO_moroso,dias_en_default,Default_90_dias,Meses facturacion filt,Importes facturacion filt,Meses Impago filt,Meses Recuperación filt,Importes impago filt,Importes recuperacion filt,Meses Impago +120 dias filt,Meses Recuperación +120 dias filt,Importes impago +120 dias filt,Importes recuperacion +120 dias filt,Importe facturacion filt,Cuotas facturacion filt,Importe impagado filt,Importe recuperado filt,Cuotas impagadas filt,Importe impagado +120 dias filt,Importe recuperado +120 dias filt,Cuotas impagadas +120 dias filt,term,installment,principal,Meses facturacion filt post,Importes facturacion filt post,Meses Impago filt post,Meses Recuperación filt post,Importes impago filt post,Importes recuperacion filt post,Meses Impago +120 dias filt post,Meses Recuperación +120 dias filt post,Importes impago +120 dias filt post,Importes recuperacion +120 dias filt post,Importe facturacion filt post,Cuotas facturacion filt post,Importe impagado filt post,Importe recuperado filt post,Cuotas impagadas filt post,Importe impagado +120 dias filt post,Importe recuperado +120 dias filt post,Cuotas impagadas +120 dias filt post
0,238891837,2022-05-09,NaN,0.519277,1,1,384,295,203,"[2463, 2463, 2463, 2463, 2463, 2463, 2463, 246...","[2022-05-09, 2022-06-10, 2022-07-12, 2022-08-1...","[2022-06-09, 2022-07-11, 2022-08-09, 2022-09-0...","[2116.26, 2009.23, 2035.11, 1974.61, 1680.0, 1...","[SI, SI, SI, SI, SI, SI, SI, SI, SI]","[2116.26, 2009.23, 2035.11, 1974.61, 1680.0, 1...","[2022-06-09, 2022-07-11, 2022-08-09, 2022-09-0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[nan, nan, nan, nan, nan, nan, nan, nan, nan]","[2116.26, 2009.23, 2035.11, 1974.61, 1680.0, 1...","[2022-06-09, 2022-07-11, 2022-08-09, 2022-09-0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[nan, nan, nan, nan, nan, nan, nan, nan, nan]","[356, 324, 295, 264, 233, 203, 173, 142, 111]","[1, 1, 1, 1, 1, 1, 1, 1, 1]","[2022-06-09, 2022-07-11, 2022-08-09, 2022-09-0...","[2116.26, 2009.23, 2035.11, 1974.61, 1680.0, 1...","[2022-06-09, 2022-07-11, 2022-08-09, 2022-09-0...","[nan, nan, nan, nan, nan, nan]","[2116.26, 2009.23, 2035.11, 1974.61, 1680.0, 1...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[2022-06-09, 2022-07-11, 2022-08-09, 2022-09-0...","[nan, nan, nan, nan, nan, nan]","[2116.26, 2009.23, 2035.11, 1974.61, 1680.0, 1...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",11495.21,6,11495.21,0.00,6,11495.21,0.0,6,1,11495.21,11495.21,"[2022-06-09, 2022-07-11, 2022-08-09, 2022-09-0...","[2116.26, 2009.23, 2035.11, 1974.61, 1680.0, 1...","[2022-06-09, 2022-07-11, 2022-08-09, 2022-09-0...","[nan, nan, nan, nan, nan, nan, nan, nan, nan]","[2116.26, 2009.23, 2035.11, 1974.61, 1680.0, 1...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[2022-06-09, 2022-07-11, 2022-08-09, 2022-09-0...","[nan, nan, nan, nan, nan, nan, nan, nan, nan]","[2116.26, 2009.23, 2035.11, 1974.61, 1680.0, 1...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",16535.21,9,16535.21,0.00,9,16535.21,0.0,9
1,214756696,2022-10-27,NaN,0.483736,0,0,213,124,32,"[11575, 11575, 11575, 11575]","[2022-10-27, 2022-11-29, 2022-12-28, 2023-01-28]","[2022-11-28, 2022-12-27, 2023-01-27, 2023-02-27]","[3296.0, 3298.96, 3296.0, 3296.0]","[SI, NO, NO, NO]","[3292.0, 0.0, 0.0, 0.0]","[2022-11-28, , , ]","[3292.0, 0.0, 0.0, 0.0]","[2022-11-29, nan, nan, nan]","[0.0, 0.0, 0.0, 0.0]","[, , , ]","[0.0, 0.0, 0.0, 0.0]","[, , , ]","[1, 0, 0, 0]","[0, 0, 0, 0]",[2022-11-28],[3296.0],[2022-11-28],[2022-11-29],[3292.0],[3292.0],[],[],[],[],3296.00,1,3292.00,3292.00,1,0.00,0.0,0,1,0.00,0.00,"[2022-11-28, 2022-12-27, 2023-01-27, 2023-02-27]","[3296.0, 3298.96, 3296.0, 3296.0]",[2022-11-28],[2022-11-29],[3292.0],[3292.0],[],[],[],[],13186.96,4,3292.00,3292.00,1,0.00,0.0,0
2,8

In [98]:
business_df_p=business_df[(business_df['Tenure_nov_22']<180)]
business_df_p.shape

(9287, 63)

In [99]:
business_df.shape

(9503, 63)

In [100]:
business_df_p = business_df_p[business_df_p['proba'].notnull()].copy()

In [101]:
business_df_p.shape

(9287, 63)

### Menores de 6 meses de antiguedad a inicio del postperiodo

In [103]:
business_df_admitido = business_df_p[business_df_p['proba']<business_df_p['proba'].quantile(0.40)].copy()
business_df_admitido['Importe_12m_admitido'] = business_df_admitido['Importe facturacion filt']
business_df_admitido['Importe_12m_moroso_admitido'] = business_df_admitido['Importe impagado +120 dias filt']
a = business_df_admitido['Importe_12m_admitido'].sum()
b = business_df_admitido['Importe_12m_moroso_admitido'].sum()
print(a)
print(b)
print(100*b/a)

32972644.69
710100.09
2.1536036817069766


In [104]:
business_df_admitido = business_df_p[business_df_p['proba']<business_df_p['proba'].quantile(0.50)].copy()
business_df_admitido['Importe_12m_admitido'] = business_df_admitido['Importe facturacion filt']
business_df_admitido['Importe_12m_moroso_admitido'] = business_df_admitido['Importe impagado +120 dias filt']
a = business_df_admitido['Importe_12m_admitido'].sum()
b = business_df_admitido['Importe_12m_moroso_admitido'].sum()
print(a)
print(b)
print(100*b/a)

40329892.39
1091644.45
2.7067874107957666


In [105]:
business_df_admitido = business_df_p[business_df_p['proba']<business_df_p['proba'].quantile(0.60)].copy()
business_df_admitido['Importe_12m_admitido'] = business_df_admitido['Importe facturacion filt']
business_df_admitido['Importe_12m_moroso_admitido'] = business_df_admitido['Importe impagado +120 dias filt']
a = business_df_admitido['Importe_12m_admitido'].sum()
b = business_df_admitido['Importe_12m_moroso_admitido'].sum()
print(a)
print(b)
print(100*b/a)

47588433.2
1378861.71
2.897472384108666


In [106]:
business_df_admitido = business_df_p[business_df_p['proba']<business_df_p['proba'].quantile(0.2308)].copy()
business_df_admitido['Importe_12m_admitido'] = business_df_admitido['Importe facturacion filt']
business_df_admitido['Importe_12m_moroso_admitido'] = business_df_admitido['Importe impagado +120 dias filt']
a = business_df_admitido['Importe_12m_admitido'].sum()
b = business_df_admitido['Importe_12m_moroso_admitido'].sum()
print(a)
print(b)
print(100*b/a)

19961985.509999998
298667.67000000004
1.4961821801262296


In [107]:
business_df_admitido = business_df_p[business_df_p['proba']<business_df_p['proba'].quantile(0.5122)].copy()
business_df_admitido['Importe_12m_admitido'] = business_df_admitido['Importe facturacion filt']
business_df_admitido['Importe_12m_moroso_admitido'] = business_df_admitido['Importe impagado +120 dias filt']
a = business_df_admitido['Importe_12m_admitido'].sum()
b = business_df_admitido['Importe_12m_moroso_admitido'].sum()
print(a)
print(b)
print(100*b/a)

41141954.36
1106449.65
2.689346355105917


In [108]:
business_df_admitido = business_df_p[business_df_p['proba']<business_df_p['proba'].quantile(0.6513)].copy()
business_df_admitido['Importe_12m_admitido'] = business_df_admitido['Importe facturacion filt']
business_df_admitido['Importe_12m_moroso_admitido'] = business_df_admitido['Importe impagado +120 dias filt']
a = business_df_admitido['Importe_12m_admitido'].sum()
b = business_df_admitido['Importe_12m_moroso_admitido'].sum()
print(a)
print(b)
print(100*b/a)

51551040.57
1553694.98
3.0138964467463514


In [108]:
business_df_admitido = business_df_p[business_df_p['proba']<business_df_p['proba'].quantile(1)].copy()
business_df_admitido['Importe_12m_admitido'] = business_df_admitido['Importe facturacion filt']
business_df_admitido['Importe_12m_moroso_admitido'] = business_df_admitido['Importe impagado +120 dias filt']
a = business_df_admitido['Importe_12m_admitido'].sum()
b = business_df_admitido['Importe_12m_moroso_admitido'].sum()
print(a)
print(b)
print(100*b/a)

83146038.31
2848202.5799999996
3.4255421399403527


### Total Cartera

In [116]:
business_df_admitido = business_df[business_df['proba']<business_df['proba'].quantile(0.3)].copy()
business_df_admitido['Importe_12m_admitido'] = business_df_admitido['Importe facturacion filt']
business_df_admitido['Importe_12m_moroso_admitido'] = business_df_admitido['Importe impagado +120 dias filt']
a = business_df_admitido['Importe_12m_admitido'].sum()
b = business_df_admitido['Importe_12m_moroso_admitido'].sum()
print(a)
print(b)
print(100*b/a)

294829637.78
12371551.54
4.196169568688876


In [117]:
business_df_admitido = business_df[business_df['proba']<business_df['proba'].quantile(0.4)].copy()
business_df_admitido['Importe_12m_admitido'] = business_df_admitido['Importe facturacion filt']
business_df_admitido['Importe_12m_moroso_admitido'] = business_df_admitido['Importe impagado +120 dias filt']
a = business_df_admitido['Importe_12m_admitido'].sum()
b = business_df_admitido['Importe_12m_moroso_admitido'].sum()
print(a)
print(b)
print(100*b/a)

391673883.74
19057319.81
4.865609018407411


In [118]:
business_df_admitido = business_df[business_df['proba']<business_df['proba'].quantile(0.5)].copy()
business_df_admitido['Importe_12m_admitido'] = business_df_admitido['Importe facturacion filt']
business_df_admitido['Importe_12m_moroso_admitido'] = business_df_admitido['Importe impagado +120 dias filt']
a = business_df_admitido['Importe_12m_admitido'].sum()
b = business_df_admitido['Importe_12m_moroso_admitido'].sum()
print(a)
print(b)
print(100*b/a)

488567165.15
27093552.639999997
5.545512382454463


In [110]:
business_df_admitido = business_df[business_df['proba']<business_df['proba'].quantile(0.6)].copy()
business_df_admitido['Importe_12m_admitido'] = business_df_admitido['Importe facturacion filt']
business_df_admitido['Importe_12m_moroso_admitido'] = business_df_admitido['Importe impagado +120 dias filt']
a = business_df_admitido['Importe_12m_admitido'].sum()
b = business_df_admitido['Importe_12m_moroso_admitido'].sum()
print(a)
print(b)
print(100*b/a)

579224159.5300001
35066472.28999999
6.054041723407045


In [111]:
business_df_admitido = business_df[business_df['proba']<business_df['proba'].quantile(0.7)].copy()
business_df_admitido['Importe_12m_admitido'] = business_df_admitido['Importe facturacion filt']
business_df_admitido['Importe_12m_moroso_admitido'] = business_df_admitido['Importe impagado +120 dias filt']
a = business_df_admitido['Importe_12m_admitido'].sum()
b = business_df_admitido['Importe_12m_moroso_admitido'].sum()
print(a)
print(b)
print(100*b/a)

667570620.9200001
44181258.10000001
6.618214869778486


In [112]:
business_df_admitido = business_df[business_df['proba']<business_df['proba'].quantile(1)].copy()
business_df_admitido['Importe_12m_admitido'] = business_df_admitido['Importe facturacion filt']
business_df_admitido['Importe_12m_moroso_admitido'] = business_df_admitido['Importe impagado +120 dias filt']
a = business_df_admitido['Importe_12m_admitido'].sum()
b = business_df_admitido['Importe_12m_moroso_admitido'].sum()
print(a)
print(b)
print(100*b/a)

946597378.52
96440157.16999999
10.188086229520692
